Description:
This script loads and transforms data from the RDV layer to the BDV layer for RSS and Competitor (Comps) analysis. The process runs across multiple stages:

Part 1: Extracts RSS and competitor voyage data, and splits it into voyage-day level detail based on itinerary and port conflicts for pre-scoring.

Pre-Scoring: Assigns trades to voyages using defined business parameters.

Scoring: Finalizes trade assignment for voyages.

Part 2: Creates RSS and competitor-specific extracts, applies business transformations, and generates BDV tables (metadata, pricing, and latest pricing).

Like-for-Like: Identifies competitor voyages corresponding to RSS voyages and assigns their comparative pricing.


Author: Rakesh J (Mu-Sigma)
Date: 2025-08-26

Updates: None
 

In [ ]:
import snowflake.snowpark as snowpark
 
connection_parameters = {
    'user': '',
    'password': '',
    'account': 'yvb85669.us-east-1',
    'warehouse': 'WH_PG200',
    'role': 'SYSADMINDEV',
    'database': 'VESSOPS_D',
    'schema': 'L10_RDV'
}
session = snowpark.Session.builder.configs(connection_parameters).create()

## Incremental_Data_Weekly_Cadence_Prev_Monday_Filter

In [ ]:
SET filter_date='2026-08-18':: DATE

## RSS_PART_1 CODE

In [ ]:
-- USE database VESSOPS_D;
-- USE SCHEMA L10_RDV;


-- Create a temporary table
CREATE OR REPLACE TEMP TABLE TEMP_U31 AS
SELECT DISTINCT 
    CruiseLineCode,
    ShipCode,
    CruiseLineName,
    SourceShipName,
    CruiseDuration,
    DisembarkDate AS DisembarkationDate,
    EmbarkationDate,
    EmbarkationPort,
    EmbarkationPortUN_LOCODE,
    DisembarkationPort,
    DisembarkationPortUN_LOCODE,
    ItineraryPortContent,
    FilePath
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Pricing_RG_Pass
WHERE TRY_TO_DATE(SUBSTRING(FilePath, 42, 10), 'YYYY-MM-DD') IS NOT NULL  -- Ensure valid dates only
  AND TRY_TO_DATE(SUBSTRING(FilePath, 42, 10), 'YYYY-MM-DD') > $filter_date  -- Ensure Date > Feb 24, 2025
  AND DAYOFWEEK(TRY_TO_DATE(SUBSTRING(FilePath, 42, 10), 'YYYY-MM-DD')) = 1  -- Ensure Monday
  -- RateType condition: handles variations like "Double", "Doublen" etc.
  AND UPPER(TRIM(RateType)) = 'DOUBLE'
  -- CruiseLineName condition: case-insensitive, trimmed
  AND UPPER(TRIM(CruiseLineName)) IN (
      'REGENT SEVEN SEAS', 
      'SILVERSEA CRUISES', 
      'SEABOURN CRUISES', 
      'CRYSTAL CRUISES', 
      'RITZCARLTON', 
      'PONANT CRUISE LINE', 
      'HAPAG-LLOYD CRUISES', 
      'VIKING OCEAN CRUISES', 
      'EXPLORAJOURNEY'
  );


--select * from SAT_GCI_NCH_RAW_PRICING_RG_PASS where ITINERARYPORTCONTENT ilike '%ST JONSFJORDEN%'


--select count(*) from TEMP_U31--8965(03)--968747(ALL)--784924
-- Drop the temp table if it exists
--select top 10 * from VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Pricing_RG_Pass

-- Create a new temporary table
CREATE OR REPLACE TEMP TABLE TEMP_U32 AS
SELECT 
    CruiseLineCode,
    ShipCode,
    CruiseLineName,
    SourceShipName,
    CruiseDuration,
    DisembarkationDate,
    EmbarkationDate,
    EmbarkationPort,
    EmbarkationPortUN_LOCODE,
    DisembarkationPort,
    DisembarkationPortUN_LOCODE,
    REPLACE(ItineraryPortContent, '@$#', '+') AS ItineraryPortContent,
    FilePath
FROM TEMP_U31;


--select * from TEMP_U32
-- Step 1: Split ItineraryPortContent into rows using LATERAL FLATTEN

CREATE OR REPLACE TEMP TABLE TEMP_U33 AS
SELECT 
    t2.*,
    BK.value AS ItineraryPortSplit
FROM TEMP_U32 t2,
LATERAL FLATTEN(input => SPLIT(t2.ItineraryPortContent, '+')) AS BK;

--select * from TEMP_U33

-- Step 2: Replace '@$' with '+' for date separation
CREATE OR REPLACE TEMP TABLE TEMP_U34 AS
SELECT *,
    REPLACE(ItineraryPortSplit, '@$', '+') AS ItinPort_ItinDate
FROM TEMP_U33;

-- Step 3: Replace '.' with '>' to handle port location name issues
CREATE OR REPLACE TEMP TABLE TEMP_U35 AS
SELECT *,
    REPLACE(ItinPort_ItinDate, '.', '>') AS stk4
FROM TEMP_U34;

--select * from U35
-- Step 4: Extract Port City Name and Itinerary Date
CREATE OR REPLACE TEMP TABLE TEMP_U36 AS
SELECT 
    CruiseLineCode,
    ShipCode,
    CruiseLineName,
    SourceShipName,
    CruiseDuration,
    DisembarkationDate,
    EmbarkationDate,
    EmbarkationPort,
    EmbarkationPortUN_LOCODE,
    DisembarkationPort,
    DisembarkationPortUN_LOCODE,
    ItineraryPortContent,
    FilePath,
    SPLIT_PART(stk4, '+', 1) AS Port_City_Nam,
    SPLIT_PART(stk4, '+', 2) AS Itinerary_Date
FROM TEMP_U35
ORDER BY Itinerary_Date;


-- Step 5: Replace '>' back to '.' in Port_City_Name
CREATE OR REPLACE TEMP TABLE TEMP_U37 AS
SELECT 
    CruiseLineCode,
    ShipCode,
    CruiseLineName,
    SourceShipName,
    CruiseDuration,
    DisembarkationDate,
    EmbarkationDate,
    EmbarkationPort,
    EmbarkationPortUN_LOCODE,
    DisembarkationPort,
    DisembarkationPortUN_LOCODE,
    ItineraryPortContent,
    FilePath,
    REPLACE(Port_City_Nam, '>', '.') AS Port_City_Nam,
    Itinerary_Date
FROM TEMP_U36;

--select count(*) from temp_U37--127473--11531086


DELETE FROM TEMP_U37
WHERE TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY') 
      NOT BETWEEN CAST(EmbarkationDate AS DATE) 
          AND CAST(DisembarkationDate AS DATE);



-- --select * from SAT_GCI_NCH_RAW_ref_merged_port_data where tb_unv_port_code='CAVAN'
-- SELECT DISTINCT 
--     a.EmbarkationPort,
--     a.EmbarkationPortUN_LOCODE,
--     a.DisembarkationPort,
--     b.*,
--     c.*
-- FROM TEMP_U37 a
-- LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Ref_Merged_Port_data b
--     ON UPPER(REGEXP_REPLACE(TRIM(a.EmbarkationPort), '\\s+', ' ')) = UPPER(REGEXP_REPLACE(TRIM(b.Dtl_Port_City_Nam), '\\s+', ' '))
--     AND UPPER(TRIM(a.EmbarkationPortUN_LOCODE)) = UPPER(TRIM(b.Dtl_PortUN_Locode))
-- LEFT JOIN SAT_GCI_NCH_RAW_Ref_Merged_Port_data c
--     ON UPPER(REGEXP_REPLACE(TRIM(a.DisembarkationPort), '\\s+', ' ')) = UPPER(REGEXP_REPLACE(TRIM(c.Dtl_Port_City_Nam), '\\s+', ' '))
--     AND UPPER(TRIM(a.DisembarkationPortUN_LOCODE)) = UPPER(TRIM(c.Dtl_PortUN_Locode))
-- WHERE b.Tb_Port_Loc_Name IS NULL 
--     OR b.Tb_Unv_Port_Code IS NULL 
--     OR b.Tb_Country_Name IS NULL 
--     OR c.Tb_Port_Loc_Name IS NULL 
--     OR c.Tb_Unv_Port_Code IS NULL 
--     OR c.Tb_Country_Name IS NULL;

    
-- --select count(*) from temp_u37--11531086

-- select distinct port_city_nam
-- from TEMP_U37 a 
-- left join VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_ref_merged_port_data b 
-- ON UPPER(TRIM(a.Port_City_Nam)) = UPPER(TRIM(b.dtl_port_city_nam))
-- WHERE b.Tb_Port_Loc_Name IS NULL 
--     OR b.Tb_Unv_Port_Code IS NULL 
--     OR b.Tb_Country_Name IS NULL --305--81

    

-- select * from temp_u38  where cruiselinecode ='OCE' and shipcode='VS' and embarkationdate='2025-04-19 00:00:00.000' and disembarkationdate='2025-05-13 00:00:00.000'
-- port_city_nam is null--105125--7654


CREATE OR REPLACE TEMP TABLE TEMP_U38 AS
SELECT 
    mbs.MP_CruiseLineCode AS CruiseLineCode,
    mbs.MP_ShipCode AS ShipCode,
    mbs.MP_CruiseLineName AS CruiseLineName,
    mbs.MP_SourceShipName AS SourceShipName,
    ps.CruiseDuration,
    ps.DisembarkationDate,
    ps.EmbarkationDate,
    nd.Tb_Port_Loc_Name AS EmbarkationPort,
    nd.Tb_Unv_Port_Code AS EmbarkationPortUN_LOCODE,
    nd1.Tb_Port_Loc_Name AS DisembarkationPort,
    nd1.Tb_Unv_Port_Code AS DisembarkationPortUN_LOCODE,
    ps.ItineraryPortContent,
    nd3.Tb_Port_Loc_Name AS Port_City_Nam,
    nd3.Tb_Unv_Port_Code AS PortUN_LOCODE,
    nd3.Tb_Country_Name AS Port_Country_Nam,
    ps.FilePath,
    ps.Itinerary_Date
FROM TEMP_U37 ps
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Ref_Merged_Port_data nd
    ON UPPER(TRIM(ps.EmbarkationPortUN_LOCODE)) = UPPER(TRIM(nd.Dtl_PortUN_Locode))
    AND UPPER(REGEXP_REPLACE(TRIM(ps.EmbarkationPort), '\\s+', ' ')) = UPPER(REGEXP_REPLACE(TRIM(nd.Dtl_Port_City_Nam), '\\s+', ' '))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Ref_Merged_Port_data nd1
    ON UPPER(TRIM(ps.DisembarkationPortUN_LOCODE)) = UPPER(TRIM(nd1.Dtl_PortUN_Locode))
    AND UPPER(REGEXP_REPLACE(TRIM(ps.DisembarkationPort), '\\s+', ' ')) = UPPER(REGEXP_REPLACE(TRIM(nd1.Dtl_Port_City_Nam), '\\s+', ' '))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Ref_Merged_Ship_data mbs
    ON UPPER(TRIM(ps.CruiseLineName)) = UPPER(TRIM(mbs.CruiseLineName))
    AND UPPER(TRIM(ps.CruiseLineCode)) = UPPER(TRIM(mbs.CruiseLineCode))
    AND UPPER(TRIM(ps.SourceShipName)) = UPPER(TRIM(mbs.SourceShipName))
    AND UPPER(TRIM(ps.ShipCode)) = UPPER(TRIM(mbs.ShipCode))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA  nd3
    ON UPPER(TRIM(ps.Port_City_Nam)) = UPPER(TRIM(nd3.dtl_port_city_nam));




--TRUNCATE table VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_VOY_DAY_LEVEL_DATA
INSERT INTO VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_VOY_DAY_LEVEL_DATA
WITH src AS (
  SELECT
    u.*,
    CONCAT(
      CruiseLineCode, '-',
      ShipCode, '-',
      CAST(EmbarkationDate AS DATE), '-',
      EmbarkationPortUN_LOCODE, '-',
      DisembarkationPortUN_LOCODE, '-',
      CruiseDuration
    ) AS NCLH_UNI_VOYAGE_CODE
  FROM TEMP_U38 u
)
SELECT
  s.*,
  /* metadata */
  CURRENT_TIMESTAMP AS LDTS,
  CURRENT_USER      AS LAST_MODIFIED_BY,
  'RG_PRICNG_DATA' AS RCSR,

  /* hash diff (stable formats for dates/numbers) */
  TO_VARCHAR(
    MD5(
      CONCAT_WS(
        '|',
        COALESCE(s.CruiseLineCode, ''),
        COALESCE(s.ShipCode, ''),
        COALESCE(TO_VARCHAR(s.EmbarkationDate, 'YYYY-MM-DD"T"HH24:MI:SS.FF9'), ''),
        COALESCE(s.EmbarkationPortUN_LOCODE, ''),
        COALESCE(s.DisembarkationPortUN_LOCODE, ''),
        COALESCE(TO_VARCHAR(s.CruiseDuration), ''),
        COALESCE(s.NCLH_UNI_VOYAGE_CODE, '')
      )
    )
  ) AS HASH_DIFF
FROM src s;


--deleting null values if any
DELETE
FROM VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_VOY_DAY_LEVEL_DATA
WHERE CruiseLineCode IS NULL OR ShipCode IS NULL OR CruiseLineName IS NULL OR SourceShipName IS NULL OR CruiseDuration IS NULL OR DisembarkationDate IS NULL OR EmbarkationDate IS NULL OR EmbarkationPort IS NULL OR EmbarkationPortUN_LOCODE IS NULL OR DisembarkationPort IS NULL OR DisembarkationPortUN_LOCODE IS NULL OR ItineraryPortContent IS NULL OR Port_City_Nam IS NULL OR PortUN_LOCODE IS NULL OR Port_Country_Nam IS NULL OR FilePath IS NULL OR Itinerary_Date IS NULL OR NCLH_UNI_VOYAGE_CODE IS NULL;


    
-- /* 1) Rebuild deduped table */
-- CREATE OR REPLACE TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA AS 
-- SELECT *
-- FROM (
--   SELECT
--     d.*,
--     RANK() OVER (PARTITION BY d.NCLH_UNI_VOYAGE_CODE ORDER BY d.FilePath DESC) AS ranker1
--   FROM VESSOPS_D.L10_RDV.GCI_INTER_PRICING_RSS_VOY_DAY_LEVEL_DATA d
-- )
-- WHERE ranker1 = 1;

-- /* 2) Add metadata/extra columns */
-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS VESSEL_IMO VARCHAR(50);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS NCLH_SHIP_CODE VARCHAR(5);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS SAIL_ID VARCHAR(50);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS MD5_VOYAGE_HASH_KEY VARCHAR;

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS HASH_DIFF VARCHAR;

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS LDTS TIMESTAMP_NTZ(9);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS RCSR VARCHAR(200);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
--   ADD COLUMN IF NOT EXISTS LAST_MODIFIED_BY VARCHAR;
--select top 10 * from VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_VOY_DAY_LEVEL_DATA

Create or replace temp table TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA as
SELECT *
FROM (
  SELECT
    d.*,
    RANK() OVER (PARTITION BY d.NCLH_UNI_VOYAGE_CODE ORDER BY d.FilePath DESC) AS ranker1
  FROM VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_VOY_DAY_LEVEL_DATA d
)
WHERE ranker1 = 1;



-- select top 10 * from VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA

--select top 10 * from TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA

alter table TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
drop column hash_diff;


ALTER TABLE TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
ADD COLUMN 
    VESSEL_IMO VARCHAR(50),
    NCLH_SHIP_CODE VARCHAR(5),
    SAIL_ID varchar(50),
    MD5_VOYAGE_HASH_KEY VARCHAR,
    HASH_DIFF VARCHAR;

-- alter table VESSOPS_D.L00_STG.GCI_PRICING_CHECK_AK_1 
-- drop column SAIL_ID,MD5_VOYAGE_HASH_KEY,HASH_DIFF,LDTS,RCSR,LAST_MODIFIED_BY,VESSEL_IMO,NCLH_SHIP_CODE


UPDATE TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA AS a
SET VESSEL_IMO = b.VESSEL_IMO_NO, NCLH_SHIP_CODE= b.ship_code_3l
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_ref_ship_data AS b
WHERE CONCAT(a.CRUISELINECODE, a.SHIPCODE) = b."5L_SHIP_CODE";


UPDATE TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
SET SAIL_ID = CONCAT(
    NCLH_SHIP_CODE,
    TO_CHAR(EMBARKATIONDATE, 'YYMMDD')
);

UPDATE TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
SET MD5_VOYAGE_HASH_KEY = MD5(SAIL_ID);

UPDATE TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
SET HASH_DIFF = TO_VARCHAR(
  MD5(
    CONCAT_WS(
      '|',
      COALESCE(CruiseLineCode, ''),
      COALESCE(ShipCode, ''),
      COALESCE(TO_VARCHAR(EmbarkationDate, 'YYYY-MM-DD"T"HH24:MI:SS.FF9'), ''),
      COALESCE(EmbarkationPortUN_LOCODE, ''),
      COALESCE(DisembarkationPortUN_LOCODE, ''),
      COALESCE(TO_VARCHAR(CruiseDuration), ''),
      COALESCE(NCLH_UNI_VOYAGE_CODE, ''),
      COALESCE(TO_VARCHAR(VESSEL_IMO), ''),
      COALESCE(NCLH_SHIP_CODE, ''),
      COALESCE(SAIL_ID, '')
    )
  )
);


-- alter table VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA
-- rename to VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_PRICING_VOY_DAY_LEVEL_DATA;

/* 1) Rebuild deduped table */
--CREATE OR REPLACE TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA as 
TRUNCATE TABLE VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_PRICING_VOY_DAY_LEVEL_DATA;
insert into VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_PRICING_VOY_DAY_LEVEL_DATA
select * from TEMP_GCI_INTER_REF_PRICING_RSS_VOY_DAY_LEVEL_DATA

--select count(*) from VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_PRICING_VOY_DAY_LEVEL_DATA;    --344764






## Pre-Scoring_Trade_Assignment_Flag

Process to assign trade flags for RSS and competitor voyages prior to scoring.

In [ ]:


CREATE OR REPLACE TEMPORARY TABLE DTL_TBPORT AS
SELECT
    DTL.*,
    TB_PORT.PORT_LOC_NAME,
    TB_PORT.CONTINENT,
    TB_PORT.COUNTRY_NAME,
    TB_PORT.PORT_LAT,
    TB_PORT.PORT_LONG,
    TB_PORT.NCLH_RM_TRADE1,
    TB_PORT.NCLH_RM_TRADE2,
    TB_PORT.NCLH_RM_TRADE3 AS TRADE_3_MS,
    TB_PORT.NCLH_RM_TRADE4 AS TRADE_4_MS,
    TB_PORT.UNV_PORT_CODE
FROM 
    VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_PRICING_VOY_DAY_LEVEL_DATA AS DTL
JOIN 
   VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA  AS TB_PORT
    ON UPPER(TRIM(DTL.PortUN_LOCODE)) = UPPER(TRIM(TB_PORT.UNV_PORT_CODE))
WHERE 
    UPPER(TRIM(DTL.PortUN_LOCODE)) NOT IN ('XXTBD', 'ATSEA', 'XXDTE', 'XXDTW')
    AND TB_PORT.PORT_COORDINATES IS NOT NULL
    AND TRIM(TB_PORT.PORT_COORDINATES) NOT IN ('0', '0,0', '0, 0');

   -- select count(*) from DTL_TBPORT limit 0
-- select top 10 * from VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_SCORING_FLAG

-- Creating a temporary table with VOYAGE_CODE logic
CREATE OR REPLACE TEMPORARY TABLE PP_OCI_Trade_Voy_Daylevel_data AS
SELECT 
    *,
    CONCAT(
        CruiseLineCode, '-', 
        ShipCode, '-', 
        TO_DATE(EmbarkationDate), '-', 
        EmbarkationPortUN_LOCODE, '-', 
        disEmbarkationPortUN_LOCODE, '-', 
        CruiseDuration
    ) AS VOYAGE_CODE
FROM 
    DTL_TBPORT;

   -- select top 10 * from PP_OCI_Trade_Voy_Daylevel_data


-- Step 1: CTE for embarkation region info
CREATE OR REPLACE TEMP TABLE NaturofVoyages AS 
WITH embark_regions AS (
    SELECT DISTINCT
        VOYAGE_CODE,
        NCLH_RM_Trade1 AS EMB_REGION1,
        NCLH_RM_TRADE2 AS EMB_REGION2,
        TRADE_3_MS AS EMB_REGION3
    FROM PP_OCI_Trade_Voy_Daylevel_data
    WHERE UPPER(TRIM(port_city_nam)) = UPPER(TRIM(embarkationport))
),

disembark_regions AS (
    SELECT DISTINCT
        VOYAGE_CODE,
        NCLH_RM_Trade1 AS DISEMB_REGION1,
        NCLH_RM_TRADE2 AS DISEMB_REGION2,
        TRADE_3_MS AS DISEMB_REGION3
    FROM PP_OCI_Trade_Voy_Daylevel_data
    WHERE UPPER(TRIM(port_city_nam)) = UPPER(TRIM(disembarkationport))
)

SELECT 
    f.*,
    e.EMB_REGION1,
    e.EMB_REGION2,
    e.EMB_REGION3,
    d.DISEMB_REGION1,
    d.DISEMB_REGION2,
    d.DISEMB_REGION3,

    CASE 
        WHEN UPPER(TRIM(f.embarkationport)) = UPPER(TRIM(f.Disembarkationport)) THEN 'RT'
        WHEN UPPER(TRIM(f.embarkationport)) != UPPER(TRIM(f.Disembarkationport))
            AND e.EMB_REGION3 = d.DISEMB_REGION3 THEN 'T3_RT'
        WHEN UPPER(TRIM(f.embarkationport)) != UPPER(TRIM(f.Disembarkationport))
            AND e.EMB_REGION3 != d.DISEMB_REGION3
            AND e.EMB_REGION2 = d.DISEMB_REGION2 THEN 'T2_RT'
        WHEN UPPER(TRIM(f.embarkationport)) != UPPER(TRIM(f.Disembarkationport))
            AND e.EMB_REGION3 != d.DISEMB_REGION3
            AND e.EMB_REGION2 != d.DISEMB_REGION2
            AND e.EMB_REGION1 = d.DISEMB_REGION1 THEN 'T1_RT'
        ELSE 'OJ'
    END AS NatureofVoyage

FROM 
    PP_OCI_Trade_Voy_Daylevel_data f
LEFT JOIN 
    embark_regions e 
    ON UPPER(TRIM(f.VOYAGE_CODE)) = UPPER(TRIM(e.VOYAGE_CODE))
LEFT JOIN 
    disembark_regions d 
    ON UPPER(TRIM(f.VOYAGE_CODE)) = UPPER(TRIM(d.VOYAGE_CODE));



-- STEP 1: Creating Temp Table to Identify Voyages With Only Sea & Embark/Disembark Ports
-- These are given a Port_Ind value of 0. If all ports have Port_Ind = 0, then it's a "CTN" candidate.
CREATE OR REPLACE TEMP TABLE Port_weight AS
SELECT 
    voyage_code
FROM (
    SELECT 
        *,
        CASE 
            WHEN UPPER(TRIM(port_city_nam)) = 'AT SEA'
              OR UPPER(TRIM(port_city_nam)) = UPPER(TRIM(embarkationport))
              OR UPPER(TRIM(port_city_nam)) = UPPER(TRIM(disembarkationport))
              OR port_city_nam IS NULL
              OR UPPER(TRIM(port_city_nam)) = 'NULL'
            THEN 0
            ELSE 1
        END AS Port_Ind
    FROM NaturofVoyages
) AS PORT_Weight
GROUP BY voyage_code
HAVING SUM(Port_Ind) = 0;

-- STEP 2: Creating Final Temp Table with CTN_FLAG
-- If a voyage_code exists in Port_weight, it gets a CTN flag, otherwise it’s marked as REGULAR.
CREATE OR REPLACE TEMP TABLE CTN_FLAG AS
SELECT 
    nv.*,
    CASE 
        WHEN nv.voyage_code IN (SELECT voyage_code FROM Port_weight) THEN 'CTN'
        ELSE 'REGULAR'
    END AS CTN_FLAG
FROM NaturofVoyages nv;


-- Rolling up required data at voyage level for further flag creations

CREATE OR REPLACE TEMP TABLE TempdatASet AS
SELECT  
    Voyage_Code,
    CruiseLineCode,
    ShipCode,
    CTN_Flag,
    NatureofVoyage,
    CruiseDuration,
    CAST(EmbarkationDate AS DATE)     AS EmbarkationDate,
    CAST(DisembarkationDate AS DATE)  AS DisembarkationDate,
    EmbarkationPort,
    DisembarkationPort,
    EMB_REGION1,
    EMB_REGION2,
    EMB_REGION3,
    DISEMB_REGION1,
    DISEMB_REGION2,
    DISEMB_REGION3,
    Port_City_Nam,
    COUNTRY_NAME,
    TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY') AS Itinerary_Date,
    Continent,
    NCLH_RM_TRADE1,
    NCLH_RM_TRADE2,
    TRADE_3_MS,
    TRADE_4_MS
FROM CTN_FLAG
WHERE 
    UPPER(TRIM(Port_city_nam)) IS NOT NULL
    AND UPPER(TRIM(Port_city_nam)) != 'NULL'
    AND TRIM(Port_city_nam) != ''
;



--select * from CTN_FLAG limit 200


--select top 10 * from ctn_flag-- where  Itinerary_Date = TO_DATE('04-25-2022', 'MM-DD-YYYY');

-- Bringing important columns at voyage level by rolling up with embarkation and disembarkation ports included
-- Creating itinerary port content (IPC)

CREATE OR REPLACE TEMP TABLE IPC AS
SELECT
    voyage_code,
    LISTAGG(port_city_nam, '#' ) 
        WITHIN GROUP (ORDER BY itinerary_date) AS ItineraryPortContent
FROM TempdatASet
GROUP BY voyage_code;

---Trade 1 
 
CREATE OR REPLACE TEMP TABLE T1 AS
SELECT
    voyage_code,
    NCLH_RM_Trade1 AS Trade1,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS Itinerary_Date,
    COUNT(NCLH_RM_Trade1) AS T1_count,
    CONCAT(NCLH_RM_Trade1, '-', COUNT(NCLH_RM_Trade1)) AS T1_Concat
FROM TempdatASet
WHERE NCLH_RM_Trade1 != 'Other'
GROUP BY voyage_code, NCLH_RM_Trade1;


CREATE OR REPLACE TEMP TABLE T1_Content AS
SELECT
    voyage_code,
    LISTAGG(T1_Concat, ', ') WITHIN GROUP (ORDER BY Itinerary_Date) AS T1_count_content,
    LISTAGG(Trade1, ', ') WITHIN GROUP (ORDER BY Itinerary_Date) AS T1_content
FROM T1
GROUP BY voyage_code;

-- Step 1: Create temp table for Trade 2 frequency per voyage
CREATE OR REPLACE TEMP TABLE T2 AS
SELECT
    voyage_code,
    NCLH_RM_Trade2,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(NCLH_RM_Trade2) AS T2_count,
    CONCAT(NCLH_RM_Trade2, '-', COUNT(NCLH_RM_Trade2)) AS T2_concat
FROM TempdatASet
WHERE NCLH_RM_Trade2 != 'Other'
GROUP BY voyage_code, NCLH_RM_Trade2;

-- Step 2: Aggregate T2 counts and distinct values by voyage
CREATE OR REPLACE TEMP TABLE T2_Content AS
SELECT
    voyage_code,
    LISTAGG(T2_concat, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T2_Count_Content,
    LISTAGG(NCLH_RM_Trade2, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T2_content
FROM T2
GROUP BY voyage_code;


-- Step 1: Create temp table for Trade 3 frequency per voyage
CREATE OR REPLACE TEMP TABLE T3 AS
SELECT
    voyage_code,
    Trade_3_MS,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(Trade_3_MS) AS T3_count,
    CONCAT(Trade_3_MS, '-', COUNT(Trade_3_MS)) AS T3_concat
FROM TempdatASet
WHERE Trade_3_MS != 'AT SEA'
GROUP BY voyage_code, Trade_3_MS;

-- Step 2: Aggregate T3 counts and distinct values by voyage
CREATE OR REPLACE TEMP TABLE T3_Content AS
SELECT
    voyage_code,
    LISTAGG(T3_concat, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T3_Count_Content,
    LISTAGG(Trade_3_MS, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T3_content
FROM T3
GROUP BY voyage_code;


-- Step 1: Create temp table for Trade 4 frequency per voyage
CREATE OR REPLACE TEMP TABLE T4 AS
SELECT
    voyage_code,
    Trade_4_MS,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(Trade_4_MS) AS T4_count,
    CONCAT(Trade_4_MS, '-', COUNT(Trade_4_MS)) AS T4_concat
FROM TempdatASet
WHERE Trade_4_MS IS NOT NULL
  AND UPPER(TRIM(Trade_4_MS)) != 'AT SEA'
  AND TRIM(Trade_4_MS) != ''
  AND UPPER(TRIM(Trade_4_MS)) != 'NULL'
GROUP BY voyage_code, Trade_4_MS;

-- Step 2: Aggregate T4 counts and distinct values by voyage
CREATE OR REPLACE TEMP TABLE T4_Content AS
SELECT
    voyage_code,
    LISTAGG(T4_concat, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T4_Count_Content,
    LISTAGG(Trade_4_MS, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T4_content
FROM T4
GROUP BY voyage_code;




-- Step 1: Create a temporary table that counts the occurrences of each continent
-- per voyage, filtering out invalid values like '', NULL, and 'NULL' (as string).
CREATE OR REPLACE TEMP TABLE con AS
SELECT
    voyage_code,
    Continent,
    MAX(Port_City_Nam) AS Port_City_Nam, -- Optional: to help with ordering
    MAX(itinerarY_date) AS itinerarY_date, -- Latest itinerary date per group
    COUNT(Continent) AS Con_count,
    CONCAT(Continent, '-', COUNT(Continent)) AS Continent_concat
FROM TempdatASet
WHERE TRIM(Continent) IS NOT NULL
  AND TRIM(Continent) != ''
  AND UPPER(TRIM(Continent)) != 'NULL'
GROUP BY voyage_code, Continent;


-- Step 2: Roll up continent info for each voyage:
-- 1. List of "Continent-Count" values
-- 2. List of distinct Continents
CREATE OR REPLACE TEMP TABLE Continent_Content AS
SELECT
    voyage_code,
    LISTAGG(Continent_concat, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS Continent_Count_Content,
    LISTAGG(Continent, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS Continent_content
FROM con
GROUP BY voyage_code;


-- ============================================================
-- Step: Create or replace a cleaned-up temp table
--       Excludes embarkation/disembarkation rows
--       and filters out invalid port names
-- ============================================================
CREATE OR REPLACE TEMP TABLE TEMP_DATASET_WO_EMB_DISEMB AS
SELECT *
FROM PP_OCI_TRADE_VOY_DAYLEVEL_DATA
WHERE 
  -- Exclude invalid or NULL Port City names
  PORT_CITY_NAM IS NOT NULL
  AND UPPER(TRIM(PORT_CITY_NAM)) <> ''
  AND UPPER(TRIM(PORT_CITY_NAM)) <> 'NULL'

  -- Exclude embarkation rows (same port & exact date match)
  AND (
    UPPER(TRIM(EMBARKATIONPORT)) <> UPPER(TRIM(PORT_CITY_NAM))
    OR TRY_TO_DATE(ITINERARY_DATE, 'MM-DD-YYYY') 
       <> CAST(EMBARKATIONDATE AS DATE)
  )

  -- Exclude disembarkation rows (same port & exact date match)
  AND (
    UPPER(TRIM(DISEMBARKATIONPORT)) <> UPPER(TRIM(PORT_CITY_NAM))
    OR TRY_TO_DATE(ITINERARY_DATE, 'MM-DD-YYYY') 
       <> CAST(DISEMBARKATIONDATE AS DATE)
  )
;

--select top 10 * from PP_OCI_TRADE_VOY_DAYLEVEL_DATA
-- ========================================================
-- Step 1: Trade 1 Aggregation (Excludes 'Other')
-- ========================================================
CREATE OR REPLACE TEMP TABLE t1emb AS
SELECT
    voyage_code,
    NCLH_RM_Trade1 AS Trade1,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(NCLH_RM_Trade1 ) AS t1_count,
    CONCAT(NCLH_RM_Trade1 , '-', COUNT(NCLH_RM_Trade1)) AS t1_concat_wo_e_d
FROM TEMP_DATASET_WO_EMB_DISEMB
WHERE NCLH_RM_Trade1  != 'Other'
GROUP BY voyage_code, NCLH_RM_Trade1 ;


CREATE OR REPLACE TEMP TABLE t1emb_content_wo_e_d AS
SELECT
    voyage_code,
    LISTAGG(t1_concat_wo_e_d, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T1_Count_content_wo_e_d,
    LISTAGG(Trade1, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T1_content_wo_e_d
FROM t1emb
GROUP BY voyage_code;

-- ========================================================
-- Step 2: Trade 2 Aggregation (Excludes 'Other')
-- ========================================================
CREATE OR REPLACE TEMP TABLE t2emb AS
SELECT
    voyage_code,
    NCLH_RM_Trade2 AS Trade2,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(NCLH_RM_Trade2) AS t2_count,
    CONCAT(NCLH_RM_Trade2, '-', COUNT(NCLH_RM_Trade2)) AS t2_concat_wo_e_d
FROM TEMP_DATASET_WO_EMB_DISEMB
WHERE NCLH_RM_Trade2 != 'Other'
GROUP BY voyage_code, NCLH_RM_Trade2;

CREATE OR REPLACE TEMP TABLE t2_content_wo_e_d AS
SELECT
    voyage_code,
    LISTAGG(t2_concat_wo_e_d, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T2_Count_content_wo_e_d,
    LISTAGG(Trade2, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T2_content_wo_e_d
FROM t2emb
GROUP BY voyage_code;

-- ========================================================
-- Step 3: Trade 3 Aggregation (Excludes 'AT SEA')
-- ========================================================
CREATE OR REPLACE TEMP TABLE t3emb AS
SELECT
    voyage_code,
    Trade_3_MS AS Trade3,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(Trade_3_MS) AS t3_count,
    CONCAT(Trade_3_MS, '-', COUNT(Trade_3_MS)) AS t3_concat_wo_e_d
FROM TEMP_DATASET_WO_EMB_DISEMB
WHERE Trade_3_MS != 'AT SEA'
GROUP BY voyage_code, Trade_3_MS;

CREATE OR REPLACE TEMP TABLE t3_content_wo_e_d AS
SELECT
    voyage_code,
    LISTAGG(t3_concat_wo_e_d, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T3_Count_content_wo_e_d,
    LISTAGG(Trade3, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T3_content_wo_e_d
FROM t3emb
GROUP BY voyage_code;

-- ========================================================
-- Step 4: Trade 4 Aggregation (Excludes '', 'AT SEA', 'NULL')
-- ========================================================
CREATE OR REPLACE TEMP TABLE t4emb AS
SELECT
    voyage_code,
    Trade_4_MS AS Trade4,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(Trade_4_MS) AS t4_count,
    CONCAT(Trade_4_MS, '-', COUNT(Trade_4_MS)) AS t4_concat_wo_e_d
FROM TEMP_DATASET_WO_EMB_DISEMB
WHERE Trade_4_MS NOT IN ('', 'AT SEA')
  AND Trade_4_MS IS NOT NULL
  AND UPPER(TRIM(Trade_4_MS)) != 'NULL'
GROUP BY voyage_code, Trade_4_MS;

CREATE OR REPLACE TEMP TABLE t4_content_wo_e_d AS
SELECT
    voyage_code,
    LISTAGG(t4_concat_wo_e_d, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T4_Count_content_wo_e_d,
    LISTAGG(Trade4, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS T4_content_wo_e_d
FROM t4emb
GROUP BY voyage_code;

-- ========================================================
-- Step 5: Continent Aggregation (Excludes '', NULL, 'NULL')
-- ========================================================
CREATE OR REPLACE TEMP TABLE conemb AS
SELECT
    voyage_code,
    Continent,
    MAX(Port_City_Nam) AS Port_City_Nam,
    MAX(itinerarY_date) AS itinerarY_date,
    COUNT(Continent) AS Con_count,
    CONCAT(Continent, '-', COUNT(Continent)) AS Continent_concat_wo_e_d
FROM TEMP_DATASET_WO_EMB_DISEMB
WHERE TRIM(Continent) IS NOT NULL
  AND TRIM(Continent) != ''
  AND UPPER(TRIM(Continent)) != 'NULL'
GROUP BY voyage_code, Continent;

CREATE OR REPLACE TEMP TABLE Continent_content_wo_e_d AS
SELECT
    voyage_code,
    LISTAGG(Continent_concat_wo_e_d, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS Continent_Count_content_wo_e_d,
    LISTAGG(Continent, ', ') WITHIN GROUP (ORDER BY itinerarY_date) AS Continent_content_wo_e_d
FROM conemb
GROUP BY voyage_code;


-- ========================================================
-- Bringing all concatenated content columns together (with UPPER(TRIM))
-- ========================================================

CREATE OR REPLACE TEMP TABLE contentioncolumn AS
SELECT  
    Tempds.*,
    ipc.ItineraryPortContent,
    con.Continent_content,
    con.Continent_Count_content,
    conemb.Continent_content_wo_e_d,
    conemb.Continent_Count_content_wo_e_d,
    t1.T1_content,
    t1.T1_Count_content,
    t2.T2_content,
    t2.T2_Count_content,
    t3.T3_Content,
    t3.T3_Count_Content,
    t4.T4_content,
    t4.T4_Count_content,
    t1emb.t1_content_wo_e_d,
    t1emb.t1_Count_content_wo_e_d,
    t2emb.T2_content_wo_e_d,
    t2emb.T2_Count_content_wo_e_d,
    t3emb.T3_Content_wo_e_d,
    t3emb.T3_Count_content_wo_e_d,
    t4emb.T4_Content_wo_e_d,
    t4emb.T4_Count_content_wo_e_d
FROM TEMPDATASET AS Tempds
LEFT JOIN IPC AS ipc
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(ipc.voyage_code))
LEFT JOIN T1_CONTENT AS t1
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t1.voyage_code))
LEFT JOIN T2_CONTENT AS t2
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t2.voyage_code))
LEFT JOIN T3_CONTENT AS t3
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t3.voyage_code))
LEFT JOIN T4_CONTENT AS t4
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t4.voyage_code))
LEFT JOIN CONTINENT_CONTENT AS con
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(con.voyage_code))
LEFT JOIN T1EMB_CONTENT_WO_E_D AS t1emb
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t1emb.voyage_code))
LEFT JOIN T2_CONTENT_WO_E_D AS t2emb
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t2emb.voyage_code))
LEFT JOIN T3_CONTENT_WO_E_D AS t3emb
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t3emb.voyage_code))
LEFT JOIN T4_CONTENT_WO_E_D AS t4emb
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(t4emb.voyage_code))
LEFT JOIN CONTINENT_CONTENT_WO_E_D AS conemb
    ON UPPER(TRIM(Tempds.voyage_code)) = UPPER(TRIM(conemb.voyage_code));



-------------------------Bringing All the flags AND important columns at voyage level------------------------------------
CREATE OR REPLACE TEMP TABLE pre_t4_PRESENCE AS
SELECT 	Voyage_Code
		,MAX(CTN_Flag) CTN_Flag
		,MAX(NatureofVoyage) NatureofVoyage
		,MAX(CruiseDuration) CruiseDuration
		,MAX(EmbarkationDate) EmbarkationDate
		,MAX(DisembarkationDate) DisembarkationDate
		,MAX(EmbarkationPort) EmbarkationPort
		,MAX(DisembarkationPort) DisembarkationPort
		,MAX(EMB_REGION1) EMB_REGION1
		,MAX(EMB_REGION2) EMB_REGION2
		,MAX(EMB_REGION3) EMB_REGION3
		,MAX(DISEMB_REGION1) DISEMB_REGION1
		,MAX(DISEMB_REGION2) DISEMB_REGION2
		,MAX(DISEMB_REGION3) DISEMB_REGION3
		,MAX(ItineraryPortcontent) ItineraryPortcontent
		,MAX(Continent_content) Continent_content
		,MAX(Continent_Count_content) Continent_Count_Content
		,MAX(Continent_content_wo_e_d) Continent_content_wo_e_d
		,MAX(Continent_Count_content_wo_e_d) Continent_Count_Content_wo_e_d
		,MAX(T1_content) T1_content
		,MAX(T1_COUNT_content) T1_COUNT_Content
		,MAX(T2_content) T2_content
		,MAX(T2_Count_Content) T2_Count_Content
		,MAX(T3_content) T3_content
		,MAX(T3_Count_Content) T3_Count_Content
		,MAX(T4_content) T4_content
		,MAX(T4_Count_Content) T4_Count_Content
		,MAX(T1_content_wo_e_d) T1_content_wo_e_d
		,MAX(T1_Count_Content_wo_e_d) T1_Count_Content_wo_e_d
		,MAX(T2_content_wo_e_d) T2_content_wo_e_d
		,MAX(T2_Count_Content_wo_e_d) T2_Count_Content_wo_e_d
		,MAX(T3_content_wo_e_d) T3_content_wo_e_d
		,MAX(T3_Count_Content_wo_e_d) T3_Count_Content_wo_e_d
		,MAX(T4_content_wo_e_d) T4_content_wo_e_d
		,MAX(T4_Count_Content_wo_e_d) T4_Count_Content_wo_e_d
FROM 	contentioncolumn
GROUP BY voyage_code
order by voyage_code, EmbarkationDate;


-----T4_presence_flag COLUMN-----------------------------------------
-- AT voyage level--19202
Create or replace temp table T4_PRESENCE as 
SELECT 	*
			,CASE 	WHEN T4_content_wo_e_d IS NOT NULL THEN 'Y' 
					ELSE 'N'
			END AS T4_presence_flag 
 FROM 	pre_t4_PRESENCE;


 -- ========================================================
-- Bring HDR_Source_itinerary_date by creating Voyage_Code in HDR
-- ========================================================
CREATE OR REPLACE TEMP TABLE hdr AS
SELECT  
    *,
    CONCAT(
        CruiseLineCode, 
        '-', ShipCode, 
        '-', TO_DATE(EmbarkationDate), 
        '-', EmbarkationPort, 
        '-', TO_DATE(DisembarkationDate), 
        '-', DisembarkationPort
    ) AS voyage_code
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRODUCT_VOY_LEVEL_RG_PASS;


-- ========================================================
-- Using CTEs to cleanly bring in SourceItineraryName from HDR
-- ========================================================

CREATE OR REPLACE TEMP TABLE sourceitnam AS
WITH hdr_clean AS (
    SELECT  
        UPPER(TRIM(Voyage_Code)) AS voyage_code,
        MAX(SOURCETINERARYNAME) AS SourceItineraryName
    FROM hdr
    WHERE 
        SOURCETINERARYNAME IS NOT NULL 
        AND TRIM(SOURCETINERARYNAME) != ''
        AND UPPER(TRIM(SOURCETINERARYNAME)) != 'NULL'
    GROUP BY UPPER(TRIM(Voyage_Code))
),

t4_clean AS (
    SELECT 
        *,
        UPPER(TRIM(Voyage_Code)) AS voyage_code_clean
    FROM T4_PRESENCE
)

SELECT  
    t4_clean.*,
    hdr_clean.SourceItineraryName
FROM t4_clean
LEFT JOIN hdr_clean
    ON UPPER(TRIM(t4_clean.voyage_code_clean)) = UPPER(TRIM(hdr_clean.voyage_code));

   -- select * from hdr limit 1 

   
-- Creating Transatlantic Flag
-- Step 1: Checking if the cruise duration is less than or equal to 50, and if the T1_content matches specific values
CREATE OR REPLACE TEMP TABLE transatlantic_flag AS
SELECT  
    *,
    CASE  
        WHEN (cruiseduration <= 50) AND (
            UPPER(TRIM(T1_content)) ILIKE '%EUROPE%AMERICA%' OR
            UPPER(TRIM(T1_content)) ILIKE '%AMERICA%EUROPE%' OR
            UPPER(TRIM(T1_content)) ILIKE '%AMERICA%AFRICA%' OR
            UPPER(TRIM(T1_content)) ILIKE '%AFRICA%AMERICA%' OR
            UPPER(TRIM(T1_content)) ILIKE '%EUROPE%CARIBBEAN%' OR
            UPPER(TRIM(T1_content)) ILIKE '%CARIBBEAN%EUROPE%'
            -- Add more conditions for SourceItineraryName if necessary
        ) THEN 'Y'
        ELSE 'N'
    END AS Transatlantic_flag
FROM sourceitnam;

-- Creating T1 and T2 counts
-- Step 2: Counting occurrences in T1_content and T2_content
CREATE OR REPLACE TEMP TABLE T1T2_count AS
SELECT  
    *,
    -- Calculate T1_count by counting commas and adding 1 (to account for the last item without a comma)
    (LENGTH(T1_content_wo_e_d) - LENGTH(REPLACE(T1_content_wo_e_d, ',', ''))) + 1 AS T1_count,
    
    -- Calculate T2_count similarly
    (LENGTH(T2_content_wo_e_d) - LENGTH(REPLACE(T2_content_wo_e_d, ',', ''))) + 1 AS T2_count,

    -- Step 3: Creating Transpacific flag
    CASE  
        WHEN (cruiseduration < 50) AND (
            UPPER(TRIM(T3_content)) ILIKE '%JAPAN%ALASKA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%ALASKA%JAPAN%' OR
            UPPER(TRIM(T3_content)) ILIKE '%WEST COAST NA%JAPAN%' OR
            UPPER(TRIM(T3_content)) ILIKE '%JAPAN%WEST COAST NA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%INDONESIA%WEST COAST NA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%WEST COAST NA%INDONESIA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%WEST COAST NA%SOUTH COAST AUSTRALIA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%SOUTH COAST AUSTRALIA%WEST COAST NA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%SOUTH AMERICA%NEW ZEALAND%' OR
            UPPER(TRIM(T3_content)) ILIKE '%NEW ZEALAND%SOUTH AMERICA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%SOUTH COAST AUSTRALIA%SOUTH AMERICA%' OR
            UPPER(TRIM(T3_content)) ILIKE '%SOUTH AMERICA%SOUTH COAST AUSTRALIA%' OR
            UPPER(TRIM(T2_content)) ILIKE '%SOUTHEAST ASIA%WEST COAST NA%' OR
            UPPER(TRIM(T2_content)) ILIKE '%WEST COAST NA%SOUTHEAST ASIA%'
        ) THEN 'Y'
        ELSE 'N'
    END AS Transpacific_flag
FROM transatlantic_flag;

-- Step 4: Creating World Flag
CREATE OR REPLACE TEMP TABLE wflag AS
SELECT  
    *,
    CASE  
        -- When cruise duration is >= 50 and T1_count >= 2
        WHEN cruiseduration >= 50 AND T1_count >= 2 THEN 'Y'
        -- When cruise duration is between 25 and 50, and T1_count > 2 and T2_count >= 4
        WHEN cruiseduration >= 25 AND cruiseduration < 50 AND T1_count > 2 AND T2_count >= 4 THEN 'Y'
        ELSE 'N'
    END AS World_flag
FROM T1T2_count;



----Bringing all flag in a table at voyage level------------------------------------

--DROP TABLE IF EXISTS TA.DS_Voy_level
--SELECT 	*
--INTO	TA.DS_Voy_level
--FROM 	#wflag


----------------------USE #WFLAG DATASET AS VOYAGE LEVEL UNTIL CHANGED-------------------------------------------------------------------------
----------------------USE #WFLAG DATASET AS VOYAGE LEVEL UNTIL CHANGED-------------------------------------------------------------------------
----------------------USE #WFLAG DATASET AS VOYAGE LEVEL UNTIL CHANGED-------------------------------------------------------------------------
----------------------USE #WFLAG DATASET AS VOYAGE LEVEL UNTIL CHANGED-------------------------------------------------------------------------
----------------------USE #WFLAG DATASET AS VOYAGE LEVEL UNTIL CHANGED-------------------------------------------------------------------------



-- Step 1: Create a temporary table #Parameter_DS from the source data
-----------------------------------Running Pattern at voy_day_level data----------
CREATE OR REPLACE TEMP TABLE Parameter_DS AS
SELECT *
FROM PP_OCI_TRADE_VOY_DAYLEVEL_DATA;


---------------Parameter--1---Distance--------------


------ Creating Embark DataSet ------
CREATE OR REPLACE TEMP TABLE finaldatASet_Embark AS
SELECT DISTINCT 
    Voyage_Code,
    UPPER(TRIM(embarkationport))   AS EmbarkationPort,
    PORT_LAT                       AS PE_Lat,
    PORT_LONG                      AS PE_Long,
    CAST(EmbarkationDate AS DATE)  AS EmbarkationDate
FROM Parameter_DS
WHERE 
    TRY_TO_DATE(NULLIF(TRIM(Itinerary_Date), ''), 'MM-DD-YYYY')
      = CAST(EmbarkationDate AS DATE)
  AND UPPER(TRIM(embarkationport)) 
      = UPPER(TRIM(Port_city_nam));


----Joining embarkation port details--AS Column--PE--Lat--AND--PE--long----215121
CREATE OR REPLACE TEMP TABLE trade_eda_d1 AS
SELECT  a.*,
        b.PE_Lat,
        b.PE_Long
FROM    Parameter_DS a
LEFT JOIN finaldatASet_Embark b
    ON  upper(trim(a.Voyage_Code)) = upper(trim(b.Voyage_Code))
    AND UPPER(TRIM(a.embarkationport)) = UPPER(TRIM(b.embarkationport));


CREATE OR REPLACE TEMP TABLE trade_eda_d2 AS
SELECT 
    *,
    CASE 
        WHEN PE_Long BETWEEN -180 AND 180 AND PE_Lat BETWEEN -90 AND 90 
        THEN TO_GEOGRAPHY('POINT(' || PE_Long || ' ' || PE_Lat || ')') 
        ELSE NULL 
    END AS geo1,

    CASE 
        WHEN Port_Long BETWEEN -180 AND 180 AND Port_Lat BETWEEN -90 AND 90 
        THEN TO_GEOGRAPHY('POINT(' || Port_Long || ' ' || Port_Lat || ')') 
        ELSE NULL 
    END AS geo2
FROM trade_eda_d1;


--select top 10 * from finaldatASet_d1_f

-- SELECT *
-- FROM trade_eda_d1
-- WHERE 
--     --PE_Lat NOT BETWEEN -90 AND 90 OR PE_Long NOT BETWEEN -180 AND 180 OR
--     Port_Lat NOT BETWEEN -90 AND 90 OR Port_Long NOT BETWEEN -180 AND 180;


-- Calculating the distance in nautical miles (meters / 1852)
CREATE OR REPLACE TEMP TABLE finaldatASet_d1_f AS
SELECT 
    *,
    ST_DISTANCE(geo1, geo2) / 1852 AS distance
FROM trade_eda_d2
WHERE Port_Lat IS NOT NULL 
  AND Port_Long IS NOT NULL
  AND UPPER(TRIM(PORT_loc_NAME)) != 'AT SEA'
  AND Port_City_Nam IS NOT NULL
ORDER BY Voyage_Code;


-- Aggregating maximum distance per voyage
CREATE OR REPLACE TEMP TABLE finaldatASet_d2_f AS
SELECT 
    Voyage_Code,
    MAX(Distance) AS Max_dis
FROM finaldatASet_d1_f
WHERE Port_Lat IS NOT NULL 
  AND Port_Long IS NOT NULL
  AND UPPER(TRIM(Port_City_Nam)) IS NOT NULL
  AND UPPER(TRIM(Port_City_Nam)) != 'AT SEA'
GROUP BY Voyage_Code
ORDER BY Voyage_Code;


-- Joining back max distance with full distance data
CREATE OR REPLACE TEMP TABLE finaldatASet_d3_f AS
SELECT 
    a.*,
    b.Max_dis
FROM finaldatASet_d1_f a
LEFT JOIN finaldatASet_d2_f b
    ON UPPER(TRIM(a.Voyage_Code)) = UPPER(TRIM(b.Voyage_Code))
ORDER BY a.Voyage_Code;


-- Final distance dataset (for ports with max distance)
CREATE OR REPLACE TEMP TABLE TA_finaldatASet_distance AS
SELECT DISTINCT 
    Voyage_Code,
    Port_City_Nam AS Dis_Port,
    nclh_rm_trade1 AS Dis_Trade1,
    nclh_rm_trade2 AS Dis_Trade2,
    TRADE_3_MS AS Dis_Trade3
FROM finaldatASet_d3_f
WHERE Distance = Max_dis;

--select top 1 * from finaldatASet_d3_f

-- Creating a dataset excluding embarkation/disembarkation ports to use for Mid-Point and Frequency
CREATE OR REPLACE TEMP TABLE finaldatASet_1 AS
SELECT *
FROM Parameter_DS
WHERE 
    UPPER(TRIM(Port_City_Nam)) <> 'AT SEA'
    AND UPPER(TRIM(Port_City_Nam)) <> 'NULL'
    AND Port_City_Nam IS NOT NULL
    AND (
        CAST(EmbarkationDate AS DATE) 
          <> TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY')
        OR UPPER(TRIM(EmbarkationPort)) 
          <> UPPER(TRIM(Port_City_Nam))
    )
    AND (
        CAST(DisembarkationDate AS DATE) 
          <> TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY')
        OR UPPER(TRIM(DisembarkationPort)) 
          <> UPPER(TRIM(Port_City_Nam))
    )
;


-- Parameter 2 - MID_Port using CTEs
CREATE OR REPLACE TEMP TABLE TA_mid_point_port AS
WITH ranked_ports AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY Voyage_Code ORDER BY TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY')) AS voyss_rank
    FROM finaldatASet_1
),
rank_bounds AS (
    SELECT 
        *,
        MAX(voyss_rank) OVER (PARTITION BY Voyage_Code) AS voyss_rank_MAX,
        MIN(voyss_rank) OVER (PARTITION BY Voyage_Code) AS voyss_rank_MIN
    FROM ranked_ports
),
mid_point_ports AS (
    SELECT DISTINCT  
        Voyage_Code,
        EmbarkationPort,
        DisembarkationPort,
        Port_City_Nam AS Mid_Point_Port,
        voyss_rank,
        "NCLH_RM_TRADE1" AS mid_point_Trade1,
        NCLH_RM_TRADE2 AS mid_point_Trade2,
        TRADE_3_MS AS mid_point_Trade3
    FROM rank_bounds
    WHERE voyss_rank = CEIL((voyss_rank_MAX + voyss_rank_MIN) / 2.0)
)

-- Final table creation
SELECT * FROM mid_point_ports;


-- Step 1: Calculate count of distinct TRADE_3_MS per voyage
CREATE OR REPLACE TEMP TABLE TA_FR_T AS
SELECT 
    UPPER(TRIM(voyage_code)) AS voyage_code,
    UPPER(TRIM(EmbarkationPort)) AS EmbarkationPort,
    UPPER(TRIM(DisembarkationPort)) AS DisembarkationPort,
    cruiseDuration,
    MAX(UPPER(TRIM(Port_City_Nam))) AS FRE_Port,
    MAX(itinerary_date) AS itinerary_date,
    UPPER(TRIM(TRADE_3_MS)) AS TRADE_3_MS,
    COUNT(TRADE_3_MS) AS count_trade3
FROM finaldatASet_1
GROUP BY 
    UPPER(TRIM(voyage_code)),
    UPPER(TRIM(EmbarkationPort)),
    UPPER(TRIM(DisembarkationPort)),
    cruiseDuration,
    UPPER(TRIM(TRADE_3_MS))
ORDER BY voyage_code, TRADE_3_MS;

-- Step 2: Rank trade region 3 based on frequency per voyage
CREATE OR REPLACE TEMP TABLE TA_FR_T_R AS
SELECT 
    voyage_code,
    EmbarkationPort,
    DisembarkationPort,
    itinerary_date,
    cruiseDuration,
    FRE_Port,
    TRADE_3_MS,
    count_trade3,
    RANK() OVER (
        PARTITION BY voyage_code
        ORDER BY count_trade3 DESC
    ) AS trade_rank
FROM TA_FR_T
WHERE TRADE_3_MS IS NOT NULL
QUALIFY trade_rank = 1;

-- Step 3: Combine ports and trade values into single row per voyage
CREATE OR REPLACE TEMP TABLE TA_fr_T_Final AS
SELECT 
    t1.voyage_code,
    MAX(t1.EmbarkationPort) AS EmbarkationPort,
    MAX(t1.DisembarkationPort) AS DisembarkationPort,
    MAX(t1.cruiseDuration) AS cruise_duration,
    LISTAGG(t2.FRE_Port, '#') WITHIN GROUP (ORDER BY t2.itinerary_date) AS Frequent_ports,
    LISTAGG(t3.TRADE_3_MS, '#') WITHIN GROUP (ORDER BY t3.itinerary_date) AS Frequency_Trade3,
    MAX(t1.count_trade3) AS Trade3_frequency,
    MAX(t1.trade_rank) AS Rank
FROM TA_FR_T_R t1
LEFT JOIN TA_FR_T_R t2 ON t2.voyage_code = t1.voyage_code
LEFT JOIN TA_FR_T_R t3 ON t3.voyage_code = t1.voyage_code
GROUP BY t1.voyage_code;

-- Step 4: Select only required columns
CREATE OR REPLACE TEMP TABLE TA_FR_TFinal AS
SELECT 
    voyage_code,
    EmbarkationPort,
    DisembarkationPort,
    cruise_duration,
    Frequent_ports,
    Frequency_Trade3,
    Trade3_frequency,
    Rank AS Fre_Rank
FROM TA_fr_T_Final;

-- Step 5: Join with original dataset to get TRADE1 and TRADE2
CREATE OR REPLACE TEMP TABLE freq AS
SELECT DISTINCT 
    T.*,
    MAX(UPPER(TRIM(F."NCLH_RM_TRADE1"))) AS Frequency_trade1,
    MAX(UPPER(TRIM(F."NCLH_RM_TRADE2"))) AS Frequency_trade2
FROM TA_FR_TFinal T
LEFT JOIN finaldatASet_1 F
    ON T.voyage_code = F.voyage_code
GROUP BY 
    T.voyage_code,
    T.EmbarkationPort,
    T.DisembarkationPort,
    T.cruise_duration,
    T.Frequent_ports,
    T.Frequency_Trade3,
    T.Trade3_frequency,
    T.Fre_Rank;


-- Step 1: Calculate frequency of distinct Port_City_Nam for every voyage
CREATE OR REPLACE TEMP TABLE TA_FP_T AS
SELECT 
    UPPER(TRIM(voyage_code)) AS voyage_code,
    UPPER(TRIM(EmbarkationPort)) AS EmbarkationPort,
    UPPER(TRIM(DisembarkationPort)) AS DisembarkationPort,
    cruiseDuration,
    MAX(Itinerary_date) AS Itinerary_date,
    UPPER(TRIM(Port_City_Nam)) AS Port_City_Nam,
    COUNT(UPPER(TRIM(port_city_nam))) AS Port_count,
    MAX(UPPER(TRIM("NCLH_RM_TRADE1"))) AS Port_trade1,
    MAX(UPPER(TRIM("NCLH_RM_TRADE2"))) AS Port_trade2,
    MAX(UPPER(TRIM(TRADE_3_MS))) AS port_trade_3
FROM finaldatASet_1
GROUP BY  
    UPPER(TRIM(voyage_code)),
    UPPER(TRIM(EmbarkationPort)),
    UPPER(TRIM(DisembarkationPort)),
    cruiseDuration,
    UPPER(TRIM(Port_City_Nam))
ORDER BY voyage_code, Port_count DESC;

-- Step 2: Rank Port_City_Nam within each voyage based on frequency
CREATE OR REPLACE TEMP TABLE TA_FP_T_R AS
SELECT *
FROM (
    SELECT 
        voyage_code,
        EmbarkationPort,
        DisembarkationPort,
        cruiseDuration,
        Itinerary_date,
        Port_City_Nam,
        Port_trade1,
        Port_trade2,
        port_trade_3,
        Port_count,
        RANK() OVER (
            PARTITION BY voyage_code
            ORDER BY Port_count DESC
        ) AS Port_rank
    FROM TA_FP_T
) t
WHERE Port_rank = 1
ORDER BY voyage_code;

-- Step 3: Combine ranked ports and trades into single row per voyage
CREATE OR REPLACE TEMP TABLE TA_fP_T_Final AS
SELECT 
    t1.voyage_code,
    MAX(t1.EmbarkationPort) AS EmbarkationPort,
    MAX(t1.DisembarkationPort) AS DisembarkationPort,
    MAX(t1.cruiseDuration) AS cruise_duration,
    MAX(t1.Port_trade1) AS Port_trade1,
    MAX(t1.Port_trade2) AS Port_trade2,
    LISTAGG(t2.Port_City_Nam, '#') WITHIN GROUP (ORDER BY t2.Itinerary_date) AS Frequent_ports,
    MAX(t1.Port_count) AS Port_frequency,
    MAX(t1.Port_rank) AS Port_Rank,
    LISTAGG(t3.port_trade_3, '#') WITHIN GROUP (ORDER BY t3.Itinerary_date) AS Port_fre_Trade3
FROM TA_FP_T_R t1
LEFT JOIN TA_FP_T_R t2 ON t2.voyage_code = t1.voyage_code
LEFT JOIN TA_FP_T_R t3 ON t3.voyage_code = t1.voyage_code
GROUP BY t1.voyage_code;

-- Step 4: Final port frequency table with required columns only
CREATE OR REPLACE TEMP TABLE TA_fP_TFinal AS
SELECT 
    voyage_code,
    Port_trade1 AS PORT_fre_NCLH_Trade1,
    Port_trade2 AS Port_fre_NCLH_Trade2,
    Frequent_ports AS Port_fre_Ports,
    Port_frequency,
    Port_fre_Trade3
FROM TA_fP_T_Final;

-- Step 5: Final merge of T3 frequency and Port frequency tables by voyage_code
CREATE OR REPLACE TEMP TABLE TA_FR_Tr3_P AS
SELECT 
    a.*, 
    b.PORT_fre_NCLH_Trade1,
    b.Port_fre_NCLH_Trade2,
    b.Port_fre_Ports,
    b.Port_frequency,
    b.Port_fre_Trade3
FROM freq a
LEFT JOIN TA_fP_TFinal b
    ON upper(trim(a.voyage_code)) = UPPER(TRIM(b.voyage_code));


------Resolving ties WHEN we have more than 1 trade 3 according to trade3 frequency rule-------------

--*In CASE WHEN both frequency rank is equal to port rank THEN SELECT Port_frequency trade3 AND corresponding Trade1, Trade2 AND port_city nam,
--otherwise SELECT all the above field a/c to T3_frequency rule--18995 rows */

CREATE OR REPLACE TEMP TABLE Frequency_PANDT3comb AS
SELECT 
    *,
    CASE 
        WHEN UPPER(TRIM(Port_frequency)) = UPPER(TRIM(Trade3_frequency)) THEN Port_fre_Trade3
        ELSE Frequency_Trade3
    END AS Freq_Trade3,
    CASE 
        WHEN Port_frequency = Trade3_frequency THEN Port_fre_Ports
        ELSE Frequent_ports
    END AS Freq_Ports,
    CASE 
        WHEN Port_frequency = Trade3_frequency THEN PORT_fre_NCLH_Trade1
        ELSE Frequency_trade1
    END AS Freq_Trade1,
    CASE 
        WHEN Port_frequency = Trade3_frequency THEN Port_fre_NCLH_Trade2
        ELSE Frequency_trade2
    END AS Freq_Trade2
FROM TA_FR_Tr3_P;



-- ========================================================================
-- PARAMATER 4 -- POPULARITY__
-- Step 1: Add flags for invalid port conditions (At Sea, NULLs, Embark/Disembark dates)
-- ========================================================================
-- ========================================================================
-- Step 1: Add flags for invalid port conditions and embark/disembark matches
-- ========================================================================
CREATE OR REPLACE TEMP TABLE test AS
SELECT 
    *,
    -- Flag for "At Sea"
    CASE 
        WHEN UPPER(TRIM(Port_City_Nam)) = 'AT SEA' THEN 'Y' 
        ELSE 'N' 
    END AS At_Sea_flag,

    -- Flag for NULL value
    CASE 
        WHEN Port_City_Nam IS NULL THEN 'Y' 
        ELSE 'N' 
    END AS null_val_flag,

    -- Flag for literal 'NULL' string
    CASE 
        WHEN UPPER(TRIM(Port_City_Nam)) = 'NULL' THEN 'Y' 
        ELSE 'N' 
    END AS null_flag,

    -- Flag for matching EmbarkationPort or EmbarkationDate = Itinerary_Date (formatted as 'MM-DD-YYYY')
    CASE 
        WHEN (
            UPPER(TRIM(Port_City_Nam)) = UPPER(TRIM(EmbarkationPort)) OR 
            TO_DATE(EmbarkationDate) = TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY')
        ) THEN 'Y' 
        ELSE 'N' 
    END AS emb_flag,

    -- Flag for matching DisembarkationPort or DisembarkationDate = Itinerary_Date (formatted as 'MM-DD-YYYY')
    CASE 
        WHEN (
            UPPER(TRIM(Port_City_Nam)) = UPPER(TRIM(DisembarkationPort)) OR 
            TO_DATE(DisembarkationDate) = TRY_TO_DATE(Itinerary_Date, 'MM-DD-YYYY')
        ) THEN 'Y' 
        ELSE 'N' 
    END AS disem_flag

FROM Parameter_DS;



-- ========================================================================
-- Step 2: Filter out invalid/irrelevant port entries
-- ========================================================================
CREATE OR REPLACE TEMP TABLE check1 AS
SELECT *
FROM test
WHERE 
    At_Sea_flag = 'N'
    AND null_val_flag = 'N'
    AND null_flag = 'N'
    AND emb_flag = 'N'
    AND disem_flag = 'N';

-- ========================================================================
-- Step 3: Count voyages for each port to determine popularity
-- ========================================================================
CREATE OR REPLACE TEMP TABLE voyage AS
SELECT 
    UPPER(TRIM(PORT_LOC_NAME)) AS PORT_LOC_NAME,
    COUNT(DISTINCT voyage_code) AS noofvoyagecode
FROM check1
GROUP BY UPPER(TRIM(PORT_LOC_NAME));

-- ========================================================================
-- Step 4: Rank ports by number of voyages
-- ========================================================================
CREATE OR REPLACE TEMP TABLE portrank AS
SELECT 
    PORT_LOC_NAME,
    noofvoyagecode,
    RANK() OVER (ORDER BY noofvoyagecode DESC) AS portrank
FROM voyage;

-- ========================================================================
-- Step 5: Assign sequential row number to each ranked port
-- ========================================================================
CREATE OR REPLACE TEMP TABLE portrowrank AS
SELECT 
    PORT_LOC_NAME,
    noofvoyagecode,
    portrank,
    ROW_NUMBER() OVER (ORDER BY portrank) AS rowrank
FROM portrank;

-- ========================================================================
-- Step 6: Join ranked port info with voyage-level data
-- ========================================================================
CREATE OR REPLACE TEMP TABLE ta_port_1 AS
SELECT 
    a.*,
    b.rowrank
FROM finaldatASet_1 a
LEFT JOIN portrowrank b
    ON UPPER(TRIM(a.PORT_LOC_NAME)) = UPPER(TRIM(b.PORT_LOC_NAME));

-- ========================================================================
-- Step 7: Get most popular port (lowest rank) for each voyage
-- ========================================================================
CREATE OR REPLACE TEMP TABLE ta_pop_1 AS
SELECT 
    voyage_code,
    MIN(rowrank) AS port_rank
FROM ta_port_1
GROUP BY voyage_code;

-- ========================================================================
-- Step 8: Join port rank with trade mappings from TB_Port_Final
-- ========================================================================
CREATE OR REPLACE TEMP TABLE ta_tbport AS
SELECT 
    p.*,
    t."NCLH_RM_TRADE1" AS popularTrade1,
    t."NCLH_RM_TRADE2" AS popularTrade2,
    t."NCLH_RM_TRADE3" AS popularTrade3
FROM portrowrank p
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA  t
    ON UPPER(TRIM(p.PORT_LOC_NAME)) = UPPER(TRIM(t.port_loc_name));

-- ========================================================================
-- Step 9: Final result — most popular port and associated trades per voyage
-- ========================================================================
CREATE OR REPLACE TEMP TABLE TA_popularPort AS
SELECT 
    p.voyage_code,
    t.PORT_LOC_NAME AS popular_city,
    t.popularTrade1,
    t.popularTrade2,
    t.popularTrade3
FROM ta_pop_1 p
LEFT JOIN ta_tbport t
    ON p.port_rank = t.rowrank;


-- Step 1: Join Distance and Mid Point
CREATE OR REPLACE TEMP TABLE Mid_point_Dis AS
SELECT  
    a.*, 
    b.Dis_port,
    b.Dis_Trade1,
    b.Dis_Trade2,
    b.Dis_Trade3
FROM TA_mid_point_port a
LEFT JOIN TA_finaldatASet_distance b
    ON UPPER(TRIM(a.Voyage_Code)) = UPPER(TRIM(b.Voyage_code));

-- Step 2: Join Mid Point with Frequency Data
CREATE OR REPLACE TEMP TABLE Mid_point_Dis_FRE AS
SELECT  
    a.*, 
    b.Freq_Ports AS Frequent_Ports,
    b.Freq_Trade1 AS Frequency_Trade1,
    b.Freq_Trade2 AS Frequency_Trade2,
    b.Freq_Trade3 AS Frequency_Trade3
FROM Mid_point_Dis a
LEFT JOIN Frequency_PANDT3comb b
    ON UPPER(TRIM(a.Voyage_Code)) = UPPER(TRIM(b.Voyage_code));

-- Step 3: Join with Popular Port Parameters
CREATE OR REPLACE TEMP TABLE Mid_Point_Dis_Fre_Pop AS
SELECT  
    M.*,
    P.popular_city,
    P.populartrade1 AS Popular_Trade1,
    P.populartrade2 AS Popular_Trade2,
    P.popularTrade3 AS Popular_Trade3
FROM Mid_point_Dis_FRE M
LEFT JOIN TA_popularPort P
    ON UPPER(TRIM(M.Voyage_Code)) = UPPER(TRIM(P.Voyage_code));

-- Step 4: Final Join with Full Voyage Data
CREATE OR REPLACE TEMP TABLE joinvoyage AS
SELECT  
    T.*,
    M.Mid_Point_Port,
    M.Mid_Point_Trade1,
    M.Mid_Point_Trade2,
    M.Mid_Point_Trade3,
    M.Dis_port,
    M.Dis_Trade1,
    M.Dis_Trade2,
    M.Dis_Trade3,
    M.Frequent_Ports,
    M.Frequency_Trade1,
    M.Frequency_Trade2,
    M.Frequency_Trade3,
    M.popular_city,
    M.Popular_Trade1,
    M.Popular_Trade2,
    M.Popular_Trade3
FROM WFLAG T
LEFT JOIN Mid_Point_Dis_Fre_Pop M
    ON UPPER(TRIM(T.Voyage_Code)) = UPPER(TRIM(M.Voyage_Code));


-- Step 1: CTE to calculate Frequency_T3_count and Fre_T3_count
CREATE OR REPLACE TEMP TABLE VOY_LEVEL_W_FT3 AS
WITH T3_Count_CTE AS (
    SELECT  
        voyage_code,
        LENGTH(Frequency_Trade3) - LENGTH(REPLACE(Frequency_Trade3, '#', '')) + 1 AS Frequency_T3_count
    FROM joinvoyage
),

Flag_CTE AS (
    SELECT 
        voyage_code,
        Frequency_T3_count,
        CASE 
            WHEN Frequency_T3_count > 1 THEN 1 
            ELSE 0 
        END AS Fre_T3_count
    FROM T3_Count_CTE
)

-- Step 2: Final Join with joinvoyage table
SELECT  
    V.*,
    F.Fre_T3_count
FROM joinvoyage V
LEFT JOIN Flag_CTE F
    ON UPPER(TRIM(V.voyage_code)) = UPPER(TRIM(F.voyage_code));


-- Step 1: Create CTE to calculate Frequency_T3_count per voyage
CREATE OR REPLACE TEMP TABLE t3count AS
WITH Frequency_T3_CTE AS (
    SELECT 
        voyage_code,
        CASE 
            WHEN Frequency_Trade3 IS NOT NULL AND TRIM(Frequency_Trade3) <> '' 
            THEN LENGTH(Frequency_Trade3) - LENGTH(REPLACE(Frequency_Trade3, '#', '')) + 1
            ELSE 0 
        END AS Frequency_T3_count
    FROM joinvoyage
)

-- Step 2: Final output with GROUP BY
SELECT 
    voyage_code,
    Frequency_T3_count,
    CASE 
        WHEN Frequency_T3_count > 1 THEN 1 
        ELSE 0 
    END AS Fre_T3_count
FROM Frequency_T3_CTE
GROUP BY voyage_code, Frequency_T3_count;


---select count(*) from t3count

-- Step: Join voyage-level data with T3 flag, using upper(trim)
CREATE OR REPLACE TEMP TABLE VOY_LEVEL_W_FT3 AS
SELECT  
    V.*,
    T.Fre_T3_count
FROM 
    joinvoyage V
LEFT JOIN 
    t3count T 
    ON UPPER(TRIM(V.voyage_code)) = UPPER(TRIM(T.voyage_code));

-----------------------------------USE #VOY_LEVEL_W_FT3 AS VOYAGE LEVEL DATASET UNTILL CHANGED----------------------------------------------
-----------------------------------USE #VOY_LEVEL_W_FT3 AS VOYAGE LEVEL DATASET UNTILL CHANGED----------------------------------------------
-----------------------------------USE #VOY_LEVEL_W_FT3 AS VOYAGE LEVEL DATASET UNTILL CHANGED----------------------------------------------
-----------------------------------USE #VOY_LEVEL_W_FT3 AS VOYAGE LEVEL DATASET UNTILL CHANGED----------------------------------------------
-----------------------------------USE #VOY_LEVEL_W_FT3 AS VOYAGE LEVEL DATASET UNTILL CHANGED----------------------------------------------


-------------------!@#$%^&*()&^%$#@#$%^&*(*&^%$#@!@#$%^&*(*&^%$#@------------


-----------------------------------------------------------Running Parameters for output at T4 level------------------------------------------------
--Creating Dataset for running parameters at t4 level------------------------------------
--Bringing T4_presence column at day level data to filter data where t4 is present\\


-- Step 1: Join voyage-level flags to main dataset
CREATE OR REPLACE TEMP TABLE Trade4_Dataset_w_flag AS
SELECT 
    a.*,
    b.T4_presence_flag
FROM 
    TEMP_DATASET_WO_EMB_DISEMB a
LEFT JOIN 
    VOY_LEVEL_W_FT3 b
    ON UPPER(TRIM(a.voyage_code)) = UPPER(TRIM(b.voyage_code));

-- Step 2: Filter dataset where T4 is present and Trade_4_ms is valid
CREATE OR REPLACE TEMP TABLE t4_dataset AS
SELECT * 
FROM Trade4_Dataset_w_flag
WHERE 
    T4_presence_flag = 'Y'
    AND Trade_4_ms IS NOT NULL
    AND Trade_4_ms != ''
    AND UPPER(TRIM(Trade_4_ms)) != 'NULL';


    

----Frequency 
-- Creating or replacing the temporary table count_t4
CREATE OR REPLACE TEMPORARY TABLE count_t4 AS
SELECT 
    voyage_code,
    MAX(itinerary_date) AS itinerary_date,
    Trade_4_ms,
    MAX(T4_PRESENCE_FLAG) AS T4_PRESENCE_FLAG,
    COUNT(TRADE_4_MS) AS count_trade4
FROM t4_dataset
GROUP BY voyage_code, trade_4_ms
ORDER BY voyage_code, TRADE_4_MS, count_trade4 DESC;

-- Creating or replacing the temporary table rankt4 with CTE for ranking trade region4
CREATE OR REPLACE TEMPORARY TABLE rankt4 AS
WITH ranked_trade AS (
    SELECT 
        Voyage_code,
        itinerary_date,
        trade_4_ms,
        T4_PRESENCE_FLAG,
        count_trade4,
        RANK() OVER (
            PARTITION BY voyage_code
            ORDER BY count_trade4 DESC
        ) AS trade_rank
    FROM count_t4
)
SELECT *
FROM ranked_trade
WHERE trade_rank = 1
ORDER BY voyage_code;


-- Creating or replacing the temporary table fre_trade4 using CTEs and UPPER(TRIM) in the join conditions
CREATE OR REPLACE TEMPORARY TABLE fre_trade4 AS
WITH ranked_trade AS (
    -- CTE to get voyages with rank 1 for trade_4
    SELECT
        t1.voyage_code,
        MAX(UPPER(TRIM(t1.T4_PRESENCE_FLAG))) AS T4_PRESENCE_FLAG,  -- Using UPPER and TRIM on T4_PRESENCE_FLAG
        -- Aggregating multiple trade_4_ms values into a single row using LISTAGG function
        LISTAGG('#' || UPPER(TRIM(t3.TRADE_4_MS)), '') WITHIN GROUP (ORDER BY t3.itinerary_date) AS Frequency_Trade4,  -- Using UPPER and TRIM on TRADE_4_MS
        MAX(t1.count_trade4) AS Trade4_frequency
    FROM rankt4 AS t1
    -- Joining with the ranked_trade table on voyage_code to get trade regions
    LEFT JOIN rankt4 AS t3
        ON UPPER(TRIM(t1.voyage_code)) = UPPER(TRIM(t3.voyage_code))  -- Using UPPER(TRIM) in join condition
    -- Group by voyage_code to consolidate the trade_4_ms values into one row
    GROUP BY t1.voyage_code
)
SELECT *
FROM ranked_trade;


-- Creating or replacing the temporary table t4count_fre using a CTE and handling the Frequency_T4_count logic
CREATE OR REPLACE TEMPORARY TABLE t4count_fre AS
WITH frequency_counts AS (
    -- CTE to calculate Frequency_T4_count
    SELECT
        voyage_code,
        UPPER(TRIM(T4_PRESENCE_FLAG)) AS T4_PRESENCE_FLAG,
        Frequency_Trade4,
        -- Calculating Frequency_T4_count based on the number of '#' characters in Frequency_Trade4
        (LEN(Frequency_Trade4) - LEN(REPLACE(Frequency_Trade4, '#', '')) + 1) AS Frequency_T4_count
    FROM fre_trade4
)
-- Main query to create the final table t4count_fre
SELECT
    voyage_code,
    MAX(T4_PRESENCE_FLAG) AS T4_PRESENCE_FLAG,
    Frequency_Trade4,
    Frequency_T4_count,
    -- Flagging when Frequency_T4_count is greater than 1
    CASE 
        WHEN Frequency_T4_count > 1 THEN 1
        ELSE 0
    END AS Fre_T4_count
FROM frequency_counts
GROUP BY
    voyage_code,
    Frequency_T4_count,
    Frequency_Trade4;


---------------------------------Popularity Parmeter---------------------------------


-- Creating or replacing the temporary table T4_Trade_rank using nested CTEs
CREATE OR REPLACE TEMPORARY TABLE T4_Trade_rank AS
WITH voyage_counts AS (
    -- Count distinct voyage_code per trade_4_ms
    SELECT
        UPPER(TRIM(trade_4_ms)) AS trade_4_ms,
        COUNT(DISTINCT voyage_code) AS noofvoyagecode
    FROM t4_dataset
    GROUP BY UPPER(TRIM(trade_4_ms))
),
ranked_trades AS (
    -- Assign rank based on number of voyages per trade_4_ms (popularity)
    SELECT
        *,
        RANK() OVER (ORDER BY noofvoyagecode DESC) AS traderank
    FROM voyage_counts
)
-- Add row number ordering by trade_4_ms
SELECT
    *,
    ROW_NUMBER() OVER (ORDER BY trade_4_ms) AS t4_rank
FROM ranked_trades;



---Note: We are alphabetically ranking the T4 because MITCH ASsigned the ranks bASed ON business rule
---- WHERE Adriatic Sea > Black Sea > Greek Isles ON bASis of popularity
---- so when we rank alphabetically the ranking remains intact
---- Hence we dON't need to create another table to rank them AS 1,2,3

------ Joining the rank to the main table at voy_day level

-- Creating or replacing the temporary table Voy_T4_Popularity
CREATE OR REPLACE TEMPORARY TABLE Voy_T4_Popularity AS
WITH joined_data AS (
    -- Joining the Trade4 dataset with the T4 trade ranks
    SELECT
        a.voyage_code,
        a.trade_4_ms,
        a.T4_PRESENCE_FLAG,
        b.t4_rank
    FROM Trade4_Dataset_w_flag a
    LEFT JOIN T4_Trade_rank b
        ON UPPER(TRIM(a.trade_4_ms)) = UPPER(TRIM(b.trade_4_ms))
),
min_rank_per_voyage AS (
    -- Getting the minimum T4 rank per voyage where T4 is present
    SELECT
        voyage_code,
        MIN(t4_rank) AS T4_pop_rank
    FROM joined_data
    WHERE T4_PRESENCE_FLAG = 'Y'
    GROUP BY voyage_code
)
-- Joining again to get the actual trade_4_ms (popular one) corresponding to the min rank
SELECT
    y.*,
    b.trade_4_ms AS Popular_Trade4
FROM min_rank_per_voyage y
LEFT JOIN T4_Trade_rank b
    ON UPPER(TRIM(y.T4_pop_rank)) = UPPER(TRIM(b.t4_rank));



-- Drop if the temporary table already exists
-- Create the final temporary table with Popular and Frequent Trade4 information
CREATE OR REPLACE TEMPORARY TABLE VOY_LEVEL_W_T4_parameters AS
SELECT  
    voy.*,
    POP.Popular_Trade4,
    FRE.Frequency_Trade4,
    FRE.Fre_T4_count
FROM 
    VOY_LEVEL_W_FT3 voy  -- LATEST VOY DAY LEVEL FILE

-- Join with Frequency-Based Trade4 Table
LEFT JOIN t4count_fre FRE
    ON UPPER(TRIM(voy.Voyage_code)) = UPPER(TRIM(FRE.voyage_code))

-- Join with Popularity-Based Trade4 Table
LEFT JOIN Voy_T4_Popularity POP
    ON UPPER(TRIM(voy.Voyage_code)) = UPPER(TRIM(POP.voyage_code));


--------------------- USE #VOY_LEVEL_W_T4_parameters TABLE INSTEAD OF TA.DS_Voy_level UNTIL CHANGED--------------------------------------
--------------------- USE #VOY_LEVEL_W_T4_parameters TABLE INSTEAD OF TA.DS_Voy_level UNTIL CHANGED--------------------------------------
--------------------- USE #VOY_LEVEL_W_T4_parameters TABLE INSTEAD OF TA.DS_Voy_level UNTIL CHANGED--------------------------------------
--------------------- USE #VOY_LEVEL_W_T4_parameters TABLE INSTEAD OF TA.DS_Voy_level UNTIL CHANGED--------------------------------------
--------------------- USE #VOY_LEVEL_W_T4_parameters TABLE INSTEAD OF TA.DS_Voy_level UNTIL CHANGED--------------------------------------
       
----------------------------------------------Scoring--------------------------------------------------------------------------------------------

-----------------------!@#$%^&*(*^%$#@#$%^&*(*&^%$#$%^&*(*&^%$#$%^*(*^%$#$%^&*&^%$#$%^&&^%$#$%^&*&^%$#$%--------------


--------------------Trade Extra---ASsignment for voyages which hAS South America AND Southern Caribbean in Caribbean Voyages
--- Adding the flags to check for caribbean region AND WHERE popular trade3 is BahamAS  
-- Create the temp table
CREATE OR REPLACE TEMPORARY TABLE Car_exception AS
SELECT  
    *,
    
    -- Derive Trade_Extra based on T3 content
    CASE 
        WHEN UPPER(TRIM(T3_content_wo_e_d)) ILIKE '%CARIBBEAN%AMERICA%' THEN 'Caribbean Other'
        WHEN UPPER(TRIM(T3_content_wo_e_d)) ILIKE '%SOUTHERN CARIBBEAN%' THEN 'Southern Caribbean'
        ELSE ''
    END AS Trade_Extra,

    -- Set P_B_Flag based on popular_trade3
    CASE 
        WHEN UPPER(TRIM(popular_trade3)) = 'BAHAMAS' THEN 'Y'
        ELSE 'N'
    END AS P_B_Flag

FROM VOY_LEVEL_W_T4_parameters;



-- Creating #extra_popdisMatch temp table for trade3 match flags
CREATE OR REPLACE TEMPORARY TABLE extra_popdisMatch AS
SELECT  *,
        CASE
            WHEN UPPER(TRIM(Frequency_Trade3)) = UPPER(TRIM(Dis_TRADE3)) THEN 'Y'
            ELSE 'N'
        END AS fre_dis_sig_match,
        CASE
            WHEN UPPER(TRIM(Dis_TRADE3)) = UPPER(TRIM(popular_TRADE3))
                 AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(Dis_TRADE3)) || '%' THEN 'Y'
            ELSE 'N'
        END AS all_para_like_match_flag,
        CASE
            WHEN UPPER(TRIM(Popular_Trade3)) = UPPER(TRIM(Dis_TRADE3)) THEN 'Y'
            ELSE 'N'
        END AS pop_dis_match
FROM Car_exception;

-- Creating #t4_temp table with derived Trade_t4 column
CREATE OR REPLACE TEMPORARY TABLE t4_temp AS
SELECT  *,
        CASE
            WHEN UPPER(TRIM(T4_presence_flag)) = 'Y' 
                 AND (UPPER(TRIM(world_flag)) = 'N' OR world_flag IS NULL) 
                 AND UPPER(TRIM(Popular_trade4)) = 'ADRIATIC SEA' THEN Popular_trade4
            WHEN UPPER(TRIM(T4_presence_flag)) = 'Y' 
                 AND (UPPER(TRIM(world_flag)) = 'N' OR world_flag IS NULL) 
                 AND Fre_T4_count != 1 THEN Frequency_trade4
            WHEN UPPER(TRIM(T4_presence_flag)) = 'Y' 
                 AND (UPPER(TRIM(world_flag)) = 'N' OR world_flag IS NULL) 
                 AND (Fre_T4_count = 1 OR Fre_T4_count IS NULL) THEN Popular_trade4
        END AS Trade_t4
FROM extra_popdisMatch;

-- Creating #tpop_temp table with pop_extra_flag
CREATE OR REPLACE TEMPORARY TABLE tpop_temp AS
SELECT  *,
        CASE
            WHEN UPPER(TRIM(popular_trade3)) IN ('BAHAMAS','EAST COAST NA','WEST COAST NA') THEN 1 
            ELSE 0
        END AS pop_extra_flag
FROM t4_temp;

-- Creating #trade_normal table with derived Trade3_normal column
CREATE OR REPLACE TEMPORARY TABLE trade_normal AS
SELECT  *,
        CASE  
            WHEN UPPER(TRIM(T4_presence_flag)) = 'N' AND all_para_like_match_flag = 'Y'  THEN Dis_Trade3  
            WHEN UPPER(TRIM(T4_presence_flag)) = 'N' AND all_para_like_match_flag = 'N' AND pop_extra_flag = 1 THEN Dis_Trade3
            WHEN UPPER(TRIM(T4_presence_flag)) = 'N' AND all_para_like_match_flag = 'N' AND pop_extra_flag = 0 THEN Popular_Trade3 
            ELSE COALESCE(Popular_Trade3, Dis_Trade3)
        END AS Trade3_normal
FROM tpop_temp;

-- Creating #PBCregion table with Panama, Bermuda, Canary flags
CREATE OR REPLACE TEMPORARY TABLE PBCregion AS
SELECT  *,
        CASE 
            WHEN UPPER(TRIM(T1_content_wo_e_d)) LIKE '%PANAMA CANAL%' 
                 AND CruiseDuration < 25 THEN 1 
            ELSE 0
        END AS Panama_flag,
        CASE 
            WHEN UPPER(TRIM(T2_content_wo_e_d)) LIKE '%BERMUDA%' 
                 AND CruiseDuration < 25 THEN 1 
            ELSE 0
        END AS Bermuda_flag,
        CASE 
            WHEN UPPER(TRIM(T3_content_wo_e_d)) LIKE '%CANARY ISLANDS%' 
                 AND CruiseDuration < 25 THEN 1 
            ELSE 0
        END AS Canary_flag
FROM trade_normal;


-- Create the CTE for flagging data
CREATE or replace TEMPORARY TABLE FLagcount AS

WITH FlaggedData AS (
    SELECT *,
        CASE 
            WHEN UPPER(TRIM(T1_content_wo_e_d)) LIKE '%CARIBBEAN%EUROPE%' 
                 OR UPPER(TRIM(T1_content_wo_e_d)) LIKE '%CARIBBEAN%AMERICA%' 
                 AND Panama_flag = 0 
                 AND Bermuda_flag = 0 
                 AND Canary_flag = 0 
            THEN 1 
            ELSE 0 
        END AS Caribbean_Other_Flag,
        
        CASE 
            WHEN UPPER(TRIM(T1_content_wo_e_d)) = 'CARIBBEAN' 
                 AND UPPER(TRIM(T3_content_wo_e_d)) LIKE '%SOUTHERN CARIBBEAN%' 
            THEN 1 
            ELSE 0 
        END AS South_Cari_flag
    FROM PBCregion
)

-- Now select from the CTE to calculate the flag count and insert it into the final table

SELECT *,
       (Panama_flag + Bermuda_flag + Canary_flag + Caribbean_Other_Flag + South_Cari_flag) AS Flag_Count
FROM FlaggedData;




-- Drop and create the exception table with flags, using UPPER(TRIM)
CREATE OR REPLACE TEMPORARY TABLE exception AS
SELECT *,
    CASE 
        WHEN UPPER(TRIM(world_flag)) = 'Y' THEN 'world'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND UPPER(TRIM(Transatlantic_flag)) = 'Y' THEN 'Transatlantic Cruise'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND UPPER(TRIM(Transatlantic_flag)) = 'N' AND UPPER(TRIM(TransPacific_Flag)) = 'Y' THEN 'Transpacific Cruise'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND UPPER(TRIM(Transatlantic_flag)) = 'N' AND UPPER(TRIM(TransPacific_Flag)) = 'N' AND UPPER(TRIM(CTN_Flag)) = 'CTN' THEN 'Cruise to NoWHERE'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND Flag_Count > 1 THEN 'Other'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND Flag_Count = 1 AND Panama_flag = 1 THEN 'Panama Canal'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND Flag_Count = 1 AND Bermuda_flag = 1 THEN 'Bermuda'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND Flag_Count = 1 AND Canary_flag = 1 THEN 'Canary Islands'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND Flag_Count = 1 AND Caribbean_Other_Flag = 1 THEN 'Caribbean Other'
        WHEN UPPER(TRIM(world_flag)) = 'N' AND South_Cari_flag = 1 THEN 'Southern Caribbean'
    END AS Exception_Trade3
FROM FLagcount;

-- Drop and create the second exception table using UPPER(TRIM)
CREATE OR REPLACE TEMPORARY TABLE EXCEPTION_VOY_LEVEL_FLAGS AS
SELECT * 
FROM exception;



-- -- Create or replace the table with flags for OJ SCORING
-- CREATE OR REPLACE TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG AS
-- select * from REF_EXCEPTION_VOY_LEVEL_FLAGS
-- SELECT *,
--     CASE 
--         WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
--              AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
--              AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(Popular_Trade3)) || '%' THEN 'Y' 
--         ELSE 'N' 
--     END AS OJ_Fr_Pop_flag,
    
--     CASE 
--         WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
--              AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
--              AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(disemb_region3)) || '%' THEN 'Y' 
--         ELSE 'N' 
--     END AS OJ_Fr_Disemb3_flag,
    
--     CASE 
--         WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
--              AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
--              AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(emb_region3)) || '%' THEN 'Y' 
--         ELSE 'N' 
--     END AS OJ_Fr_emb3_flag,
    
--     CASE 
--         WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
--              AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
--              AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(dis_Trade3)) || '%' THEN 'Y' 
--         ELSE 'N' 
--     END AS OJ_Fr_Dis_flag,
    
--     CASE 
--         WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
--              AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
--              AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(mid_point_Trade3)) || '%' THEN 'Y' 
--         ELSE 'N' 
--     END AS OJ_Fr_MID_flag
-- FROM EXCEPTION_VOY_LEVEL_FLAGS;

--select top 10 * from VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG where voyage_code='VOC-VM-2025-12-30-USPEF-USPEF-14'

-- /* 2) Add metadata/extra columns */
-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS VESSEL_IMO VARCHAR(50);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS NCLH_SHIP_CODE VARCHAR(5);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS SAIL_ID VARCHAR(50);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS MD5_VOYAGE_HASH_KEY VARCHAR;

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS HASH_DIFF VARCHAR;

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS LDTS TIMESTAMP_NTZ(9);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS RCSR VARCHAR(200);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG 
--   ADD COLUMN IF NOT EXISTS LAST_MODIFIED_BY VARCHAR;


-- /* 3) Populate VESSEL_IMO by joining ref ship (5L code = CruiseLineCode||ShipCode) */
-- UPDATE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG a
-- SET VESSEL_IMO = TO_VARCHAR(b.VESSEL_IMO_NO)
-- FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA b
-- WHERE REPLACE(SUBSTRING(a.VOYAGE_CODE, 1, 6), '-', '') = b."5L_SHIP_CODE";


-- /* 4) Map NCLH_SHIP_CODE from ship name (case-insensitive) */
-- UPDATE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG C
-- SET NCLH_SHIP_CODE = CASE SUBSTRING(C.voyage_code, 5, 2)
--     WHEN 'SX' THEN 'EXP'   -- Seven Seas Explorer
--     WHEN 'SN' THEN 'MAR'   -- Seven Seas Mariner
--     WHEN 'SP' THEN 'SPL'   -- Seven Seas Splendor
--     WHEN 'SV' THEN 'VOY'   -- Seven Seas Voyager
--     WHEN 'SS' THEN 'NAV'   -- Seven Seas Navigator
--     WHEN 'GD' THEN 'GRA'   -- Seven Seas Grandeur
--     WHEN 'D1' THEN 'PRT'   -- Seven Seas Prestige
--     ELSE NCLH_SHIP_CODE
-- END;


-- /* 5) Compute SAIL_ID and key hash */
-- UPDATE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG
-- SET SAIL_ID = CONCAT(NCLH_SHIP_CODE, TO_CHAR(EmbarkationDate, 'YYMMDD'));

-- UPDATE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG
-- SET MD5_VOYAGE_HASH_KEY = TO_VARCHAR(MD5(SAIL_ID));

-- /* 6) Set load metadata */
-- UPDATE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG
-- SET LDTS = CURRENT_TIMESTAMP(),
--     LAST_MODIFIED_BY = CURRENT_USER(),
--     RCSR = 'RSS|STEP_PART1|OUTPUT|PORT_DATA';

-- select top 10 * from VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG

CREATE OR REPLACE TEMP TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS as 
SELECT *,
    CASE 
        WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
             AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
             AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(Popular_Trade3)) || '%' THEN 'Y' 
        ELSE 'N' 
    END AS OJ_Fr_Pop_flag,
    
    CASE 
        WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
             AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
             AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(disemb_region3)) || '%' THEN 'Y' 
        ELSE 'N' 
    END AS OJ_Fr_Disemb3_flag,
    
    CASE 
        WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
             AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
             AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(emb_region3)) || '%' THEN 'Y' 
        ELSE 'N' 
    END AS OJ_Fr_emb3_flag,
    
    CASE 
        WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
             AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
             AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(dis_Trade3)) || '%' THEN 'Y' 
        ELSE 'N' 
    END AS OJ_Fr_Dis_flag,
    
    CASE 
        WHEN UPPER(TRIM(NatureofVoyage)) != 'RT' 
             AND UPPER(TRIM(NatureofVoyage)) != 'T3_RT'
             AND UPPER(TRIM(Frequency_Trade3)) LIKE '%' || UPPER(TRIM(mid_point_Trade3)) || '%' THEN 'Y' 
        ELSE 'N' 
    END AS OJ_Fr_MID_flag
FROM EXCEPTION_VOY_LEVEL_FLAGS;


/* 2) Add metadata/extra columns */
ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS VESSEL_IMO VARCHAR(50);

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS NCLH_SHIP_CODE VARCHAR(5);

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS SAIL_ID VARCHAR(50);

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS MD5_VOYAGE_HASH_KEY VARCHAR;

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS HASH_DIFF VARCHAR;

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS LDTS TIMESTAMP_NTZ(9);

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS RCSR VARCHAR(200);

ALTER TABLE REF_EXCEPTION_VOY_LEVEL_FLAGS
  ADD COLUMN IF NOT EXISTS LAST_MODIFIED_BY VARCHAR;

/* 3) Populate VESSEL_IMO by joining ref ship (5L code = CruiseLineCode||ShipCode) */
UPDATE REF_EXCEPTION_VOY_LEVEL_FLAGS a
SET VESSEL_IMO = TO_VARCHAR(b.VESSEL_IMO_NO),NCLH_SHIP_CODE= b.ship_code_3l
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA b
WHERE REPLACE(SUBSTRING(a.VOYAGE_CODE, 1, 6), '-', '') = b."5L_SHIP_CODE";


/* 4) Map NCLH_SHIP_CODE from ship name (case-insensitive) */
-- UPDATE REF_EXCEPTION_VOY_LEVEL_FLAGS C
-- SET NCLH_SHIP_CODE = CASE SUBSTRING(C.voyage_code, 5, 2)
--     WHEN 'SX' THEN 'EXP'   -- Seven Seas Explorer
--     WHEN 'SN' THEN 'MAR'   -- Seven Seas Mariner
--     WHEN 'SP' THEN 'SPL'   -- Seven Seas Splendor
--     WHEN 'SV' THEN 'VOY'   -- Seven Seas Voyager
--     WHEN 'SS' THEN 'NAV'   -- Seven Seas Navigator
--     WHEN 'GD' THEN 'GRA'   -- Seven Seas Grandeur
--     WHEN 'D1' THEN 'PRT'   -- Seven Seas Prestige
--     ELSE NCLH_SHIP_CODE
-- END;


/* 5) Compute SAIL_ID and key hash */
UPDATE REF_EXCEPTION_VOY_LEVEL_FLAGS
SET SAIL_ID = CONCAT(NCLH_SHIP_CODE, TO_CHAR(EmbarkationDate, 'YYMMDD'));

UPDATE REF_EXCEPTION_VOY_LEVEL_FLAGS
SET MD5_VOYAGE_HASH_KEY = TO_VARCHAR(MD5(SAIL_ID));

/* 6) Set load metadata */
UPDATE REF_EXCEPTION_VOY_LEVEL_FLAGS
SET LDTS = CURRENT_TIMESTAMP(),
    LAST_MODIFIED_BY = CURRENT_USER(),
    RCSR = 'RSS|STEP_PART1|OUTPUT|PORT_DATA';

UPDATE REF_EXCEPTION_VOY_LEVEL_FLAGS
SET HASH_DIFF = MD5(
      COALESCE(VOYAGE_CODE, '') || '|' ||
      COALESCE(CTN_FLAG, '') || '|' ||
      COALESCE(NATUREOFVOYAGE, '') || '|' ||
      COALESCE(CRUISEDURATION::STRING, '') || '|' ||
      COALESCE(EMBARKATIONDATE::STRING, '') || '|' ||
      COALESCE(DISEMBARKATIONDATE::STRING, '') || '|' ||
      COALESCE(EMBARKATIONPORT, '') || '|' ||
      COALESCE(DISEMBARKATIONPORT, '') || '|' ||
      COALESCE(EMB_REGION1, '') || '|' ||
      COALESCE(EMB_REGION2, '') || '|' ||
      COALESCE(EMB_REGION3, '') || '|' ||
      COALESCE(DISEMB_REGION1, '') || '|' ||
      COALESCE(DISEMB_REGION2, '') || '|' ||
      COALESCE(DISEMB_REGION3, '') || '|' ||
      COALESCE(ITINERARYPORTCONTENT, '') || '|' ||
      COALESCE(CONTINENT_CONTENT, '') || '|' ||
      COALESCE(CONTINENT_COUNT_CONTENT, '') || '|' ||
      COALESCE(CONTINENT_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(CONTINENT_COUNT_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T1_CONTENT, '') || '|' ||
      COALESCE(T1_COUNT_CONTENT, '') || '|' ||
      COALESCE(T2_CONTENT, '') || '|' ||
      COALESCE(T2_COUNT_CONTENT, '') || '|' ||
      COALESCE(T3_CONTENT, '') || '|' ||
      COALESCE(T3_COUNT_CONTENT, '') || '|' ||
      COALESCE(T4_CONTENT, '') || '|' ||
      COALESCE(T4_COUNT_CONTENT, '') || '|' ||
      COALESCE(T1_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T1_COUNT_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T2_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T2_COUNT_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T3_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T3_COUNT_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T4_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T4_COUNT_CONTENT_WO_E_D, '') || '|' ||
      COALESCE(T4_PRESENCE_FLAG, '') || '|' ||
      COALESCE(VOYAGE_CODE_CLEAN, '') || '|' ||
      COALESCE(SOURCEITINERARYNAME, '') || '|' ||
      COALESCE(TRANSATLANTIC_FLAG, '') || '|' ||
      COALESCE(T1_COUNT::STRING, '') || '|' ||
      COALESCE(T2_COUNT::STRING, '') || '|' ||
      COALESCE(TRANSPACIFIC_FLAG, '') || '|' ||
      COALESCE(WORLD_FLAG, '') || '|' ||
      COALESCE(MID_POINT_PORT, '') || '|' ||
      COALESCE(MID_POINT_TRADE1, '') || '|' ||
      COALESCE(MID_POINT_TRADE2, '') || '|' ||
      COALESCE(MID_POINT_TRADE3, '') || '|' ||
      COALESCE(DIS_PORT, '') || '|' ||
      COALESCE(DIS_TRADE1, '') || '|' ||
      COALESCE(DIS_TRADE2, '') || '|' ||
      COALESCE(DIS_TRADE3, '') || '|' ||
      COALESCE(FREQUENT_PORTS, '') || '|' ||
      COALESCE(FREQUENCY_TRADE1, '') || '|' ||
      COALESCE(FREQUENCY_TRADE2, '') || '|' ||
      COALESCE(FREQUENCY_TRADE3, '') || '|' ||
      COALESCE(POPULAR_CITY, '') || '|' ||
      COALESCE(POPULAR_TRADE1, '') || '|' ||
      COALESCE(POPULAR_TRADE2, '') || '|' ||
      COALESCE(POPULAR_TRADE3, '') || '|' ||
      COALESCE(FRE_T3_COUNT::STRING, '') || '|' ||
      COALESCE(POPULAR_TRADE4, '') || '|' ||
      COALESCE(FREQUENCY_TRADE4, '') || '|' ||
      COALESCE(FRE_T4_COUNT::STRING, '') || '|' ||
      COALESCE(TRADE_EXTRA, '') || '|' ||
      COALESCE(P_B_FLAG, '') || '|' ||
      COALESCE(FRE_DIS_SIG_MATCH, '') || '|' ||
      COALESCE(ALL_PARA_LIKE_MATCH_FLAG, '') || '|' ||
      COALESCE(POP_DIS_MATCH, '') || '|' ||
      COALESCE(TRADE_T4, '') || '|' ||
      COALESCE(POP_EXTRA_FLAG::STRING, '') || '|' ||
      COALESCE(TRADE3_NORMAL, '') || '|' ||
      COALESCE(PANAMA_FLAG::STRING, '') || '|' ||
      COALESCE(BERMUDA_FLAG::STRING, '') || '|' ||
      COALESCE(CANARY_FLAG::STRING, '') || '|' ||
      COALESCE(CARIBBEAN_OTHER_FLAG::STRING, '') || '|' ||
      COALESCE(SOUTH_CARI_FLAG::STRING, '') || '|' ||
      COALESCE(FLAG_COUNT::STRING, '') || '|' ||
      COALESCE(EXCEPTION_TRADE3, '') || '|' ||
      COALESCE(OJ_FR_POP_FLAG, '') || '|' ||
      COALESCE(OJ_FR_DISEMB3_FLAG, '') || '|' ||
      COALESCE(OJ_FR_EMB3_FLAG, '') || '|' ||
      COALESCE(OJ_FR_DIS_FLAG, '') || '|' ||
      COALESCE(OJ_FR_MID_FLAG, '') || '|' ||
      COALESCE(VESSEL_IMO, '') || '|' ||
      COALESCE(NCLH_SHIP_CODE, '') || '|' ||
      COALESCE(SAIL_ID, '')
);


-- alter table VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG
-- rename to VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_SCORING_FLAG;

-- Create or replace the table with flags for OJ SCORING
TRUNCATE TABLE VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_SCORING_FLAG;
INSERT INTO VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_SCORING_FLAG
select * from REF_EXCEPTION_VOY_LEVEL_FLAGS;


-- ALTER TABLE  VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_SCORING_FLAG
-- ADD PRIMARY KEY (VOYAGE_CODE);





## Trade_Scoring_Assignment_RSS_Competitors

## Scoring process to assign trades for RSS and competitor voyages, based on pre-assigned flags.

In [ ]:

CREATE OR REPLACE TEMPORARY TABLE PP_RSS_MS_Recommended_Trade AS
SELECT *,
    CASE 
        WHEN Exception_Trade3 IS NOT NULL THEN Exception_Trade3

        WHEN Exception_Trade3 IS NULL AND Natureofvoyage = 'RT' THEN
            CASE 
                WHEN trade_t4 IS NOT NULL THEN trade_t4
                WHEN trade_t4 IS NULL AND trade_extra != '' THEN trade_extra
                ELSE trade3_normal
            END

        WHEN Exception_Trade3 IS NULL 
             AND (Natureofvoyage = 'T3_RT' OR Natureofvoyage = 'T2_RT' OR Natureofvoyage = 'T1_RT') THEN
            CASE 
                WHEN trade_t4 IS NOT NULL THEN trade_t4
                WHEN trade_t4 IS NULL AND trade_extra != '' THEN trade_extra
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count = 0 THEN Frequency_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'Y' AND OJ_Fr_DIS_flag = 'Y' AND OJ_Fr_Pop_flag = 'Y' THEN Popular_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'Y' AND OJ_Fr_DIS_flag = 'Y' AND OJ_Fr_Pop_flag = 'N' THEN mid_point_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'Y' AND OJ_Fr_DIS_flag = 'N' AND OJ_Fr_Pop_flag = 'Y' THEN Popular_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'N' AND OJ_Fr_DIS_flag = 'Y' AND OJ_Fr_Pop_flag = 'Y' THEN Popular_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'Y' AND OJ_Fr_DIS_flag = 'N' AND OJ_Fr_Pop_flag = 'N' THEN mid_point_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'N' AND OJ_Fr_DIS_flag = 'N' AND OJ_Fr_Pop_flag = 'Y' THEN Popular_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_MID_flag = 'N' AND OJ_Fr_DIS_flag = 'Y' AND OJ_Fr_Pop_flag = 'N' THEN DIS_Trade3
                ELSE Popular_Trade3
            END

        WHEN Exception_Trade3 IS NULL AND Natureofvoyage = 'OJ' THEN
            CASE 
                WHEN trade_t4 IS NOT NULL THEN trade_t4
                WHEN trade_t4 IS NULL AND trade_extra != '' THEN trade_extra
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count = 0 THEN Frequency_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND OJ_Fr_DIS_flag = 'Y' THEN DIS_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND OJ_Fr_DIS_flag = 'N' AND OJ_Fr_MID_flag = 'Y' THEN mid_point_Trade3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_Pop_flag = 'N' AND OJ_Fr_MID_flag = 'N' AND OJ_Fr_DIS_flag = 'N' AND OJ_Fr_Disemb3_flag = 'Y' THEN disemb_region3
                WHEN trade_t4 IS NULL AND trade_extra = '' AND Fre_T3_count != 0 AND 
                     OJ_Fr_Pop_flag = 'N' AND OJ_Fr_MID_flag = 'N' AND OJ_Fr_DIS_flag = 'N' AND OJ_Fr_Disemb3_flag = 'N' AND OJ_Fr_emb3_flag = 'Y' THEN emb_region3
                ELSE Popular_Trade3
            END
    END AS MS_Recommended_T3
FROM VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_SCORING_FLAG;

-- select count(*) from VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_SCORING_FLAG where sail_id is null

---select * from PP_OCI_MS_Recommended_Trade where MS_Recommended_T3 is null;


-- ----QC-----
-- select voyage_code, count(*) 
-- from #PP_OCI_MS_Recommeded_Trade
-- group by voyage_code
-- having count(*) >1 --nulls   

-- select * from #PP_OCI_MS_Recommeded_Trade
-- where MS_recommended_T3 is null 

-- select top 10 * from #PP_OCI_MS_Recommeded_Trade

-- --select voyage_code, count(*) 
-- --from TA.ALL_scoring_flag
-- --group by voyage_code
-- --having count(*) >1

-- --select *
-- --from TA.DS_Voy_level
-- --where voyage_code='RCI-MP-2020-10-31-USMIA-USMIA-8'

-- --select * from TA.DS_Voy_level
-- --where VOYAGE_CODE='RCI-MP-2020-10-31-USMIA-USMIA-8'

alter table PP_RSS_MS_Recommended_Trade
  DROP COLUMN IF EXISTS VESSEL_IMO,
                       NCLH_SHIP_CODE,
                       SAIL_ID,
                       MD5_VOYAGE_HASH_KEY,
                       HASH_DIFF,
                       LDTS,
                       RCSR,
                       LAST_MODIFIED_BY;


---------adding other trade columns based on MS_recommended_t3

CREATE OR REPLACE temp TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY AS
SELECT 
    a.*,
    b.NCLH_RM_TRADE1,
    b.NCLH_RM_TRADE2,
    b."Market 1",
    b."Market 2",
    b."Trade_3_MS" AS NCLH_RM_TRADE3,
    b."Trade_4_MS" AS NCLH_RM_TRADE4
FROM PP_RSS_MS_Recommended_Trade a
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_TRADE_MARKET_HIRERACHY b
    ON upper(trim(a.MS_Recommended_T3)) = upper(trim(b."MS_recommended_T3" ))
WHERE a.MS_Recommended_T3 != 'NA';

--select top 10 * from REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY

--SELECT coun(*) FROM GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY

/* 2) Add metadata/extra columns */
ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS VESSEL_IMO VARCHAR(50);

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS NCLH_SHIP_CODE VARCHAR(5);

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS SAIL_ID VARCHAR(50);

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS MD5_VOYAGE_HASH_KEY VARCHAR;

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS HASH_DIFF VARCHAR;

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS LDTS TIMESTAMP_NTZ(9);

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS RCSR VARCHAR(200);

ALTER TABLE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
  ADD COLUMN IF NOT EXISTS LAST_MODIFIED_BY VARCHAR;

/* 3) Populate VESSEL_IMO by joining ref ship (5L code = CruiseLineCode||ShipCode) */
UPDATE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY a
SET VESSEL_IMO = TO_VARCHAR(b.VESSEL_IMO_NO),NCLH_SHIP_CODE= b.ship_code_3l
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA b
WHERE REPLACE(SUBSTRING(a.VOYAGE_CODE, 1, 6), '-', '') = b."5L_SHIP_CODE";


-- /* 4) Map NCLH_SHIP_CODE from ship name (case-insensitive) */
-- UPDATE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY C
-- SET NCLH_SHIP_CODE = CASE SUBSTRING(C.voyage_code, 5, 2)
--     WHEN 'SX' THEN 'EXP'   -- Seven Seas Explorer
--     WHEN 'SN' THEN 'MAR'   -- Seven Seas Mariner
--     WHEN 'SP' THEN 'SPL'   -- Seven Seas Splendor
--     WHEN 'SV' THEN 'VOY'   -- Seven Seas Voyager
--     WHEN 'SS' THEN 'NAV'   -- Seven Seas Navigator
--     WHEN 'GD' THEN 'GRA'   -- Seven Seas Grandeur
--     WHEN 'D1' THEN 'PRT'   -- Seven Seas Prestige
--     ELSE NCLH_SHIP_CODE
-- END;


/* 5) Compute SAIL_ID and key hash */
UPDATE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
SET SAIL_ID = CONCAT(NCLH_SHIP_CODE, TO_CHAR(EmbarkationDate, 'YYMMDD'));

UPDATE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
SET MD5_VOYAGE_HASH_KEY = TO_VARCHAR(MD5(SAIL_ID));

/* 6) Set load metadata */
UPDATE REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
SET LDTS = CURRENT_TIMESTAMP(),
    LAST_MODIFIED_BY = CURRENT_USER(),
    RCSR = 'RSS|STEP_PRE_SCORING|OUTPUT|TRADE_HIERARCHY';
    

-- Create or replace table VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY as 

TRUNCATE TABLE VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_HIERARACHY;
INSERT INTO VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_HIERARACHY
select * from REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY;

-- select top 10 * from VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY

-- select top 10 * from REF_GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY


-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY
-- ADD PRIMARY KEY (VOYAGE_CODE);





## Part2_RSS_Competitor_To_Dashboard

Transformation process to prepare RSS and competitor data for the dashboard tables.

In [ ]:
CREATE OR REPLACE TEMP TABLE da1 AS
SELECT *
FROM VESSOPS_D.L10_RDV.GCI_NCH_INTER_REF_PRICING_DATA
WHERE 
    upper(trim(ShopFrequency)) IN ('D', 'W', 'M')
    AND TRY_TO_DATE(TRIM(delivery_date), 'YYYY-MM-DD') > $filter_date
    AND UPPER(TRIM(RateType)) = 'DOUBLE'
    AND UPPER(TRIM(cruiselinecode)) IN ('RSS', 'SIL', 'SEA', 'CRY', 'RCT', 'PON', 'HPG', 'VOC', 'EXP');


-- select * from da1 where embarkationdate='Invalid date'

-- SELECT * FROM DA1 WHERE TRY_TO_DATE(embarkationdate::string) IS NULL AND embarkationdate IS NOT NULL

-- select * from GCI_NCH_RAW_pricing_rg_pass where cachepagelink='http://PGCruise-CP010.rategain.com/CP3/4/20105/4563/1587/135/EXC0001_53_5295070571.HTML'

--select * from GCI_NCH_RAW_ref_ship_data where brand_category='NCLH' or brand_folio='NCLH'

-- SELECT *
-- FROM VESSOPS_D.L00_STG.GCI_NCH_RAW_REF_SHIP_DATA_MATILLION
-- LIMIT 10;

-- Reassign ownership of the table
-- ALTER TABLE VESSOPS_D.L00_STG.GCI_NCH_RAW_REF_SHIP_DATA_MATILLION
--   OWNER TO ROLE SYSADMINDEV;

--select top 10 * from da1;

CREATE OR REPLACE TEMP TABLE t0015 AS
SELECT 
    CRUISELINECODE,
    shipcode,
    embarkationDate,
    embarkationportUN_LOCODE,
    Day_of_Week,
    Week_of_Year,
    NCLH_UNI_VOYAGE_CODE,
    ' ' AS RM_ROLLUP_PRODUCT_DESC,
    ' ' AS PRODUCT_DESC,
    NCL_UNI_VOYAGE_CODE,
    DisEmbarkationDate,
    DisembarkationPortUN_LOCODE,
    TaxExclusiveRatePP,
    CabinType,
    RateInquiryDate,
    CachePageLink,
    ShopFrequency,
    LastShoppedSubCategory,
    RatePPUSD,
    TaxesandFees,
    TaxesandFeesUSD,
    RateType,
    TaxInclusiveRatePP,
    AvailableOffer,
    Availability,
    ConvertionRate,
    SourceCurrency,
    SourceMarket,
    OrigCurrency,
    "OBSTRUCTED_Y_N",
    "Gratuity (Y/N)",
    categorycode,
    "FARE_PASSENGERTYPE",
    ldts as created_date,
    File_NM,
    delivery_date,
    NCL_CABINTYPE,
    OCI_CABINTYPE,
    RSS_CABINTYPE
FROM da1;


CREATE OR REPLACE TEMP TABLE j1 AS
SELECT 
    NCLH_UNI_VOYAGE_CODE,
    DATEDIFF(WEEK, TO_DATE(delivery_date), TO_DATE(embarkationDate)) AS Adv_Week,
    Availability,
    AvailableOffer,
    "Market" AS Brand_Market,
    "Segment" AS Brand_Segment,
    "Sub-Segment" AS Brand_Sub_Segment,
    CabinType,
    CachePageLink,
    categorycode,
    'CRUISE ONLY' AS CruiseType,
    DATEDIFF(DAY, TO_DATE(embarkationDate), TO_DATE(DisEmbarkationDate)) AS CruiseDuration,

    b.CRUISE_LINE_CODE AS CRUISE_LINE_CODE,
    b.BRAND_NAME AS Cruise_Line_Name,
    b.VESSEL_IMO_NO,
    b.ship_code_3l,

    (TaxExclusiveRatePP / NULLIF(DATEDIFF(DAY, TO_DATE(embarkationDate), TO_DATE(DisEmbarkationDate)), 0)) AS TaxExclusiveRatePPD,
    (TaxInclusiveRatePP / NULLIF(DATEDIFF(DAY, TO_DATE(embarkationDate), TO_DATE(DisEmbarkationDate)), 0)) AS TaxInclusiveRatePPD,
    (TaxesandFees / NULLIF(DATEDIFF(DAY, TO_DATE(embarkationDate), TO_DATE(DisEmbarkationDate)), 0)) AS TaxesandFeesPPD,

    RM_ROLLUP_PRODUCT_DESC,
    PRODUCT_DESC,
    RateType,

    ' ' AS GCI_Trade_Level_1,
    ' ' AS GCI_Trade_Level_2,
    ' ' AS GCI_Trade_Level_3,
    ' ' AS GCI_Trade_Level_4,
    ' ' AS NCL_Trade_1,
    ' ' AS NCL_Trade_2,
    ' ' AS NCL_Trade_3,
    ' ' AS OCI_Trade_1,
    ' ' AS OCI_Trade_2,
    ' ' AS OCI_Trade_3,
    ' ' AS RSSC_Trade_1,
    ' ' AS RSSC_Trade_2,
    ' ' AS RSSC_Trade_3,

    DisEmbarkationDate,
    d.PORT_LOC_NAME AS Disembark_Port_Name,
    DisembarkationPortUN_LOCODE AS DisEmbark_Port_Code_GCI,
    d.Seaware_Port_Code AS Disembark_Port_Code_NCL,
    d.NVS_Port_Code AS Disembark_Port_Code_OCI,
    d.NVS_Port_Code AS Disembark_Port_Code_RSSC,

    CASE 
        WHEN CruiseDuration BETWEEN 1 AND 5 THEN '1-5 days'
        WHEN CruiseDuration BETWEEN 6 AND 12 THEN '6-12 days'
        WHEN CruiseDuration BETWEEN 13 AND 20 THEN '13-20 days'
        WHEN CruiseDuration >= 21 THEN '21+ days'
    END AS Duration_Level_1,

    CASE 
        WHEN CruiseDuration BETWEEN 1 AND 2 THEN '1-2 days'
        WHEN CruiseDuration BETWEEN 3 AND 5 THEN '3-5 days'
        WHEN CruiseDuration BETWEEN 6 AND 8 THEN '6-8 days'
        WHEN CruiseDuration BETWEEN 9 AND 12 THEN '9-12 days'
        WHEN CruiseDuration BETWEEN 13 AND 15 THEN '13-15 days'
        WHEN CruiseDuration BETWEEN 16 AND 20 THEN '16-20 days'
        WHEN CruiseDuration BETWEEN 21 AND 30 THEN '21-30 days'
        WHEN CruiseDuration BETWEEN 31 AND 50 THEN '31-50 days'
        WHEN CruiseDuration >= 51 THEN '51+ days'
    END AS Duration_Level_2,

    TO_DATE(embarkationDate) AS EmbarkationDate,
    c.PORT_LOC_NAME AS Embark_Port_Name,
    embarkationportUN_LOCODE AS Embark_Port_Code_GCI,
    c.Seaware_Port_Code AS Embark_Port_Code_NCL,
    c.NVS_Port_Code AS Embark_Port_Code_OCI,
    c.NVS_Port_Code AS Embark_Port_Code_RSSC,
    c."NEW_EMBARK_REGION_1" AS EMBARK_REGION_1,
    c."NEW_EMBARK_REGION_2" AS EMBARK_REGION_2,
    c."NEW_EMBARK_REGION_3" AS EMBARK_REGION_3,

    "FARE_PASSENGERTYPE",
    TO_DATE(delivery_date) AS Shop_Date,
    b."5L_SHIP_CODE" AS Ship_Code_GCI,
    b.NCLH_DW_SHIP_CD AS Ship_Code_NCL,
    b.NCLH_DW_SHIP_CD AS Ship_Code_OCI,
    b.NCLH_DW_SHIP_CD AS Ship_Code_RSSC,

    ' ' AS Itinerary_Port_String,
    ' ' AS Itinerary_Code_String,
    LastShoppedSubCategory,

    DATEADD(DAY, -1, TO_DATE(delivery_date)) AS Last_Shop_Date,
    CAST(created_date AS DATE) AS Load_Date,

    "OBSTRUCTED_Y_N" AS "Obstructed(Y/N)",
    RateInquiryDate,

    CASE 
        WHEN embarkationportUN_LOCODE = DisembarkationPortUN_LOCODE THEN 'Y'
        ELSE 'N'
    END AS "RountTrip(Y/N)",

    TO_CHAR(TO_DATE(embarkationDate), 'Day') AS Embark_DOW,
    Day_of_Week AS Embark_DOW_No,
    EXTRACT(MONTH FROM TO_DATE(embarkationDate)) AS Embark_Month,
    WEEK(TO_DATE(embarkationDate)) AS Embark_Week,
    EXTRACT(YEAR FROM TO_DATE(embarkationDate)) AS Embark_Year,

    CASE 
        WHEN b.TOTAL_ALBS BETWEEN 1 AND 1000 THEN 'Small'
        WHEN b.TOTAL_ALBS BETWEEN 1001 AND 2500 THEN 'Medium'
        WHEN b.TOTAL_ALBS BETWEEN 2501 AND 4000 THEN 'Large'
        WHEN b.TOTAL_ALBS >= 4001 THEN 'Mega'
    END AS SHIP_SIZE,

    TO_CHAR(TO_DATE(delivery_date), 'Day') AS Shop_DOW,
    DAYOFWEEK(TO_DATE(delivery_date)) AS Shop_Dow_No,
    EXTRACT(MONTH FROM TO_DATE(delivery_date)) AS Shop_Month,
    WEEK(TO_DATE(delivery_date)) AS Shop_Week,
    EXTRACT(YEAR FROM TO_DATE(delivery_date)) AS Shop_Year,

    ShopFrequency,
    SourceCurrency,
    ' ' AS SourceItineraryName,
    SourceMarket,

    b.SHIP_NAME AS Ship_Name_GCI,
    a.File_NM AS Src_File_Name,
    TaxesandFees,
    TaxExclusiveRatePP,
    TaxInclusiveRatePP,
    "Gratuity (Y/N)",
    ' ' AS Website,

    CASE 
        WHEN TO_CHAR(TO_DATE(embarkationDate), 'Day') IN ('Sunday', 'Saturday') THEN 'Y'
        ELSE 'N'
    END AS WEEKEND_EMBARK,

    ConvertionRate,
    RatePPUSD,
    TaxesandFeesUSD,
    OrigCurrency,
    NCL_CABINTYPE,
    OCI_CABINTYPE,
    RSS_CABINTYPE

FROM t0015 a
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA b
    ON UPPER(TRIM(CONCAT(a.cruiselinecode, a.shipcode))) = UPPER(TRIM(b."5L_SHIP_CODE"))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA c
    ON UPPER(TRIM(a.embarkationportUN_LOCODE)) = UPPER(TRIM(c.UNV_PORT_CODE))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA d
    ON UPPER(TRIM(a.DisembarkationPortUN_LOCODE)) = UPPER(TRIM(d.UNV_PORT_CODE));


-- select top 10 * from j1 where ship_code_3l is null

ALTER TABLE j1
ADD COLUMN 
    SAIL_ID varchar(50),
    MD5_VOYAGE_HASH_KEY VARCHAR,
    HASH_DIFF VARCHAR,
    LDTS TIMESTAMP_NTZ(9),
    RCSR VARCHAR(200),
    LAST_MODIFIED_BY VARCHAR(500);



UPDATE j1
SET SAIL_ID = CONCAT(
    SHIP_CODE_3l,
    TO_CHAR(EMBARKATIONDATE, 'YYMMDD')
);

UPDATE j1
SET MD5_VOYAGE_HASH_KEY = MD5(SAIL_ID);

UPDATE j1
SET 
    LDTS = CURRENT_TIMESTAMP(),
    RCSR = 'RG_REF_PRICING_DATA|TB_PORT|TB_SHIP ',
    LAST_MODIFIED_BY = CURRENT_USER();

UPDATE J1
SET HASH_DIFF = MD5(
      NVL(NCLH_UNI_VOYAGE_CODE, '') || '|' ||
      NVL(CAST(ADV_WEEK AS STRING), '') || '|' ||
      NVL(CAST(AVAILABILITY AS STRING), '') || '|' ||
      NVL(AVAILABLEOFFER, '') || '|' ||
      NVL(BRAND_MARKET, '') || '|' ||
      NVL(BRAND_SEGMENT, '') || '|' ||
      NVL(BRAND_SUB_SEGMENT, '') || '|' ||
      NVL(CABINTYPE, '') || '|' ||
      NVL(CACHEPAGELINK, '') || '|' ||
      NVL(CATEGORYCODE, '') || '|' ||
      NVL(CRUISETYPE, '') || '|' ||
      NVL(CAST(CRUISEDURATION AS STRING), '') || '|' ||
      NVL(CRUISE_LINE_CODE, '') || '|' ||
      NVL(CRUISE_LINE_NAME, '') || '|' ||
      NVL(CAST(VESSEL_IMO_NO AS STRING), '') || '|' ||
      NVL(SHIP_CODE_3L, '') || '|' ||
      NVL(CAST(TAXEXCLUSIVERATEPPD AS STRING), '') || '|' ||
      NVL(CAST(TAXINCLUSIVERATEPPD AS STRING), '') || '|' ||
      NVL(CAST(TAXESANDFEESPPD AS STRING), '') || '|' ||
      NVL(RM_ROLLUP_PRODUCT_DESC, '') || '|' ||
      NVL(PRODUCT_DESC, '') || '|' ||
      NVL(RATETYPE, '') || '|' ||
      NVL(GCI_TRADE_LEVEL_1, '') || '|' ||
      NVL(GCI_TRADE_LEVEL_2, '') || '|' ||
      NVL(GCI_TRADE_LEVEL_3, '') || '|' ||
      NVL(GCI_TRADE_LEVEL_4, '') || '|' ||
      NVL(NCL_TRADE_1, '') || '|' ||
      NVL(NCL_TRADE_2, '') || '|' ||
      NVL(NCL_TRADE_3, '') || '|' ||
      NVL(OCI_TRADE_1, '') || '|' ||
      NVL(OCI_TRADE_2, '') || '|' ||
      NVL(OCI_TRADE_3, '') || '|' ||
      NVL(RSSC_TRADE_1, '') || '|' ||
      NVL(RSSC_TRADE_2, '') || '|' ||
      NVL(RSSC_TRADE_3, '') || '|' ||
      NVL(TO_VARCHAR(DISEMBARKATIONDATE), '') || '|' ||
      NVL(DISEMBARK_PORT_NAME, '') || '|' ||
      NVL(DISEMBARK_PORT_CODE_GCI, '') || '|' ||
      NVL(DISEMBARK_PORT_CODE_NCL, '') || '|' ||
      NVL(DISEMBARK_PORT_CODE_OCI, '') || '|' ||
      NVL(DISEMBARK_PORT_CODE_RSSC, '') || '|' ||
      NVL(DURATION_LEVEL_1, '') || '|' ||
      NVL(DURATION_LEVEL_2, '') || '|' ||
      NVL(TO_VARCHAR(EMBARKATIONDATE), '') || '|' ||
      NVL(EMBARK_PORT_NAME, '') || '|' ||
      NVL(EMBARK_PORT_CODE_GCI, '') || '|' ||
      NVL(EMBARK_PORT_CODE_NCL, '') || '|' ||
      NVL(EMBARK_PORT_CODE_OCI, '') || '|' ||
      NVL(EMBARK_PORT_CODE_RSSC, '') || '|' ||
      NVL(EMBARK_REGION_1, '') || '|' ||
      NVL(EMBARK_REGION_2, '') || '|' ||
      NVL(EMBARK_REGION_3, '') || '|' ||
      NVL(FARE_PASSENGERTYPE, '') || '|' ||
      NVL(TO_VARCHAR(SHOP_DATE), '') || '|' ||
      NVL(SHIP_CODE_GCI, '') || '|' ||
      NVL(SHIP_CODE_NCL, '') || '|' ||
      NVL(SHIP_CODE_OCI, '') || '|' ||
      NVL(SHIP_CODE_RSSC, '') || '|' ||
      NVL(ITINERARY_PORT_STRING, '') || '|' ||
      NVL(ITINERARY_CODE_STRING, '') || '|' ||
      NVL(LASTSHOPPEDSUBCATEGORY, '') || '|' ||
      NVL(TO_VARCHAR(LAST_SHOP_DATE), '') || '|' ||
      NVL(TO_VARCHAR(LOAD_DATE), '') || '|' ||
      NVL("Obstructed(Y/N)", '') || '|' ||
      NVL(TO_VARCHAR(RATEINQUIRYDATE), '') || '|' ||
      NVL("RountTrip(Y/N)", '') || '|' ||
      NVL(EMBARK_DOW, '') || '|' ||
      NVL(CAST(EMBARK_DOW_NO AS STRING), '') || '|' ||
      NVL(CAST(EMBARK_MONTH AS STRING), '') || '|' ||
      NVL(CAST(EMBARK_WEEK AS STRING), '') || '|' ||
      NVL(CAST(EMBARK_YEAR AS STRING), '') || '|' ||
      NVL(SHIP_SIZE, '') || '|' ||
      NVL(SHOP_DOW, '') || '|' ||
      NVL(CAST(SHOP_DOW_NO AS STRING), '') || '|' ||
      NVL(CAST(SHOP_MONTH AS STRING), '') || '|' ||
      NVL(CAST(SHOP_WEEK AS STRING), '') || '|' ||
      NVL(CAST(SHOP_YEAR AS STRING), '') || '|' ||
      NVL(CAST(SHOPFREQUENCY AS STRING), '') || '|' ||
      NVL(SOURCECURRENCY, '') || '|' ||
      NVL(SOURCEITINERARYNAME, '') || '|' ||
      NVL(SOURCEMARKET, '') || '|' ||
      NVL(SHIP_NAME_GCI, '') || '|' ||
      NVL(SRC_FILE_NAME, '') || '|' ||
      NVL(CAST(TAXESANDFEES AS STRING), '') || '|' ||
      NVL(CAST(TAXEXCLUSIVERATEPP AS STRING), '') || '|' ||
      NVL(CAST(TAXINCLUSIVERATEPP AS STRING), '') || '|' ||
      NVL("Gratuity (Y/N)", '') || '|' ||
      NVL(WEBSITE, '') || '|' ||
      NVL(WEEKEND_EMBARK, '') || '|' ||
      NVL(CAST(CONVERTIONRATE AS STRING), '') || '|' ||
      NVL(CAST(RATEPPUSD AS STRING), '') || '|' ||
      NVL(CAST(TAXESANDFEESUSD AS STRING), '') || '|' ||
      NVL(ORIGCURRENCY, '') || '|' ||
      NVL(NCL_CABINTYPE, '') || '|' ||
      NVL(OCI_CABINTYPE, '') || '|' ||
      NVL(RSS_CABINTYPE, '') || '|' ||
      NVL(SAIL_ID, '')
);


-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_COMP_BRANDS 
--     ALTER COLUMN NCL_CABINTYPE SET DATA TYPE VARCHAR(500);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_COMP_BRANDS 
--     ALTER COLUMN RSS_CABINTYPE SET DATA TYPE VARCHAR(500);

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_COMP_BRANDS 
--     ALTER COLUMN OCI_CABINTYPE SET DATA TYPE VARCHAR(500);

-- Create or replace table VESSOPS_D.L10_RDV.GCI_INTER_PP_RSS_COMP_BRANDS as 
--TRUNCATE table VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_COMP_BRANDS;
INSERT INTO VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_COMP_BRANDS
select * from j1;

-- select top 10 * from GCI_RSS_INTER_PRICING_COMP_BRANDS

-- select * from VESSOPS_D.L10_RDV.GCI_INTER_PP_RSS_COMP_BRANDS where nclh_uni_voyage_code='PON-EC-2024-11-12-MQFDF-MQFDF-8' and shop_date='2024-03-04' and cabintype='Balcony'

-- ALTER TABLE VESSOPS_D.L10_RDV.GCI_INTER_PP_RSS_COMP_BRANDS
-- ADD CONSTRAINT PK_GCI_INTER_PP_RSS_COMP_BRANDS
-- PRIMARY KEY (
--     NCLH_UNI_VOYAGE_CODE,
--     SHOP_DATE,
--     CABINTYPE,
--     TAXINCLUSIVERATEPP
-- );



CREATE OR REPLACE TEMP TABLE VOYAGE_MIN_RID AS
SELECT 
    CONCAT(
        bs.mp_CruiseLineCode, '-',
        bs.mp_ShipCode, '-',
        TO_CHAR(TO_DATE(pric.EmbarkationDate), 'YYYY-MM-DD'), '-',
        dt.Tb_Unv_Port_Code, '-',
        dtp.Tb_Unv_Port_Code, '-',
        pric.CruiseDuration
    ) AS NCLH_UNI_VOYAGE_CODE,
    
    MIN(TO_DATE(pric.RateInquiryDate)) AS Min_RateInquiryDate

FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRICING_RG_PASS pric
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_SHIP_DATA bs
    ON UPPER(TRIM(pric.cruiselinecode)) = UPPER(TRIM(bs.cruiselinecode))
    AND UPPER(TRIM(pric.shipcode)) = UPPER(TRIM(bs.shipcode))
    AND UPPER(TRIM(pric.CruiseLineName)) = UPPER(TRIM(bs.CruiseLineName))
    AND UPPER(TRIM(pric.SourceShipName)) = UPPER(TRIM(bs.SourceShipName))

LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA dt
    ON UPPER(TRIM(pric.embarkationportun_locode)) = UPPER(TRIM(dt.Dtl_PortUN_Locode))
    AND UPPER(TRIM(pric.embarkationport)) = UPPER(TRIM(dt.Dtl_Port_City_Nam))

LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA dtp
    ON UPPER(TRIM(pric.DisembarkationPortUN_LOCODE)) = UPPER(TRIM(dtp.Dtl_PortUN_Locode))
    AND UPPER(TRIM(pric.DisembarkationPort)) = UPPER(TRIM(dtp.Dtl_Port_City_Nam))

WHERE 
    UPPER(TRIM(pric.cruiselinecode)) IN ('RSS', 'SIL', 'SEA', 'CRY', 'RCT', 'PON', 'HPG', 'VOC', 'EXP')

GROUP BY 
    CONCAT(
        bs.mp_CruiseLineCode, '-',
        bs.mp_ShipCode, '-',
        TO_CHAR(TO_DATE(pric.EmbarkationDate), 'YYYY-MM-DD'), '-',
        dt.Tb_Unv_Port_Code, '-',
        dtp.Tb_Unv_Port_Code, '-',
        pric.CruiseDuration
    );

--select count(*) from voyage_min_rid

-- Step 1: Create TEMP TABLE p2 from Pricing_RG_Pass

CREATE OR REPLACE TEMP TABLE p2 AS
SELECT 
    TO_DATE(pric.RateInquiryDate) AS RateInquiryDate,
    CONCAT(
        bs.mp_CruiseLineCode, '-',
        bs.mp_ShipCode, '-',
        TO_CHAR(TO_DATE(pric.EmbarkationDate), 'YYYY-MM-DD'), '-',
        dt.Tb_Unv_Port_Code, '-',
        dtp.Tb_Unv_Port_Code, '-',
        pric.CruiseDuration
    ) AS NCLH_UNI_VOYAGE_CODE,
    pric.SourceItineraryName,
    pric.ItineraryPortContent
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Pricing_RG_Pass pric
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_SHIP_DATA bs
    ON UPPER(TRIM(pric.cruiselinecode)) = UPPER(TRIM(bs.cruiselinecode))
    AND UPPER(TRIM(pric.shipcode)) = UPPER(TRIM(bs.shipcode))
    AND UPPER(TRIM(pric.CruiseLineName)) = UPPER(TRIM(bs.CruiseLineName))
    AND UPPER(TRIM(pric.SourceShipName)) = UPPER(TRIM(bs.SourceShipName))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA dt
    ON UPPER(TRIM(pric.embarkationportun_locode)) = UPPER(TRIM(dt.Dtl_PortUN_Locode))
    AND UPPER(TRIM(pric.embarkationport)) = UPPER(TRIM(dt.Dtl_Port_City_Nam))
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA dtp
    ON UPPER(TRIM(pric.DisembarkationPortUN_LOCODE)) = UPPER(TRIM(dtp.Dtl_PortUN_Locode))
    AND UPPER(TRIM(pric.DisembarkationPort)) = UPPER(TRIM(dtp.Dtl_Port_City_Nam))
WHERE 
    UPPER(TRIM(pric.ShopFrequency)) IN ('D', 'W', 'M')
    AND TRY_TO_DATE(SUBSTRING(pric.FilePath, 42, 10), 'YYYY-MM-DD') >= '2020-10-01'
    AND DAYOFWEEK(TRY_TO_DATE(SUBSTRING(pric.FilePath, 42, 10), 'YYYY-MM-DD')) = 1 -- Monday
    AND UPPER(TRIM(pric.RateType)) = 'DOUBLE'
    AND UPPER(TRIM(pric.cruiselinecode)) IN ('RSS', 'SIL', 'SEA', 'CRY', 'RCT', 'PON', 'HPG', 'VOC', 'EXP');

-- Step 2: Deduplicate using ROW_NUMBER and store in TEMP TABLE px

CREATE OR REPLACE TEMP TABLE VOYAGE_LATEST_SIN_IPC AS
WITH cte AS (
    SELECT 
        NCLH_UNI_VOYAGE_CODE,
        SourceItineraryName,
        ItineraryPortContent,
        ROW_NUMBER() OVER (
            PARTITION BY NCLH_UNI_VOYAGE_CODE
            ORDER BY RateInquiryDate, NCLH_UNI_VOYAGE_CODE DESC
        ) AS row1
    FROM p2
)
SELECT *
FROM cte
WHERE row1 = 1;

--select * from VOYAGE_LATEST_SIN_IPC where SOURCEITINERARYNAME is null
--select top 1 * from  GCI_INTER_PP_RSS_COMP_BRANDS

CREATE OR REPLACE TEMP TABLE PP_RSS_Pricing_collated_RID_SIN_IPC AS
SELECT 
    a.NCLH_UNI_VOYAGE_CODE,
    Adv_Week,
    Availability,
    AvailableOffer,
    Brand_Market,
    Brand_Segment,
    Brand_Sub_Segment,
    CabinType,
    CachePageLink,
    categorycode,
    CruiseType,
    CruiseDuration,
    CRUISE_LINE_CODE,
    Cruise_Line_Name,
    VESSEL_IMO_NO,
    SHIP_CODE_3L, -----------------------------
    CAST(TaxExclusiveRatePPD AS DECIMAL(10, 2)) AS TaxExclusiveRatePPD,
    CAST(TaxInclusiveRatePPD AS DECIMAL(10, 2)) AS TaxInclusiveRatePPD,
    CAST(TAXESANDFEESPPD AS DECIMAL(10, 2)) AS TaxesandFeesPPD,---------------
    RM_ROLLUP_PRODUCT_DESC,
    PRODUCT_DESC,
    RateType,
    GCI_Trade_Level_1,
    GCI_Trade_Level_2,
    GCI_Trade_Level_3,
    GCI_Trade_Level_4,
    NCL_Trade_1,
    NCL_Trade_2,
    NCL_Trade_3,
    OCI_Trade_1,
    OCI_Trade_2,
    OCI_Trade_3,
    RSSC_Trade_1,
    RSSC_Trade_2,
    RSSC_Trade_3,
    TO_DATE(DisEmbarkationDate) AS DisEmbarkationDate,
    Disembark_Port_Name,
    DisEmbark_Port_Code_GCI,
    DisEmbark_Port_Code_NCL,
    DisEmbark_Port_Code_OCI,
    DisEmbark_Port_Code_RSSC,
    Duration_Level_1,
    Duration_Level_2,
    EmbarkationDate,
    Embark_Port_Name,
    Embark_Port_Code_GCI,
    Embark_Port_Code_NCL,
    Embark_Port_Code_OCI,
    Embark_Port_Code_RSSC,
    EMBARK_REGION_1,
    EMBARK_REGION_2,
    EMBARK_REGION_3,
    "FARE_PASSENGERTYPE",
    TO_DATE(Shop_Date) AS Shop_Date,
    Ship_Code_GCI,
    Ship_Code_NCL,
    Ship_Code_OCI,
    Ship_Code_RSSC,
    Itinerary_Code_String,
    LastShoppedSubCategory,
    TO_DATE(Last_Shop_Date) AS Last_Shop_Date,
    TO_DATE(Load_Date) AS Load_Date,
    "Obstructed(Y/N)",
    TO_DATE(RateInquiryDate) AS RateInquiryDate,
    "RountTrip(Y/N)",
    Embark_DOW,
    Embark_DOW_No,
    Embark_Month,
    Embark_Week,
    Embark_Year,
    SHIP_SIZE,
    Shop_DOW,
    Shop_Dow_No,
    Shop_Month,
    Shop_Week,
    Shop_Year,
    ShopFrequency,
    SourceCurrency,
    SourceMarket,
    Ship_Name_GCI,
    Src_File_Name,
    TaxesandFees,
    TaxExclusiveRatePP,
    TaxInclusiveRatePP,
    "Gratuity (Y/N)",
    Website,
    WEEKEND_EMBARK,
    ConvertionRate,
    RatePPUSD,
    TaxesandFeesUSD,
    OrigCurrency,
    NCL_CABINTYPE,     ---
    OCI_CABINTYPE,  ----------
    RSS_CABINTYPE,   ------------
    SAIL_ID,         ------------
    MD5_VOYAGE_HASH_KEY, ----------
    b.SourceItineraryName,
    b.ItineraryPortContent,
    TO_DATE(c.Min_RateInquiryDate) AS Min_RateInquiryDate,
    HASH_DIFF,
    current_timestamp as LDTS,
    'RSS_COMP_BRANDS | RG_PRICING_PASS | RSS TRADE' as RCSR,
    current_user as LAST_MODIFIED_BY

FROM VESSOPS_D.L10_RDV.GCI_RSS_INTER_PRICING_COMP_BRANDS a
LEFT JOIN Voyage_Latest_SIN_IPC b
    ON UPPER(TRIM(a.NCLH_UNI_VOYAGE_CODE)) = UPPER(TRIM(b.NCLH_UNI_VOYAGE_CODE))
LEFT JOIN Voyage_MIN_RID c
    ON UPPER(TRIM(a.NCLH_UNI_VOYAGE_CODE)) = UPPER(TRIM(c.NCLH_UNI_VOYAGE_CODE));


--select count(*) from PP_RSS_Pricing_collated_RID_SIN_IPC--3456075


-- select * from GCI_INTER_PP_RSS_PRICING_COLLATED_RID_SIN_IPC_TRADE
-- where Embark_Port_Code_GCI is null;


-- select * from GCI_INTER_PP_RSS_PRICING_COLLATED_RID_SIN_IPC_TRADE
-- where GCI_Trade_Level_2 is null

-- select * from GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY where nclh_rm_trade1 is null and voyage_code ='PON-SO-2022-11-22-NCNOU-NZAKL-10'


-- delete from GCI_INTER_PP_RSS_PRICING_COLLATED_RID_SIN_IPC_TRADE 
-- where NCLH_UNI_VOYAGE_CODE ='PON-SO-2022-11-22-NCNOU-NZAKL-10';
--select * from GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY where nclh_rm_trade1 is null
CREATE OR REPLACE TEMP TABLE GCI_Inter_PP_RSS_Pricing_collated_RID_SIN_IPC_Trade AS
  SELECT
      /* a.* */
      a.NCLH_UNI_VOYAGE_CODE,
      a.Adv_Week,
      a.Availability,
      a.AvailableOffer,
      a.Brand_Market,
      a.Brand_Segment,
      a.Brand_Sub_Segment,
      a.CabinType,
      a.CachePageLink,
      a.categorycode,
      a.CruiseType,
      a.CruiseDuration,
      a.CRUISE_LINE_CODE,
      a.Cruise_Line_Name,
      a.VESSEL_IMO_NO,
      a.SHIP_CODE_3L,
      a.TaxExclusiveRatePPD,
      a.TaxInclusiveRatePPD,
      a.TaxesandFeesPPD,
      a.RM_ROLLUP_PRODUCT_DESC,
      a.PRODUCT_DESC,
      a.RateType,

      /* trade hierarchy (left join; may be NULL for non-RSS) */
      b.NCLH_RM_TRADE1 AS GCI_Trade_Level_1,
      b.NCLH_RM_TRADE2 AS GCI_Trade_Level_2,
      b.NCLH_RM_TRADE3 AS GCI_Trade_Level_3,
      b.NCLH_RM_TRADE4 AS GCI_Trade_Level_4,
      b."Market 1",
      b."Market 2",

      a.NCL_Trade_1,
      a.NCL_Trade_2,
      a.NCL_Trade_3,

      b.NCLH_RM_TRADE1 AS OCI_Trade_1,
      b.NCLH_RM_TRADE2 AS OCI_Trade_2,
      b.NCLH_RM_TRADE3 AS OCI_Trade_3,
      b.NCLH_RM_TRADE4 AS OCI_Trade_4,

      a.RSSC_Trade_1,
      a.RSSC_Trade_2,
      a.RSSC_Trade_3,

      a.DisEmbarkationDate,
      a.Disembark_Port_Name,
      a.DisEmbark_Port_Code_GCI,
      a.DisEmbark_Port_Code_NCL,
      a.DisEmbark_Port_Code_OCI,
      a.DisEmbark_Port_Code_RSSC,
      a.Duration_Level_1,
      a.Duration_Level_2,
      a.EmbarkationDate,
      a.Embark_Port_Name,
      a.Embark_Port_Code_GCI,
      a.Embark_Port_Code_NCL,
      a.Embark_Port_Code_OCI,
      a.Embark_Port_Code_RSSC,
      a.EMBARK_REGION_1,
      a.EMBARK_REGION_2,
      a.EMBARK_REGION_3,
      a."FARE_PASSENGERTYPE",
      a.Shop_Date,
      a.Ship_Code_GCI,
      a.Ship_Code_NCL,
      a.Ship_Code_OCI,
      a.Ship_Code_RSSC,
      a.LastShoppedSubCategory,
      a.Last_Shop_Date,
      a.Load_Date,
      a."Obstructed(Y/N)",
      a.RateInquiryDate,
      a."RountTrip(Y/N)",
      a.Embark_DOW,
      a.Embark_DOW_No,
      a.Embark_Month,
      a.Embark_Week,
      a.Embark_Year,
      a.SHIP_SIZE,
      a.Shop_DOW,
      a.Shop_Dow_No,
      a.Shop_Month,
      a.Shop_Week,
      a.Shop_Year,
      a.ShopFrequency,
      a.SourceCurrency,
      a.SourceMarket,
      a.Ship_Name_GCI,
      a.Src_File_Name,
      a.TaxesandFees,
      a.TaxExclusiveRatePP,
      a.TaxInclusiveRatePP,
      a."Gratuity (Y/N)",
      a.Website,
      a.WEEKEND_EMBARK,
      a.ConvertionRate,
      a.RatePPUSD,
      a.TaxesandFeesUSD,
      a.OrigCurrency,
      a.SourceItineraryName,
      a.ItineraryPortContent,
      a.Min_RateInquiryDate,

      b.WORLD_FLAG,
      'NA' AS Supermkt,
      CASE 
        WHEN UPPER(TRIM(a.CRUISE_LINE_CODE)) = 'RSS' AND NVL(a.Ship_Code_RSSC,'') <> '' THEN
          CONCAT(
            a.Ship_Code_RSSC,
            SUBSTRING(TO_VARCHAR(a.EmbarkationDate), 3, 2),
            SUBSTRING(TO_VARCHAR(a.EmbarkationDate), 6, 2),
            SUBSTRING(TO_VARCHAR(a.EmbarkationDate), 9, 2)
          )
        ELSE NULL
      END AS Regent_Cruise_CD,

      a.NCL_CABINTYPE,
      a.OCI_CABINTYPE,
      a.RSS_CABINTYPE,

      /* keys/meta */
      a.SAIL_ID,
      a.MD5_VOYAGE_HASH_KEY,
      a.HASH_DIFF,
      a.LDTS,
      a.RCSR,
      a.LAST_MODIFIED_BY

  FROM PP_RSS_Pricing_collated_RID_SIN_IPC a
  LEFT JOIN VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_HIERARACHY b
         ON UPPER(TRIM(a.NCLH_UNI_VOYAGE_CODE)) = UPPER(TRIM(b.voyage_code))
   WHERE UPPER(TRIM(a.NCLH_UNI_VOYAGE_CODE)) IN (
    SELECT DISTINCT UPPER(TRIM(voyage_code)) 
    FROM VESSOPS_D.L10_RDV.GCI_RSS_INTER_REF_TRADE_HIERARACHY
);

-- select count(*) from VESSOPS_D.L00_STG.GCI_INTER_REF_PP_RSS_TRADE_HIERARACHY

-- select  count(*) from GCI_INTER_PP_RSS_PRICING_COLLATED_RID_SIN_IPC_TRADE where nclh_uni_voyage_code='RSS-SS-2023-10-29-GRPIR-TRIST-12' and shop_date='2023-07-04'

-- select  count(*) from vessops_d.l00_stg.GCI_CONSUMPTION_PP_RSS_DASHBOARD where nclh_uni_voyage_code='RSS-SS-2023-10-29-GRPIR-TRIST-12' and shop_date='2023-07-04', cabintype,rss_cabintype ,taxinclsiveratepp


-- select distinct cruise_line_code from GCI_INTER_PP_RSS_PRICING_COLLATED_RID_SIN_IPC_TRADE ;

DELETE FROM GCI_Inter_PP_RSS_Pricing_collated_RID_SIN_IPC_Trade
WHERE NCLH_UNI_VOYAGE_CODE = 'PON-SO-2022-11-22-NCNOU-NZAKL-10';--399


ALTER TABLE GCI_Inter_PP_RSS_Pricing_collated_RID_SIN_IPC_Trade
        ADD taxexclusive_regent_air_rate_pp DECIMAL(28,10),
            taxexclusive_regent_air_rate_ppd DECIMAL(28,10),
            taxinclusive_regent_air_rate_pp DECIMAL(28,10),
            taxinclusive_regent_air_rate_ppd DECIMAL(28,10);


UPDATE GCI_Inter_PP_RSS_Pricing_collated_RID_SIN_IPC_Trade
SET 
    taxinclusive_regent_air_rate_pp = CASE 
        WHEN taxinclusiveratepp = -1 THEN -1
        ELSE (taxinclusiveratepp - CAST(REPLACE(AvailableOffer, ',', '') AS DECIMAL(28,10)))
    END,
    taxexclusive_regent_air_rate_pp = CASE 
        WHEN taxexclusiveratepp = -1 THEN -1
        ELSE (taxexclusiveratepp - CAST(REPLACE(AvailableOffer, ',', '') AS DECIMAL(28,10)))
    END,
    taxinclusive_regent_air_rate_ppd = CASE 
        WHEN taxinclusive_regent_air_rate_pp = -1 THEN -1
        ELSE (taxinclusive_regent_air_rate_pp / CruiseDuration)
    END,
    taxexclusive_regent_air_rate_ppd = CASE 
        WHEN taxexclusive_regent_air_rate_pp = -1 THEN -1
        ELSE (taxexclusive_regent_air_rate_pp / CruiseDuration)
    END
WHERE UPPER(TRIM(CRUISE_LINE_CODE)) = 'RSS'
  AND RateInquiryDate > '2022-08-01'
  AND AvailableOffer IS NOT NULL;




-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
--     ALTER COLUMN NCL_CABINTYPE SET DATA TYPE VARCHAR(500);

-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD 
--     ALTER COLUMN RSS_CABINTYPE SET DATA TYPE VARCHAR(500);

-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
--     ALTER COLUMN OCI_CABINTYPE SET DATA TYPE VARCHAR(500);


--create or replace table VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_REF_PRICING_DASHBOARD as 
TRUNCATE Table VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD;
insert into VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
select distinct * from GCI_Inter_PP_RSS_Pricing_collated_RID_SIN_IPC_Trade;

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
SET HASH_DIFF = MD5(
    COALESCE(NCLH_UNI_VOYAGE_CODE,'') || '|' ||
    COALESCE(TO_VARCHAR(ADV_WEEK),'') || '|' ||
    COALESCE(AVAILABILITY,'') || '|' ||
    COALESCE(AVAILABLEOFFER,'') || '|' ||
    COALESCE(BRAND_MARKET,'') || '|' ||
    COALESCE(BRAND_SEGMENT,'') || '|' ||
    COALESCE(BRAND_SUB_SEGMENT,'') || '|' ||
    COALESCE(CABINTYPE,'') || '|' ||
    COALESCE(CACHEPAGELINK,'') || '|' ||
    COALESCE(CATEGORYCODE,'') || '|' ||
    COALESCE(CRUISETYPE,'') || '|' ||
    COALESCE(TO_VARCHAR(CRUISEDURATION),'') || '|' ||
    COALESCE(CRUISE_LINE_CODE,'') || '|' ||
    COALESCE(CRUISE_LINE_NAME,'') || '|' ||
    COALESCE(TO_VARCHAR(VESSEL_IMO_NO),'') || '|' ||
    COALESCE(SHIP_CODE_3L,'') || '|' ||
    COALESCE(TO_VARCHAR(TAXEXCLUSIVERATEPPD),'') || '|' ||
    COALESCE(TO_VARCHAR(TAXINCLUSIVERATEPPD),'') || '|' ||
    COALESCE(TO_VARCHAR(TAXESANDFEESPPD),'') || '|' ||
    COALESCE(RM_ROLLUP_PRODUCT_DESC,'') || '|' ||
    COALESCE(PRODUCT_DESC,'') || '|' ||
    COALESCE(RATETYPE,'') || '|' ||
    COALESCE(GCI_TRADE_LEVEL_1,'') || '|' ||
    COALESCE(GCI_TRADE_LEVEL_2,'') || '|' ||
    COALESCE(GCI_TRADE_LEVEL_3,'') || '|' ||
    COALESCE(GCI_TRADE_LEVEL_4,'') || '|' ||
    COALESCE("Market 1",'') || '|' ||
    COALESCE("Market 2",'') || '|' ||
    COALESCE(NCL_TRADE_1,'') || '|' ||
    COALESCE(NCL_TRADE_2,'') || '|' ||
    COALESCE(NCL_TRADE_3,'') || '|' ||
    COALESCE(OCI_TRADE_1,'') || '|' ||
    COALESCE(OCI_TRADE_2,'') || '|' ||
    COALESCE(OCI_TRADE_3,'') || '|' ||
    COALESCE(OCI_TRADE_4,'') || '|' ||
    COALESCE(RSSC_TRADE_1,'') || '|' ||
    COALESCE(RSSC_TRADE_2,'') || '|' ||
    COALESCE(RSSC_TRADE_3,'') || '|' ||
    COALESCE(TO_VARCHAR(DISEMBARKATIONDATE),'') || '|' ||
    COALESCE(DISEMBARK_PORT_NAME,'') || '|' ||
    COALESCE(DISEMBARK_PORT_CODE_GCI,'') || '|' ||
    COALESCE(DISEMBARK_PORT_CODE_NCL,'') || '|' ||
    COALESCE(DISEMBARK_PORT_CODE_OCI,'') || '|' ||
    COALESCE(DISEMBARK_PORT_CODE_RSSC,'') || '|' ||
    COALESCE(DURATION_LEVEL_1,'') || '|' ||
    COALESCE(DURATION_LEVEL_2,'') || '|' ||
    COALESCE(TO_VARCHAR(EMBARKATIONDATE),'') || '|' ||
    COALESCE(EMBARK_PORT_NAME,'') || '|' ||
    COALESCE(EMBARK_PORT_CODE_GCI,'') || '|' ||
    COALESCE(EMBARK_PORT_CODE_NCL,'') || '|' ||
    COALESCE(EMBARK_PORT_CODE_OCI,'') || '|' ||
    COALESCE(EMBARK_PORT_CODE_RSSC,'') || '|' ||
    COALESCE(EMBARK_REGION_1,'') || '|' ||
    COALESCE(EMBARK_REGION_2,'') || '|' ||
    COALESCE(EMBARK_REGION_3,'') || '|' ||
    COALESCE(FARE_PASSENGERTYPE,'') || '|' ||
    COALESCE(TO_VARCHAR(SHOP_DATE),'') || '|' ||
    COALESCE(SHIP_CODE_GCI,'') || '|' ||
    COALESCE(SHIP_CODE_NCL,'') || '|' ||
    COALESCE(SHIP_CODE_OCI,'') || '|' ||
    COALESCE(SHIP_CODE_RSSC,'') || '|' ||
    COALESCE(LASTSHOPPEDSUBCATEGORY,'') || '|' ||
    COALESCE(TO_VARCHAR(LAST_SHOP_DATE),'') || '|' ||
    COALESCE(TO_VARCHAR(LOAD_DATE),'') || '|' ||
    COALESCE("Obstructed(Y/N)",'') || '|' ||
    COALESCE(TO_VARCHAR(RATEINQUIRYDATE),'') || '|' ||
    COALESCE("RountTrip(Y/N)",'') || '|' ||
    COALESCE(EMBARK_DOW,'') || '|' ||
    COALESCE(TO_VARCHAR(EMBARK_DOW_NO),'') || '|' ||
    COALESCE(TO_VARCHAR(EMBARK_MONTH),'') || '|' ||
    COALESCE(TO_VARCHAR(EMBARK_WEEK),'') || '|' ||
    COALESCE(TO_VARCHAR(EMBARK_YEAR),'') || '|' ||
    COALESCE(SHIP_SIZE,'') || '|' ||
    COALESCE(SHOP_DOW,'') || '|' ||
    COALESCE(TO_VARCHAR(SHOP_DOW_NO),'') || '|' ||
    COALESCE(TO_VARCHAR(SHOP_MONTH),'') || '|' ||
    COALESCE(TO_VARCHAR(SHOP_WEEK),'') || '|' ||
    COALESCE(TO_VARCHAR(SHOP_YEAR),'') || '|' ||
    COALESCE(SHOPFREQUENCY,'') || '|' ||
    COALESCE(SOURCECURRENCY,'') || '|' ||
    COALESCE(SOURCEMARKET,'') || '|' ||
    COALESCE(SHIP_NAME_GCI,'') || '|' ||
    COALESCE(SRC_FILE_NAME,'') || '|' ||
    COALESCE(TO_VARCHAR(TAXESANDFEES),'') || '|' ||
    COALESCE(TO_VARCHAR(TAXEXCLUSIVERATEPP),'') || '|' ||
    COALESCE(TO_VARCHAR(TAXINCLUSIVERATEPP),'') || '|' ||
    COALESCE("Gratuity (Y/N)",'') || '|' ||
    COALESCE(WEBSITE,'') || '|' ||
    COALESCE(WEEKEND_EMBARK,'') || '|' ||
    COALESCE(TO_VARCHAR(CONVERTIONRATE),'') || '|' ||
    COALESCE(TO_VARCHAR(RATEPPUSD),'') || '|' ||
    COALESCE(TO_VARCHAR(TAXESANDFEESUSD),'') || '|' ||
    COALESCE(ORIGCURRENCY,'') || '|' ||
    COALESCE(SOURCEITINERARYNAME,'') || '|' ||
    COALESCE(ITINERARYPORTCONTENT,'') || '|' ||
    COALESCE(TO_VARCHAR(MIN_RATEINQUIRYDATE),'') || '|' ||
    COALESCE(WORLD_FLAG,'') || '|' ||
    COALESCE(SUPERMKT,'') || '|' ||
    COALESCE(REGENT_CRUISE_CD,'') || '|' ||
    COALESCE(NCL_CABINTYPE,'') || '|' ||
    COALESCE(OCI_CABINTYPE,'') || '|' ||
    COALESCE(RSS_CABINTYPE,'') || '|' ||
    COALESCE(SAIL_ID,'')
);

-- select top 10 * from VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_REF_PRICING_DASHBOARD
-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_REF_PRICING_DASHBOARD
-- ADD CONSTRAINT PK_SAT_GCI_CONS_RSS_REF_PRICING_DASHBOARD
-- PRIMARY KEY (
--     NCLH_UNI_VOYAGE_CODE,
--     SHOP_DATE,
--     CABINTYPE,
--     RSS_CABINTYPE,
--     TAXINCLUSIVERATEPP
-- );

-- select top 10 * from VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_REF_PRICING_DASHBOARD


-- -- Step 1: Create a new table with an added 'Supermkt' column
-- CREATE OR REPLACE TEMP TABLE GCI_INTER_PP_RSS_DASHBOARD_INPUT_DATA_1 AS
-- SELECT *, 'NA' AS Supermkt
-- FROM GCI_INTER_PP_RSS_PRICING_COLLATED_RID_SIN_IPC_TRADE;


-- -- Step 2: Create another table with the derived 'Regent_Cruise_CD' column
-- CREATE OR REPLACE TEMP  TABLE GCI_INTER_PP_RSS_DASHBOARD_INPUT_DATA_2 AS
-- SELECT  
--     *, 
--     CASE 
--         WHEN Ship_Code_RSSC = '' THEN ' ' 
--         ELSE CONCAT(
--             Ship_Code_RSSC,
--             SUBSTRING(TO_VARCHAR(EmbarkationDate), 3, 2),
--             SUBSTRING(TO_VARCHAR(EmbarkationDate), 6, 2),
--             SUBSTRING(TO_VARCHAR(EmbarkationDate), 9, 2)
--         )
--     END AS Regent_Cruise_CD
-- FROM GCI_INTER_PP_RSS_DASHBOARD_INPUT_DATA_1;



-- CREATE OR REPLACE TEMP TABLE RSS2 As
-- SELECT 
-- 		RSS_CabinType,
-- 		cruiselinecode,
-- 		TaxInclusiveRatePP,
-- 		CONCAT(CRUISELINECODE,SHIPCODE) AS CRUISELINECODE_SHIPCODE,
-- 		cabintype,
-- 		CAST(rateinquirydate as date) AS rateinquirydate,
-- 		CAST(embarkationdate as date) AS embarkationdate,
-- 		CAST(disembarkdate as date) AS disembarkdate
-- FROM GCI_NCH_RAW_PRICING_RG_PASS;
-- -- SELECT TOP 10 * FROM CTE_2


-- CREATE OR REPLACE TEMP TABLE GCI_INTER_PP_RSS_DATA AS 
-- SELECT a.*,b.RSS_CabinType
-- from GCI_INTER_PP_RSS_DASHBOARD_INPUT_DATA_2 a
-- LEFT JOIN RSS2 b 
-- on a.rateinquirydate= b.rateinquirydate
-- and a.embarkationdate=b.embarkationdate
-- and a.DisEmbarkationDate=b.disembarkdate
-- and a.CRUISE_LINE_CODE=b.cruiselinecode
-- AND a.Ship_Code_GCI=b.CRUISELINECODE_SHIPCODE
-- and a.cabintype = b.cabintype
-- AND a.TaxInclusiveRatePP = b.TaxInclusiveRatePP;

-- ALTER TABLE GCI_INTER_PP_RSS_DATA
--         ADD taxexclusive_regent_air_rate_pp DECIMAL(28,10),
--             taxexclusive_regent_air_rate_ppd DECIMAL(28,10),
--             taxinclusive_regent_air_rate_pp DECIMAL(28,10),
--             taxinclusive_regent_air_rate_ppd DECIMAL(28,10);



-- TRUNCATE TABLE GCI_CONSUMPTION_PP_RSS_DASHBOARD;
-- INSERT INTO GCI_CONSUMPTION_PP_RSS_DASHBOARD
-- SELECT DISTINCT *  FROM GCI_INTER_PP_RSS_DATA;

-- select top 10 * from VESSOPS_D.L00_STG.GCI_CONSUMPTION_PP_RSS_DASHBOARD


-- --select top 10 * from GCI_CONSUMPTION_PP_RSS_DASHBOARD;

-- -- ALTER TABLE GCI_CONSUMPTION_PP_RSS_DASHBOARD
-- --         ADD taxexclusive_regent_air_rate_pp DECIMAL(28,10),
-- --             taxexclusive_regent_air_rate_ppd DECIMAL(28,10),
-- --             taxinclusive_regent_air_rate_pp DECIMAL(28,10),
-- --             taxinclusive_regent_air_rate_ppd DECIMAL(28,10);


-- UPDATE GCI_CONSUMPTION_PP_RSS_DASHBOARD
-- SET 
--     taxinclusive_regent_air_rate_pp = CASE 
--         WHEN taxinclusiveratepp = -1 THEN -1
--         ELSE (taxinclusiveratepp - CAST(REPLACE(AvailableOffer, ',', '') AS DECIMAL(28,10)))
--     END,
--     taxexclusive_regent_air_rate_pp = CASE 
--         WHEN taxexclusiveratepp = -1 THEN -1
--         ELSE (taxexclusiveratepp - CAST(REPLACE(AvailableOffer, ',', '') AS DECIMAL(28,10)))
--     END,
--     taxinclusive_regent_air_rate_ppd = CASE 
--         WHEN taxinclusive_regent_air_rate_pp = -1 THEN -1
--         ELSE (taxinclusive_regent_air_rate_pp / CruiseDuration)
--     END,
--     taxexclusive_regent_air_rate_ppd = CASE 
--         WHEN taxexclusive_regent_air_rate_pp = -1 THEN -1
--         ELSE (taxexclusive_regent_air_rate_pp / CruiseDuration)
--     END
-- WHERE UPPER(TRIM(CRUISE_LINE_CODE)) = 'RSS'
--   AND RateInquiryDate > '2022-08-01'
--   AND AvailableOffer IS NOT NULL;

  

-- UPDATE GCI_CONSUMPTION_PP_RSS_DASHBOARD
-- SET rss_cabintype = UPPER(TRIM(cabintype))
-- WHERE UPPER(TRIM(cabintype)) = 'BALCONY' AND rss_cabintype IS NULL;

-- select top 10 * from vessops_d.l00_stg.GCI_CONSUMPTION_PP_RSS_DASHBOARD


CREATE OR REPLACE temp TABLE R1 AS
SELECT 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    RSS_CabinType,
    categorycode,
    CRUISE_LINE_CODE AS Brand_name,
    Ship_Name_GCI AS Ship_name,
    Cruise_Line_Name AS brand,
    MAX(EmbarkationDate) AS Sail_date,
    MAX(TaxInclusiveRatePP) AS TaxInclusiveRatePP,
    MAX(TaxInclusiveRatePPD) AS TaxInclusiveRatePPD,
    MAX(TaxExclusiveRatePPD) AS TaxexclusiveRatePPD,
    MAX(TaxExclusiveRatePP) AS TaxexclusiveRatePP,
    ItineraryPortContent,
    MAX(SHOP_DATE) AS LATEST_VALUE
FROM VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
GROUP BY 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    RSS_CabinType,
    categorycode,
    CRUISE_LINE_CODE,
    Ship_Name_GCI,
    Cruise_Line_Name,
    ItineraryPortContent;

--select count(*) from r1

-- Step 1: Add Freshest=1 column based on Sail_date > 2021
CREATE OR REPLACE TEMP TABLE R2 AS
SELECT 
    a.*, 
    DATEDIFF('week', latest_value, CURRENT_DATE) AS Freshest1
FROM R1 a
WHERE EXTRACT(YEAR FROM Sail_date) > 2021;

-- Step 2: Filter Freshest1 <= 5 and order by Brand, Ship, etc.
CREATE OR REPLACE TEMP TABLE R3 AS
SELECT *
FROM R2
WHERE Freshest1 <= 5
ORDER BY Brand_name, Ship_name, Sail_date, CabinType, RSS_CabinType, Freshest1;

-- Step 3: Use window function to assign dense rank based on voyage and cabin type
CREATE OR REPLACE TEMP TABLE R4 AS
SELECT *,
       DENSE_RANK() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE, CabinType ORDER BY Freshest1) AS dense_rank_row
FROM R3;

-- Step 4: Select required columns and rename Freshest1 to Freshest=1
CREATE OR REPLACE TEMP TABLE R5 AS
SELECT 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    RSS_CabinType,
    categorycode,
    Brand_name,
    Ship_name,
    brand,
    Sail_date,
    TaxInclusiveRatePP,
    TaxInclusiveRatePPD,
    TaxexclusiveRatePPD,
    TaxExclusiveRatePP,
    ItineraryPortContent,
    Latest_value,
    Freshest1 AS "Freshest=1"
FROM R4;

-- Step 5: Replace special characters in itinerary string
CREATE OR REPLACE TEMP TABLE R6 AS
SELECT *,
       REPLACE(ItineraryPortContent, '@$#', '@') AS ItinPort_ItinDate
FROM R5;

CREATE OR REPLACE TEMP TABLE R7 AS
SELECT *,
       REPLACE(ItinPort_ItinDate, '@$', '+') AS port_content
FROM R6;

-- Step 6: Remove all digits from port_content to form 'serach'
CREATE OR REPLACE TEMP TABLE R8 AS
SELECT *,
       REGEXP_REPLACE(port_content, '[0-9]', '') AS serach
FROM R7;

-- Step 7: Final transformation of itinerary string
CREATE OR REPLACE TEMP TABLE R9 AS
SELECT *,
       REPLACE(serach, '+--@', '@') AS Itinerary_String_name
FROM R8;

-- Optional join with UNLocCodeContent (commented out as reference for future)
-- SELECT a.*, b.UNLocCodeContent 
-- FROM R9 a 
-- LEFT JOIN rawzone.header_pass b 
-- ON a.ItineraryPortContent = b.ItineraryPortContent;

-- Step 1: Create initial working table
CREATE OR REPLACE TEMP TABLE RR1 AS
SELECT *
FROM R9;  -- Replace with the correct final staging table if different

-- Step 2: Extract itinerary-related distinct values
CREATE OR REPLACE TEMP TABLE RR2 AS
SELECT DISTINCT NCLH_UNI_VOYAGE_CODE,
                itinerary_string_name,
                ItinPort_ItinDate
FROM RR1;

-- Step 3: Explode itinerary string and assign row numbers
CREATE OR REPLACE TEMP TABLE RR3 AS
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE ORDER BY NULL) AS row_number,
       TRIM(SPLIT_PART(itinerary_string_name, '@', SEQ4() + 1)) AS Port_Value
FROM RR2,
     TABLE(GENERATOR(ROWCOUNT => 50)) -- adjust ROWCOUNT based on max number of ports in any itinerary
WHERE SPLIT_PART(itinerary_string_name, '@', SEQ4() + 1) IS NOT NULL;

-- Step 4: Join with reference port data
CREATE OR REPLACE TEMP TABLE RR4 AS
SELECT a.*, 
       COALESCE(UPPER(TRIM(b.Tb_Unv_Port_Code)), 'XXTBD') AS Tb_Unv_Port_Code
FROM RR3 a
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA b
  ON UPPER(TRIM(a.Port_Value)) = UPPER(TRIM(b.Dtl_Port_City_Nam));

-- Step 5: Reorder by row number for consistency
CREATE OR REPLACE TEMP TABLE RR5 AS
SELECT * FROM RR4
ORDER BY NCLH_UNI_VOYAGE_CODE, row_number;

-- Step 6: Concatenate port codes into a single string per voyage
CREATE OR REPLACE TEMP TABLE RR6 AS
SELECT NCLH_UNI_VOYAGE_CODE,
       ItinPort_ItinDate,
       LISTAGG(Tb_Unv_Port_Code, '@') WITHIN GROUP (ORDER BY row_number) AS concatenated
FROM RR5
GROUP BY NCLH_UNI_VOYAGE_CODE, ItinPort_ItinDate;

-- Step 7: Join back with original RSS pricing table
CREATE OR REPLACE TEMP TABLE RR7 AS
SELECT a.*, b.concatenated
FROM R9 a
LEFT JOIN RR6 b
  ON a.ItinPort_ItinDate = b.ItinPort_ItinDate;

-- Step 8: Remove duplicates if any
CREATE OR REPLACE TEMP TABLE RR8 AS
SELECT DISTINCT * FROM RR7;

-- Step 1: Create final table with itinerary port code string
--drop table RR9;

CREATE OR REPLACE temp TABLE RR9 AS
SELECT *,
       REPLACE(Concatenated, '@', '-') AS Itinerary_string_port_code
FROM RR8;

--select top 100 * from rr9

CREATE OR REPLACE TEMP TABLE LATEST_PRICING as 
SELECT 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    RSS_CabinType,
    categorycode,
    Brand_Name,
    Ship_name,
    brand,
    Sail_date,
    TaxInclusiveRatePP,
    TaxInclusiveRatePPD,
    TaxExclusiveRatePP,
    TaxExclusiveRatePPD,
    Itinerary_string_name,
    Itinerary_string_port_code,
    Latest_value,
    'Freshest=1' AS "Freshest=1"  -- Replace with actual logic if needed
FROM RR9;

-- select * from LATEST_PRICING where nclh_uni_voyage_code='VOC-VV-2026-08-03-USNYC-NOBGO-28'

-- select * from VESSOPS_D.l00_STG.GCI_CONSUMPTION_PP_RSS_LATEST_PRICING where nclh_uni_voyage_code='VOC-VV-2026-08-03-USNYC-NOBGO-28'

alter table LATEST_PRICING
ADD COLUMN 
    VESSEL_IMO VARCHAR(50),
    NCLH_SHIP_CODE VARCHAR(5),
    SAIL_ID varchar(50),
    MD5_VOYAGE_HASH_KEY VARCHAR,
    HASH_DIFF VARCHAR,
    LDTS TIMESTAMP_NTZ(9),
    RCSR VARCHAR(200),
    LAST_MODIFIED_BY VARCHAR;


UPDATE LATEST_PRICING AS a
SET 
    VESSEL_IMO     = b.VESSEL_IMO_NO,
    NCLH_SHIP_CODE = b.SHIP_CODE_3L
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA AS b
WHERE CONCAT(
          SPLIT_PART(a.NCLH_UNI_VOYAGE_CODE, '-', 1),  -- CRUISELINECODE
          SPLIT_PART(a.NCLH_UNI_VOYAGE_CODE, '-', 2)   -- SHIPCODE
      ) = b."5L_SHIP_CODE";

    

-- select top 0 * from LATEST_PRICING where md5_voyage_hash_key is not  null



UPDATE LATEST_PRICING
SET SAIL_ID = CONCAT(
    NCLH_SHIP_CODE,
    TO_CHAR(SAIL_DATE, 'YYMMDD')
);

UPDATE LATEST_PRICING
SET MD5_VOYAGE_HASH_KEY = MD5(SAIL_ID);


UPDATE LATEST_PRICING
SET HASH_DIFF = MD5(
    CONCAT_WS(
        '|',
        COALESCE(NCLH_UNI_VOYAGE_CODE, ''),
        COALESCE(CABINTYPE, ''),
        COALESCE(RSS_CABINTYPE, ''),
        COALESCE(CATEGORYCODE, ''),
        COALESCE(BRAND_NAME, ''),
        COALESCE(SHIP_NAME, ''),
        COALESCE(BRAND, ''),
        COALESCE(TO_VARCHAR(SAIL_DATE), ''),
        COALESCE(TO_VARCHAR(TAXINCLUSIVERATEPP), ''),
        COALESCE(TO_VARCHAR(TAXINCLUSIVERATEPPD), ''),
        COALESCE(TO_VARCHAR(TAXEXCLUSIVERATEPP), ''),
        COALESCE(TO_VARCHAR(TAXEXCLUSIVERATEPPD), ''),
        COALESCE(ITINERARY_STRING_NAME, ''),
        COALESCE(ITINERARY_STRING_PORT_CODE, ''),
        COALESCE(LATEST_VALUE, ''),
        COALESCE(TO_VARCHAR("Freshest=1"), ''),
        COALESCE(TO_VARCHAR(VESSEL_IMO), ''),
        COALESCE(NCLH_SHIP_CODE, ''),
        COALESCE(TO_VARCHAR(SAIL_ID), '')
    )
);


UPDATE LATEST_PRICING
SET 
    LDTS = CURRENT_TIMESTAMP(),
    RCSR = 'SAT_GCI_PRICING_RSS_DASHBOARD',
    LAST_MODIFIED_BY = CURRENT_USER();


--create or replace table VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING as 

-- alter table VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING
-- rename to VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING

-- alter table VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING
--  ALTER COLUMN RSS_CABINTYPE SET DATA TYPE VARCHAR(500);

TRUNCATE TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING;
insert into VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING
select * from LATEST_PRICING;

-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING
-- ADD CONSTRAINT PK_SAT_GCI_CONS_RSS_LATEST_PRICING
-- PRIMARY KEY (
--     NCLH_UNI_VOYAGE_CODE,
--     CABINTYPE,
--     RSS_CABINTYPE,
--     CATEGORYCODE,
--     LATEST_VALUE,
--     TAXINCLUSIVERATEPP
-- );



-- -- Step 2: Create the final pricing table in GCI_Consumption schema
-- TRUNCATE TABLE GCI_CONSUMPTION_PP_RSS_LATEST_PRICING;
-- INSERT INTO GCI_CONSUMPTION_PP_RSS_LATEST_PRICING
-- SELECT 
--     NCLH_UNI_VOYAGE_CODE,
--     CabinType,
--     RSS_CabinType,
--     categorycode,
--     Brand_Name,
--     Ship_name,
--     brand,
--     Sail_date,
--     TaxInclusiveRatePP,
--     TaxInclusiveRatePPD,
--     TaxExclusiveRatePP,
--     TaxExclusiveRatePPD,
--     Itinerary_string_name,
--     Itinerary_string_port_code,
--     Latest_value,
--     'Freshest=1' AS "Freshest=1"  -- Replace with actual logic if needed
-- FROM RR9;




---PRIICING TABLE AA PER THE OCI as per the sandra's request

-- Step: Create table TEMP_I21 with distinct voyage and pricing details

--drop table TEMP_I27;

CREATE OR REPLACE temp TABLE TEMP_I21 AS
SELECT 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    categorycode,
    CRUISE_LINE_CODE AS Brand_name,
    Ship_Name_GCI AS Ship_name,
    Cruise_Line_Name AS brand,
    MAX(EmbarkationDate) AS Sail_date,
    MAX(TaxInclusiveRatePP) AS TaxInclusiveRatePP,
    MAX(TaxInclusiveRatePPD) AS TaxInclusiveRatePPD,
    MAX(SHOP_DATE) AS Latest_value
FROM 
   VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
GROUP BY 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    categorycode,
    CRUISE_LINE_CODE,
    Ship_Name_GCI,
    Cruise_Line_Name;

 
-- Step 1: Add Freshest1 column (difference in weeks from latest_value to today) and filter Sail_date > 2021
CREATE OR REPLACE temp  TABLE TEMP_I22 AS
SELECT 
    a.*, 
    DATEDIFF(week, a.Latest_value, CURRENT_DATE) AS Freshest1
FROM TEMP_I21 a
WHERE EXTRACT(YEAR FROM Sail_date) > 2021;

-- Step 2: Filtered and ordered data (no need to physically order for select * into)
CREATE OR REPLACE temp  TABLE TEMP_I23 AS
SELECT *
FROM TEMP_I22
-- Uncomment if you need to filter where Freshest1 <= 5
-- WHERE Freshest1 <= 5
ORDER BY 
    Brand_Name,
    Ship_name,
    Sail_date,
    CabinType,
    Freshest1;

-- Step 3: Assign dense_rank within each Voyage and CabinType partition, ordered by Freshest1
CREATE OR REPLACE temp TABLE TEMP_I24 AS
SELECT 
    *,
    DENSE_RANK() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE, CabinType ORDER BY Freshest1 ASC) AS dense_rank_row
FROM TEMP_I23;

-- Step 4: Keep only the top-ranked (freshest) records
CREATE OR REPLACE temp TABLE TEMP_I25 AS
SELECT *
FROM TEMP_I24
WHERE dense_rank_row = 1;

-- Step 5: Select the required columns
CREATE OR REPLACE temp TABLE TEMP_I26 AS
SELECT 
    NCLH_UNI_VOYAGE_CODE,
    CabinType,
    categorycode,
    Brand_Name,
    Ship_name,
    brand,
    Sail_date,
    TaxInclusiveRatePP,
    TaxInclusiveRatePPD,
    Latest_value,
    Freshest1 AS "Freshest=1"
FROM TEMP_I25;

Create or replace temp table TEMP_I27 as 
SELECT 
    a.*,
    b.cachepagelink
FROM TEMP_I26 a
LEFT JOIN VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD b
    ON UPPER(TRIM(a.NCLH_UNI_VOYAGE_CODE)) = UPPER(TRIM(b.NCLH_UNI_VOYAGE_CODE))
   AND a.Latest_value = b.Shop_Date
   AND UPPER(TRIM(a.CabinType)) = UPPER(TRIM(b.CabinType));


--select * from TEMP_I27 where nclh_uni_voyage_code='SEA-PS-2026-04-02-CLSAI-AUBRO-57'


alter table TEMP_I27
ADD COLUMN 
    VESSEL_IMO VARCHAR(50),
    NCLH_SHIP_CODE VARCHAR(5),
    SAIL_ID varchar(50),
    MD5_VOYAGE_HASH_KEY VARCHAR,
    HASH_DIFF VARCHAR,
    LDTS TIMESTAMP_NTZ(9),
    RCSR VARCHAR(200),
    LAST_MODIFIED_BY VARCHAR;


UPDATE TEMP_I27 AS a
SET 
    VESSEL_IMO     = b.VESSEL_IMO_NO,
    NCLH_SHIP_CODE = b.SHIP_CODE_3L
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA AS b
WHERE CONCAT(
          SPLIT_PART(a.NCLH_UNI_VOYAGE_CODE, '-', 1),  -- CRUISELINECODE
          SPLIT_PART(a.NCLH_UNI_VOYAGE_CODE, '-', 2)   -- SHIPCODE
      ) = b."5L_SHIP_CODE";


UPDATE TEMP_I27
SET SAIL_ID = CONCAT(
    NCLH_SHIP_CODE,
    TO_CHAR(SAIL_DATE, 'YYMMDD')
);

UPDATE TEMP_I27
SET MD5_VOYAGE_HASH_KEY = MD5(SAIL_ID);

--select top 0 * from TEMP_I27
UPDATE TEMP_I27
SET HASH_DIFF = MD5(
    CONCAT_WS(
        '|',
        COALESCE(NCLH_UNI_VOYAGE_CODE, ''),
        COALESCE(CABINTYPE, ''),
        COALESCE(CATEGORYCODE, ''),
        COALESCE(BRAND_NAME, ''),
        COALESCE(SHIP_NAME, ''),
        COALESCE(BRAND, ''),
        COALESCE(TO_VARCHAR(SAIL_DATE), ''),
        COALESCE(TO_VARCHAR(TAXINCLUSIVERATEPP), ''),
        COALESCE(TO_VARCHAR(TAXINCLUSIVERATEPPD), ''),
        COALESCE(TO_VARCHAR(LATEST_VALUE), ''),
        COALESCE(CACHEPAGELINK, ''),
        COALESCE(VESSEL_IMO, ''),
        COALESCE(NCLH_SHIP_CODE, ''),
        COALESCE(SAIL_ID, '')
    )
);



UPDATE TEMP_I27
SET 
    LDTS = CURRENT_TIMESTAMP(),
    RCSR = 'SAT_GCI_PRICING_RSS_DASHBOARD',
    LAST_MODIFIED_BY = CURRENT_USER();


-- create or replace table VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING_CABIN as 

-- alter table VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING_CABIN
-- rename to VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING_CABIN;

TRUNCATE TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING_CABIN;
insert into VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LATEST_PRICING_CABIN
select * from TEMP_I27;

-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING_CABIN
-- ADD CONSTRAINT PK_SAT_GCI_CONS_RSS_LATEST_PRICING
-- PRIMARY KEY (
--     NCLH_UNI_VOYAGE_CODE,
--     CABINTYPE,
--     CATEGORYCODE,
--     LATEST_VALUE,
--     TAXINCLUSIVERATEPP
-- );

-- select * from VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_LATEST_PRICING_CABIN where nclh_uni_voyage_code='PON-CC-2026-04-30-FRBES-ISREY-4'

-- -- Step 6: Create the final latest pricing table with cachepagelink using LEFT JOIN
-- TRUNCATE TABLE GCI_CONSUMPTION_PP_RSS_LATEST_PRICING_V2;
-- INSERT into GCI_CONSUMPTION_PP_RSS_LATEST_PRICING_V2
-- SELECT 
--     a.*,
--     b.cachepagelink
-- FROM TEMP_I26 a
-- LEFT JOIN GCI_CONSUMPTION_PP_RSS_Dashboard b
--     ON UPPER(TRIM(a.NCLH_UNI_VOYAGE_CODE)) = UPPER(TRIM(b.NCLH_UNI_VOYAGE_CODE))
--    AND a.Latest_value = b.Shop_Date
--    AND UPPER(TRIM(a.CabinType)) = UPPER(TRIM(b.CabinType));

## LIKE FOR LIKE 
RSS AND COMP 

In [ ]:
-- Step 1: Get latest RateInquiryDate per voyage
CREATE OR REPLACE TEMP TABLE temp1 AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE ORDER BY RateInquiryDate DESC) AS rownum
    FROM VESSOPS_D.l20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
) x
WHERE rownum = 1;

--select top 10 * from temp1
--select top 10 * from GCI_CONSUMPTION_PP_RSS_DASHBOARD

--select count(*) from temp1 where nclh_uni_voyage_code='RSS-SX-2026-09-30-USSEA-JPTYO-20' 

--select count(distinct(nclh_uni_voyage_code)) from GCI_CONSUMPTION_PP_RSS_DASHBOARD where nclh_uni_voyage_code='RSS-SX-2026-09-30-USSEA-JPTYO-20' and adv_week='82'--70

-- Step 2: Join with embarkation port data for coordinates

CREATE OR REPLACE TEMP TABLE temp2 AS
SELECT
    VDF.Cruise_Line_Name AS CRUISELINENAME,
    VDF.Ship_Name_GCI AS SOURCESHIPNAME,
    VDF.CRUISE_LINE_CODE AS CRUISELINECODE,
    RIGHT(VDF.Ship_Code_GCI, 2) AS SHIPCODE,
    VDF.EmbarkationDate AS EMBARKATIONDATE,
    VDF.Embark_Port_Name AS EMBARKATIONPORT,
    VDF.Embark_Port_Code_GCI AS EMBARKATIONPORTUN_LOCODE,
    VDF.Disembark_Port_Name AS DISEMBARKATIONPORT,
    VDF.DisEmbark_Port_Code_GCI AS DISEMBARKATIONPORTUN_LOCODE,
    VDF.CruiseDuration AS CRUISEDURATION,
    VDF.NCLH_UNI_VOYAGE_CODE,
    VDF.GCI_Trade_Level_1 AS TRADE_HIERARCHY1,
    VDF.GCI_Trade_Level_2 AS TRADE_HIERARCHY2,
    VDF.GCI_Trade_Level_3 AS TRADE_HIERARCHY3,
    VDF.Ship_Code_GCI AS UNIQUE_SHIP_CODE,
    P.PORT_LAT AS EMBPORT_LAT,
    P.PORT_LONG AS EMBPORT_LONG,
    VDF.SHIP_CODE_3L,
    VDF.NCL_CABINTYPE,
    VDF.OCI_CABINTYPE,
    VDF.RSS_CABINTYPE,
    VDF.SAIL_ID,
    VDF.MD5_VOYAGE_HASH_KEY
FROM temp1 VDF
JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA P
    ON UPPER(TRIM(VDF.Embark_Port_Code_GCI)) = UPPER(TRIM(P.UNV_PORT_CODE))
WHERE UPPER(TRIM(VDF.Embark_Port_Code_GCI)) NOT IN ('XXTBD', 'ATSEA', 'XXDTE', 'XXDTW')
  AND P.PORT_COORDINATES IS NOT NULL
  AND P.PORT_COORDINATES NOT IN ('0,0', '0, 0', '0');

-- Step 3: Join with disembark port coordinates
CREATE OR REPLACE TEMP TABLE temp3 AS
SELECT
    t2.*,
    P.PORT_LAT AS DISEMBPORT_LAT,
    P.PORT_LONG AS DISEMBPORT_LONG
FROM temp2 t2
JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA P
    ON UPPER(TRIM(t2.DISEMBARKATIONPORTUN_LOCODE)) = UPPER(TRIM(P.UNV_PORT_CODE))
WHERE UPPER(TRIM(t2.DISEMBARKATIONPORTUN_LOCODE)) NOT IN ('XXTBD', 'ATSEA', 'XXDTE', 'XXDTW')
  AND P.PORT_COORDINATES IS NOT NULL
  AND P.PORT_COORDINATES NOT IN ('0,0', '0, 0', '0');

-- Step 4: Join ship details
CREATE OR REPLACE TEMP TABLE temp4 AS
SELECT
    t3.*,
    S.BRAND_FOLIO AS BRANDCOMPANY,
    S."Segment" AS SHIP_SEGMENT,
    S.ALB AS ALB
FROM temp3 t3
JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA S
    ON UPPER(TRIM(t3.UNIQUE_SHIP_CODE)) = UPPER(TRIM(S."5L_SHIP_CODE"));

--select top 10 * from temp5

-- Step 5: Add trip characteristics and ship size
CREATE OR REPLACE TEMP TABLE temp5 AS
SELECT *,
    CASE
        WHEN CRUISEDURATION IN (1, 2) THEN '1-2'
        WHEN CRUISEDURATION BETWEEN 3 AND 5 THEN '3-5'
        WHEN CRUISEDURATION BETWEEN 6 AND 8 THEN '6-8'
        WHEN CRUISEDURATION BETWEEN 9 AND 12 THEN '9-12'
        WHEN CRUISEDURATION BETWEEN 13 AND 15 THEN '13-15'
        WHEN CRUISEDURATION BETWEEN 16 AND 20 THEN '16-20'
        WHEN CRUISEDURATION BETWEEN 21 AND 30 THEN '21-30'
        WHEN CRUISEDURATION BETWEEN 31 AND 50 THEN '31-50'
        ELSE '51+'
    END AS DAY_FLAG,
    CASE
        WHEN UPPER(TRIM(EMBARKATIONPORTUN_LOCODE)) = UPPER(TRIM(DISEMBARKATIONPORTUN_LOCODE)) THEN 'RT'
        ELSE 'OJ'
    END AS TRIP_STATUS,
    CASE
        WHEN MONTH(EMBARKATIONDATE) IN (3, 4, 5) THEN 'SPRING'
        WHEN MONTH(EMBARKATIONDATE) IN (6, 7, 8) THEN 'SUMMER'
        WHEN MONTH(EMBARKATIONDATE) IN (9, 10, 11) THEN 'FALL'
        ELSE 'WINTER'
    END AS SEASON,
    CASE
        WHEN ALB BETWEEN 1 AND 1000 THEN 'Small'
        WHEN ALB BETWEEN 1001 AND 2500 THEN 'Medium'
        WHEN ALB BETWEEN 2501 AND 4000 THEN 'Large'
        WHEN ALB >= 4001 THEN 'Mega'
    END AS SHIP_SIZE
FROM temp4;

-- Step 6: Duplicate table for comparison
CREATE OR REPLACE TEMP TABLE temp6 AS
SELECT
    b.CRUISELINENAME AS CRUISELINENAME1,
    b.SOURCESHIPNAME AS SOURCESHIPNAME1,
    b.CRUISELINECODE AS CRUISELINECODE1,
    b.SHIPCODE AS SHIPCODE1,
    b.EMBARKATIONDATE AS EMBARKATIONDATE1,
    b.EMBARKATIONPORT AS EMBARKATIONPORT1,
    b.EMBARKATIONPORTUN_LOCODE AS EMBARKATIONPORTUN_LOCODE1,
    b.DISEMBARKATIONPORT AS DISEMBARKATIONPORT1,
    b.DISEMBARKATIONPORTUN_LOCODE AS DISEMBARKATIONPORTUN_LOCODE1,
    b.CRUISEDURATION AS CRUISEDURATION1,
    b.NCLH_UNI_VOYAGE_CODE AS NCLH_UNI_VOYAGE_CODE1,
    b.TRADE_HIERARCHY1 AS TRADE_HIERARCHY11,
    b.TRADE_HIERARCHY2 AS TRADE_HIERARCHY22,
    b.TRADE_HIERARCHY3 AS TRADE_HIERARCHY33,
    b.UNIQUE_SHIP_CODE AS UNIQUE_SHIP_CODE1,
    b.EMBPORT_LAT AS EMBPORT_LAT1,
    b.EMBPORT_LONG AS EMBPORT_LONG1,
    b.DISEMBPORT_LAT AS DISEMBPORT_LAT1,
    b.DISEMBPORT_LONG AS DISEMBPORT_LONG1,
    b.BRANDCOMPANY AS BRANDCOMPANY1,
    b.SHIP_SEGMENT AS SHIP_SEGMENT1,
    b.ALB AS ALB1,
    b.DAY_FLAG AS DAY_FLAG1,
    b.TRIP_STATUS AS TRIP_STATUS1,
    b.SEASON AS SEASON1,
    b.SHIP_SIZE AS SHIP_SIZE1,
    b.SHIP_CODE_3L as SHIP_CODE_3L1,
    b.NCL_CABINTYPE as NCL_CABINTYPE1,
    b.OCI_CABINTYPE as OCI_CABINTYPE1,
    b.RSS_CABINTYPE as RSS_CABINTYPE1,
    b.SAIL_ID as SAIL_ID1,
    b.MD5_VOYAGE_HASH_KEY as MD5_VOYAGE_HASH_KEY1
FROM temp5 b;


-- Filter OCE voyages (RSS brand)
CREATE OR REPLACE TEMP TABLE temp7 AS
SELECT DISTINCT * 
FROM temp5 
WHERE TRIM(UPPER(CRUISELINECODE)) = 'RSS';

--select count(*) from temp7


-- Filter competitor voyages (non-OCE)
CREATE OR REPLACE TEMP TABLE temp10 AS
SELECT DISTINCT * 
FROM temp6 
WHERE TRIM(UPPER(CRUISELINECODE1)) != 'RSS';


-- Step 8: Cross join RSS voyages with competitors, compute match score
CREATE OR REPLACE  TEMP TABLE GCI_INTER_LFL_RSS AS
SELECT *
FROM (
    SELECT PR_Rank.*,
           (N_Count * 9) AS Matching_percentage,
           ROW_NUMBER() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE ORDER BY (N_Count * 9) DESC) AS rownumber
    FROM (
        SELECT score_table.*,
               (N_day_flag + N_Trade_Hierarchy1_flag + N_Trade_hierarchy2_flag + N_TRADE_HIERARCHY3_flag +
                N_EMBARKATIONDATE_flag + N_EMBARKATIONPORTUN_LOCODE_flag + N_DISEMBARKATIONPORTUN_LOCODE_flag +
                N_Season_flag + N_Ship_size_flag + N_Trip_status_flag + N_Ship_segment_flag) AS N_Count
        FROM (
            SELECT a.*, b.*,
                   CASE WHEN a.DAY_FLAG = b.DAY_FLAG1 THEN 1 ELSE 0 END AS N_day_flag,
                   CASE WHEN a.TRADE_HIERARCHY1 = b.TRADE_HIERARCHY11 THEN 1 ELSE 0 END AS N_Trade_Hierarchy1_flag,
                   CASE WHEN a.TRADE_HIERARCHY2 = b.TRADE_HIERARCHY22 THEN 1 ELSE 0 END AS N_Trade_hierarchy2_flag,
                   CASE WHEN a.TRADE_HIERARCHY3 = b.TRADE_HIERARCHY33 THEN 1 ELSE 0 END AS N_TRADE_HIERARCHY3_flag,
                   CASE 
                       WHEN DATEDIFF(DAY, a.EMBARKATIONDATE, b.EMBARKATIONDATE1) BETWEEN -14 AND 14 THEN 1
                       ELSE 0
                   END AS N_EMBARKATIONDATE_flag,
                   CASE WHEN TRIM(UPPER(a.EMBARKATIONPORTUN_LOCODE)) = TRIM(UPPER(b.EMBARKATIONPORTUN_LOCODE1)) THEN 1 ELSE 0 END AS N_EMBARKATIONPORTUN_LOCODE_flag,
                   CASE WHEN TRIM(UPPER(a.DISEMBARKATIONPORTUN_LOCODE)) = TRIM(UPPER(b.DISEMBARKATIONPORTUN_LOCODE1)) THEN 1 ELSE 0 END AS N_DISEMBARKATIONPORTUN_LOCODE_flag,
                   CASE WHEN a.SEASON = b.SEASON1 THEN 1 ELSE 0 END AS N_Season_flag,
                   CASE WHEN a.SHIP_SIZE = b.SHIP_SIZE1 THEN 1 ELSE 0 END AS N_Ship_size_flag,
                   CASE WHEN a.SHIP_SEGMENT = b.SHIP_SEGMENT1 THEN 1 ELSE 0 END AS N_Ship_segment_flag,
                   CASE WHEN a.TRIP_STATUS = b.TRIP_STATUS1 THEN 1 ELSE 0 END AS N_Trip_status_flag
            FROM temp7 a
            CROSS JOIN temp10 b
            WHERE TRIM(UPPER(a.NCLH_UNI_VOYAGE_CODE)) != TRIM(UPPER(b.NCLH_UNI_VOYAGE_CODE1))
        ) score_table
    ) PR_Rank
) final
WHERE rownumber <= 10
ORDER BY rownumber;

--select top 10 *  from GCI_INTER_LFL_RSS--14080

-- select count(*) from vessops_d.l00_stg.GCI_CONSUMPTION_LFL_RSS_VOYAGES--14080

-- Step 9: Final update for perfect matches
UPDATE GCI_Inter_LFL_RSS
SET Matching_percentage = 100
WHERE Matching_percentage = 99;

--select top 10 * from GCI_Inter_LFL_RSS

--select  * from GCI_CONSUMPTION_LFL_RSS_VOYAGES where nclh_uni_voyage_code='RSS-SP-2025-11-17-USMIA-BRCCP-30'

-- Create REF_GCI_Inter_LFL_RSS temp table with HASH_DIFF in one step
CREATE OR REPLACE TEMP TABLE REF_GCI_Inter_LFL_RSS AS 
SELECT 
    BRANDCOMPANY,
    CRUISELINENAME,
    SOURCESHIPNAME,
    CRUISELINECODE,
    SHIPCODE,
    EMBARKATIONDATE,
    EMBARKATIONPORT,
    EMBARKATIONPORTUN_LOCODE,
    EMBPORT_LAT,
    EMBPORT_LONG,
    DISEMBARKATIONPORT,
    DISEMBARKATIONPORTUN_LOCODE,
    DISEMBPORT_LAT,
    DISEMBPORT_LONG,
    CRUISEDURATION,
    SAIL_ID AS RSS_SAIL_ID,
    MD5_VOYAGE_HASH_KEY AS RSS_MD5_VOYAGE_HASH_KEY,
    NCLH_UNI_VOYAGE_CODE,
    DAY_FLAG,
    TRADE_HIERARCHY1,
    TRADE_HIERARCHY2,
    TRADE_HIERARCHY3,
    TRIP_STATUS,
    SHIP_SIZE,
    BRANDCOMPANY1,
    CRUISELINENAME1,
    SOURCESHIPNAME1,
    CRUISELINECODE1,
    SHIPCODE1,
    EMBARKATIONDATE1,
    EMBARKATIONPORT1,
    EMBARKATIONPORTUN_LOCODE1,
    EMBPORT_LAT1,
    EMBPORT_LONG1,
    DISEMBARKATIONPORT1,
    DISEMBARKATIONPORTUN_LOCODE1,
    DISEMBPORT_LAT1,
    DISEMBPORT_LONG1,
    CRUISEDURATION1,
    NCLH_UNI_VOYAGE_CODE1,
    DAY_FLAG1,
    TRADE_HIERARCHY11,
    TRADE_HIERARCHY22,
    TRADE_HIERARCHY33,
    TRIP_STATUS1,
    SHIP_SIZE1,
    SEASON,
    Ship_Segment,
    SEASON1,
    Ship_Segment1,
    N_COUNT,
    ROWNUMBER,
    Matching_percentage,
    SAIL_ID1 AS RSS_COMP_SAIL_ID,
    MD5_VOYAGE_HASH_KEY1 AS RSS_COMP_MDS_MD5_VOYAGE_HASH_KEY1,

    -- ✅ Directly generate HASH_DIFF here
    MD5(
        COALESCE(BRANDCOMPANY,'') || '|' ||
        COALESCE(CRUISELINENAME,'') || '|' ||
        COALESCE(SOURCESHIPNAME,'') || '|' ||
        COALESCE(CRUISELINECODE,'') || '|' ||
        COALESCE(SHIPCODE,'') || '|' ||
        COALESCE(TO_VARCHAR(EMBARKATIONDATE),'') || '|' ||
        COALESCE(EMBARKATIONPORT,'') || '|' ||
        COALESCE(EMBARKATIONPORTUN_LOCODE,'') || '|' ||
        COALESCE(TO_VARCHAR(EMBPORT_LAT),'') || '|' ||
        COALESCE(TO_VARCHAR(EMBPORT_LONG),'') || '|' ||
        COALESCE(DISEMBARKATIONPORT,'') || '|' ||
        COALESCE(DISEMBARKATIONPORTUN_LOCODE,'') || '|' ||
        COALESCE(TO_VARCHAR(DISEMBPORT_LAT),'') || '|' ||
        COALESCE(TO_VARCHAR(DISEMBPORT_LONG),'') || '|' ||
        COALESCE(TO_VARCHAR(CRUISEDURATION),'') || '|' ||
        COALESCE(SAIL_ID,'') || '|' ||
        COALESCE(MD5_VOYAGE_HASH_KEY,'') || '|' ||
        COALESCE(NCLH_UNI_VOYAGE_CODE,'') || '|' ||
        COALESCE(DAY_FLAG,'') || '|' ||
        COALESCE(TRADE_HIERARCHY1,'') || '|' ||
        COALESCE(TRADE_HIERARCHY2,'') || '|' ||
        COALESCE(TRADE_HIERARCHY3,'') || '|' ||
        COALESCE(TRIP_STATUS,'') || '|' ||
        COALESCE(SHIP_SIZE,'') || '|' ||
        COALESCE(BRANDCOMPANY1,'') || '|' ||
        COALESCE(CRUISELINENAME1,'') || '|' ||
        COALESCE(SOURCESHIPNAME1,'') || '|' ||
        COALESCE(CRUISELINECODE1,'') || '|' ||
        COALESCE(SHIPCODE1,'') || '|' ||
        COALESCE(TO_VARCHAR(EMBARKATIONDATE1),'') || '|' ||
        COALESCE(EMBARKATIONPORT1,'') || '|' ||
        COALESCE(EMBARKATIONPORTUN_LOCODE1,'') || '|' ||
        COALESCE(TO_VARCHAR(EMBPORT_LAT1),'') || '|' ||
        COALESCE(TO_VARCHAR(EMBPORT_LONG1),'') || '|' ||
        COALESCE(DISEMBARKATIONPORT1,'') || '|' ||
        COALESCE(DISEMBARKATIONPORTUN_LOCODE1,'') || '|' ||
        COALESCE(TO_VARCHAR(DISEMBPORT_LAT1),'') || '|' ||
        COALESCE(TO_VARCHAR(DISEMBPORT_LONG1),'') || '|' ||
        COALESCE(TO_VARCHAR(CRUISEDURATION1),'') || '|' ||
        COALESCE(NCLH_UNI_VOYAGE_CODE1,'') || '|' ||
        COALESCE(DAY_FLAG1,'') || '|' ||
        COALESCE(TRADE_HIERARCHY11,'') || '|' ||
        COALESCE(TRADE_HIERARCHY22,'') || '|' ||
        COALESCE(TRADE_HIERARCHY33,'') || '|' ||
        COALESCE(TRIP_STATUS1,'') || '|' ||
        COALESCE(SHIP_SIZE1,'') || '|' ||
        COALESCE(SEASON,'') || '|' ||
        COALESCE(Ship_Segment,'') || '|' ||
        COALESCE(SEASON1,'') || '|' ||
        COALESCE(Ship_Segment1,'') || '|' ||
        COALESCE(TO_VARCHAR(N_COUNT),'') || '|' ||
        COALESCE(TO_VARCHAR(ROWNUMBER),'') || '|' ||
        COALESCE(TO_VARCHAR(Matching_percentage),'') || '|' ||
        COALESCE(SAIL_ID1,'') || '|' ||
        COALESCE(MD5_VOYAGE_HASH_KEY1,'')
    ) AS HASH_DIFF,

    CURRENT_TIMESTAMP AS LDTS,
    'SAT_GCI_CONSUMPTION_RSS_PRICING_DASHBOARD' AS RCSR,
    CURRENT_USER AS LAST_MODIFIED_BY

FROM GCI_INTER_LFL_RSS;

--select * from REF_GCI_Inter_LFL_RSS where nclh_uni_voyage_code='RSS-SN-2027-06-20-SESTO-NLAMS-10' and nclh_uni_voyage_code1 ='SIL-DA-2023-08-22-SESTO-DKCPH-17'

-- alter table VESSOPS_D.L20_BDV.SAT_RMS_GCI_CONSUMPTION_RSS_LFL
-- rename to VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LFL

--CREATE OR REPLACE TABLE VESSOPS_D.L20_BDV.SAT_RMS_GCI_CONSUMPTION_RSS_LFL as 
TRUNCATE TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LFL;
INSERT INTO VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LFL 
select * from REF_GCI_Inter_LFL_RSS;

-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_RMS_GCI_CONSUMPTION_RSS_LFL
-- ADD CONSTRAINT PK_SAT_RMS_GCI_CONS_RSS_LFL
-- PRIMARY KEY (NCLH_UNI_VOYAGE_CODE, NCLH_UNI_VOYAGE_CODE1);


-- select top 10 * from GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS   where nclh_uni_voyage_code='HAL-NO-2025-07-27-USSEA-CAVAN-11'--GCI_CONSUMPTION_LFL_RSS_VOYAGES

-- SELECT *
-- FROM GCI_NCH_RAW_pricing_rg_pass
-- WHERE UPPER(cruiselinecode) = 'MSC'
--   AND UPPER(shipcode) = 'SH'
--   AND CAST(embarkationdate AS DATE) = '2022-04-23'
--   AND UPPER(TRIM(embarkationportun_locode)) = 'USMIA'
--   AND UPPER(TRIM(disembarkationportun_locode)) = 'USMIA'
--   AND cruiseduration = '7'; --SEATTLE, WA, USA@$07-27-2025@$#WHITTIER, AK, USA@$07-28-2025@$#WHITTIER, AK, USA@$07-29-2025@$#WHITTIER, AK, USA@$07-30-2025@$#WHITTIER, AK, USA@$07-31-2025@$#WHITTIER, AK, USA@$07-31-2025@$#SCENIC CRUISING HUBBARD GLACIER, AK, USA@$08-01-2025@$#GLACIER BAY@$08-02-2025@$#SKAGWAY, AK, USA@$08-03-2025@$#JUNEAU, AK, USA@$08-04-2025@$#KETCHIKAN, AK, USA@$08-05-2025@$#SCENIC CRUISING INSIDE PASSAGE, AK, USA@$08-06-2025@$#VANCOUVER, BC, CANADA@$08-07-2025
-- --MSC-SH-2022-04-23-USMIA-USMIA-7


-- Step: Create trends for RSS and Like-for-Like voyages
--select top 10 * from VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LFL

CREATE OR REPLACE TEMPORARY TABLE TEMP15 AS
SELECT 
    RSS_SAIL_ID,
    RSS_MD5_VOYAGE_HASH_KEY,
    UPPER(TRIM(NCLH_UNI_VOYAGE_CODE)) AS NCLH_UNI_VOYAGE_CODE,
    UPPER(TRIM(NCLH_UNI_VOYAGE_CODE1)) AS NCLH_UNI_VOYAGE_CODE1,
    RSS_COMP_SAIL_ID,
    N_COUNT,
    ROWNUMBER,
    Matching_percentage
FROM 
    VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_LFL;


-- Step 1: Create TEMP15 (already created before)
-- Assuming TEMP15 exists (from previous step)

-- Step 2: Create TEMP17 - Enrich Like-for-Like voyages with RSS voyage details

--select top 10 *  from TEMP17 where CATEGORYCODE_LFL is not null


CREATE OR REPLACE TEMP TABLE TEMP17 AS
SELECT
    CASE 
    WHEN LEFT(c.NCLH_UNI_VOYAGE_CODE, 3) = 'NCL' THEN 120
    WHEN LEFT(c.NCLH_UNI_VOYAGE_CODE, 3) = 'RSS' THEN 110
    WHEN LEFT(c.NCLH_UNI_VOYAGE_CODE, 3) IN ('OCE', 'OCI') THEN 100
    ELSE NULL
    END AS COMPANY_ID,

    c.RSS_SAIL_ID as SAIL_ID,
    c.NCLH_UNI_VOYAGE_CODE,
    c.RSS_COMP_SAIL_ID,
    '' AS BRAND_META_CATEGORY_DESC,
    c.N_COUNT,
    c.ROWNUMBER,
    c.Matching_percentage,

    d.NCLH_UNI_VOYAGE_CODE AS NCLH_UNI_VOYAGE_CODE_LFL,
    d.ADV_WEEK as ADV_WEEL_LFL,
    d.SAIL_ID AS COMP_SAILING_ID,
    d.Cruise_Line_Name AS CRUISELINENAME_LFL,
    d.Ship_Name_GCI AS SOURCESHIPNAME_LFL,
    d.CRUISE_LINE_CODE AS CRUISELINECODE_LFL,
    RIGHT(d.Ship_Code_GCI, 2) AS SHIPCODE_LFL,
    d.EmbarkationDate AS EMBARKATIONDATE_LFL,
    d.Embark_Port_Name AS EMBARKATIONPORT_LFL,
    d.Embark_Port_Code_GCI AS EMBARKATIONPORTUN_LOCODE_LFL,
    d.Disembark_Port_Name AS DISEMBARKATIONPORT_LFL,
    d.DisEmbark_Port_Code_GCI AS DISEMBARKATIONPORTUN_LOCODE_LFL,
    d.CruiseDuration AS CRUISEDURATION_LFL,
    d.GCI_Trade_Level_1 AS TRADE_HIERARCHY1_LFL,
    d.GCI_Trade_Level_2 AS TRADE_HIERARCHY2_LFL,
    d.GCI_Trade_Level_3 AS TRADE_HIERARCHY3_LFL,
    d.Ship_Code_GCI AS UNIQUE_SHIP_CODE_LFL,
    d.Adv_Week AS Adv_Week_LFL,
    d.CabinType AS CabinType_LFL,
    d.categorycode AS CategoryCode_LFL,   -- added new column
    d.RSS_CabinType AS RSS_CabinType_LFL,
    d."Obstructed(Y/N)" AS Obstructed,    -- added new column
    d."Market 1" AS Market1_LFL,
    d."Market 2" AS Market2_LFL,
    d.EMBARK_REGION_1 AS Embark_Region1_LFL,
    d.EMBARK_REGION_2 AS Embark_Region2_LFL,
    d.EMBARK_REGION_3 AS Embark_Region3_LFL,
    d.Regent_Cruise_CD AS Regent_Cruise_CD_LFL,
    d.TaxExclusiveRatePPD AS TaxExclusiveRatePPD_LFL,
    d.TaxInclusiveRatePPD AS TaxInclusiveRatePPD_LFL,
    d.TaxInclusiveRatePP AS TaxInclusiveRatePP_LFL,
    d.TaxExclusiveRatePP AS TaxExclusiveRatePP_LFL,
    d.TAXESANDFEES AS TaxesandFeesPP_LFL,
    d.TAXESANDFEESPPD AS TaxesandFeesPPD_LFL,
    d.DisEmbarkationDate AS DisembarkationDate_LFL,
    d.taxinclusive_regent_air_rate_pp AS taxinclusive_regent_air_rate_pp_LFL,
    d.taxinclusive_regent_air_rate_ppd AS taxinclusive_regent_air_rate_ppd_LFL,
    d.taxexclusive_regent_air_rate_pp AS taxexclusive_regent_air_rate_pp_LFL,
    d.taxexclusive_regent_air_rate_ppd AS taxexclusive_regent_air_rate_ppd_LFL,
    d.RateInquiryDate AS RateInquiryDate_LFL,
    d.shop_date AS shop_date_LFL,
    d.ItineraryPortContent AS ItineraryPortContent_LFL,
    d.CachePageLink AS cachepagelink_LFL,
    d.SOURCECURRENCY AS SOURCECURRENCY_LFL,
    d.SOURCEMARKET AS SOURCEMARKET_LFL,
        c.RSS_MD5_VOYAGE_HASH_KEY,
    '' AS HASH_DIFF,
    CURRENT_TIMESTAMP AS LDTS,
    'SAT_RMS_GCI_CONSUMPTION_RSS_LFL | SAT_RMS_GCI_CONSUMPTION_RSS_LFL' AS RCSR,
    CURRENT_USER AS LASTMODIFIED_BY

FROM TEMP15 c
LEFT JOIN VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD d
    ON UPPER(TRIM(c.NCLH_UNI_VOYAGE_CODE1)) = UPPER(TRIM(d.NCLH_UNI_VOYAGE_CODE));

-- select top 10 * from temp17
-- select count(*) from VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
-- select top 10 * from VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL
--CREATE OR REPLACE TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL as 
TRUNCATE TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL;
INSERT INTO VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL
select * from TEMP17;




-- -- create or replace table VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL as 

-- TRUNCATE TABLE VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL;

-- INSERT INTO VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL
-- SELECT
--     COMPANY_ID,
--     SAIL_ID AS NCLH_SAILING_ID,
--     NCLH_UNI_VOYAGE_CODE AS GCI_VOYAGE_CODE,
--     BRAND_META_CATEGORY_DESC,
--     CRUISELINENAME_LFL AS BRAND_COMPETITOR,
--     SOURCESHIPNAME_LFL AS SHIP,
--     RSS_COMP_SAIL_ID AS COMP_SAIL_ID,
--     NCLH_UNI_VOYAGE_CODE_LFL AS GCI_VOYAGE_CODE_COMPETITOR,
--     ADV_WEEK_LFL as ADVANCE_WEEKS_TO_SAIL,
--     CabinType_LFL AS ORIGINAL_META,
--     RSS_CabinType_LFL AS TRANSLATED_META,
--     TRADE_HIERARCHY1_LFL AS COMP_REGION_1,
--     TRADE_HIERARCHY2_LFL AS COMP_REGION_2,
--     TRADE_HIERARCHY3_LFL AS COMP_REGION_3,
--     CRUISEDURATION_LFL AS DURATION,
--     EMBARKATIONDATE_LFL AS DEPARTURE_DATE,
--     EMBARKATIONPORT_LFL AS EMBARK_PORT,
--     EMBARKATIONPORTUN_LOCODE_LFL AS EMBARK_PORT_UNLOCODE,
--     DisembarkationDate_LFL AS DISEMBARK_DATE,
--     DISEMBARKATIONPORT_LFL AS DISEMBARK_PORT,
--     DISEMBARKATIONPORTUN_LOCODE_LFL AS DISEMBARK_PORT_UNLOCODE,
--     TaxExclusiveRatePPD_LFL AS LEAD_TAX_EXCLUSIVE_PRICE_PPD,
--     TaxInclusiveRatePPD_LFL AS LEAD_TAX_INCLUSIVE_PRICE_PPD,
--     TaxExclusiveRatePP_LFL AS LEAD_TAX_EXCLUSIVE_RATE_PP,
--     TaxInclusiveRatePP_LFL AS LEAD_TAX_INCLUSIVE_RATE_PP,
--     TAXESANDFEESPPD_LFL AS LEAD_TAX_FEES_PPD,
--     TAXESANDFEESPP_LFL AS LEAD_TAX_FEES_PP,
--     SOURCECURRENCY_LFL AS CURRENCY,
--     RateInquiryDate_LFL AS SCRAPE_DATE,
--     CACHEPAGELINK_LFL AS CACHEPAGELINK,
--     ItineraryPortContent_LFL AS ITINERARY_PORT_CONTENT,
--     RSS_MD5_VOYAGE_HASH_KEY,

--     /* ✅ Always cast to VARCHAR before hashing */
--     MD5(
--         COALESCE(CAST(COMPANY_ID AS VARCHAR),'') || '|' ||
--         COALESCE(CAST(SAIL_ID AS VARCHAR),'') || '|' ||
--         COALESCE(NCLH_UNI_VOYAGE_CODE,'') || '|' ||
--         COALESCE(CRUISELINENAME_LFL,'') || '|' ||
--         COALESCE(SOURCESHIPNAME_LFL,'') || '|' ||
--         COALESCE(RSS_COMP_SAIL_ID,'') || '|' ||
--         COALESCE(NCLH_UNI_VOYAGE_CODE_LFL,'') || '|' ||
--         COALESCE(TRADE_HIERARCHY1_LFL,'') || '|' ||
--         COALESCE(TRADE_HIERARCHY2_LFL,'') || '|' ||
--         COALESCE(TRADE_HIERARCHY3_LFL,'') || '|' ||
--         COALESCE(CAST(CRUISEDURATION_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(EMBARKATIONPORT_LFL,'') || '|' ||
--         COALESCE(EMBARKATIONPORTUN_LOCODE_LFL,'') || '|' ||
--         COALESCE(DISEMBARKATIONPORT_LFL,'') || '|' ||
--         COALESCE(DISEMBARKATIONPORTUN_LOCODE_LFL,'') || '|' ||
--         COALESCE(CAST(TaxExclusiveRatePPD_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(CAST(TaxInclusiveRatePPD_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(CAST(TaxExclusiveRatePP_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(CAST(TaxInclusiveRatePP_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(CAST(TAXESANDFEESPPD_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(CAST(TAXESANDFEESPP_LFL AS VARCHAR),'') || '|' ||
--         COALESCE(SOURCECURRENCY_LFL,'') || '|' ||
--         COALESCE(ItineraryPortContent_LFL,'')
--     ) AS HASH_DIFF,

--     current_timestamp as LDTS,
--     RCSR,
--     LASTMODIFIED_BY
-- FROM VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL;


-- COMMENT ON TABLE VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL 
-- IS 'This table stores competitor pricing data for like-for-like analysis ';


-- ALTER TABLE VESSOPS_D.L30_ID.GCI_NCH_PRICING_LFL
-- ADD CONSTRAINT PK_GCI_NCH_PRICING_LFL
-- PRIMARY KEY (
--     GCI_VOYAGE_CODE,
--     GCI_VOYAGE_CODE_COMPETITOR,
--     ORIGINAL_META,
--     TRANSLATED_META,
--     SCRAPE_DATE,
--     CURRENCY
-- );

-- select top 10 * from VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL

-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_PRICING_LFL
-- ADD CONSTRAINT PK_SAT_GCI_RSS_CONSUMPTION_PRICING_LFL
-- PRIMARY KEY (
--     NCLH_UNI_VOYAGE_CODE,
--     NCLH_UNI_VOYAGE_CODE_LFL,
--     CABINTYPE_LFL,
--     CATEGORYCODE_LFL,
--     RSS_CABINTYPE_LFL,
--     RATEINQUIRYDATE_LFL,
--     TAXINCLUSIVERATEPP_LFL
-- );




--select top 10 * from VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_REF_PRICING_DASHBOARD where RSS_SAIL_ID is null 27 lakhs
-- Step 3: Create final target table GCI_CONSUMPTION.PRICING_DATA_LFL_RSS_COMPS

-- TRUNCATE TABLE GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS;
-- INSERT INTO VESSOPS_D.L20_BDV.SAT_GCI_CONSUMPTION_RSS_REF_PRICING_DASHBOARD
-- SELECT *
-- FROM TEMP17;

--select * from GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS where  rss_ship_code is null and cruise_line_code='RSS' and ship_name_gci='Seven Seas Prestige'

--select count(*) from GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS where rss_ship_code is null

-- Step 1: Create GCI_INTER.RSS_PRICING_DATA by copying data where CRUISE_LINE_CODE = 'RSS'

--select count(*) from GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS   where obstructed is null


--select distinct ship_name, ship_code from GCI_NCH_RAW_ref_ship_data where brand_folio='NCLH' and cruise_line_code='RSS'

CREATE OR REPLACE TEMP TABLE GCI_INTER_RSS_PRICING_DATA AS
SELECT *
FROM VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD
WHERE UPPER(TRIM(CRUISE_LINE_CODE)) = 'RSS';

--select top 10 * from GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS

--select top 10 * from gci_inter_ref_pp_ncl_dashboard

-- select  distinct * from GCI_CONSUMPTION_PRICING_DATA_LFL_RSS_COMPS 
-- where nclh_uni_voyage_code='RSS-SX-2026-09-30-USSEA-JPTYO-20' and nclh_uni_voyage_code_lfl='SEA-SO-2024-09-02-USSEA-JPYOK-22' and rateinquirydate_lfl='2024-07-28'  and cabintype_lfl='Suite' and rss_cabintype_lfl='Distinctive Suite'

-- Step 1: Create a temp table with distinct voyage and itinerary port content
CREATE OR REPLACE TEMP TABLE temp50 AS
SELECT DISTINCT 
    NCLH_UNI_VOYAGE_CODE, 
    ItineraryPortContent
FROM VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_REF_PRICING_DASHBOARD;

-- Step 2: Replace @$# with @ to clean up the itinerary string
CREATE OR REPLACE TEMP TABLE temp51 AS
SELECT *,
    REPLACE(ItineraryPortContent, '@$#', '@') AS ItinPort_ItinDate
FROM temp50;

-- Step 3: Replace @$ with + to separate port name and date
CREATE OR REPLACE TEMP TABLE temp52 AS
SELECT *,
    REPLACE(ItinPort_ItinDate, '@$', '+') AS port_content
FROM temp51;

-- Step 4: Split port_content by '@' and assign row numbers
CREATE OR REPLACE TEMP TABLE temp53 AS
SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE ORDER BY NCLH_UNI_VOYAGE_CODE) AS row_number,
    val.VALUE AS split_value
FROM temp52
, LATERAL FLATTEN(INPUT => SPLIT(port_content, '@')) val;

-- Step 5: Extract port location name and itinerary date
CREATE OR REPLACE TEMP TABLE temp54 AS
SELECT *,
    CASE 
        WHEN split_value LIKE '%+%' THEN LEFT(split_value, CHARINDEX('+', split_value) - 1)
        ELSE split_value
    END AS port_loc_name,
    CASE 
        WHEN split_value LIKE '%+%' THEN LTRIM(SUBSTR(split_value, CHARINDEX('+', split_value) + 1))
        ELSE '-'
    END AS itinerary_date
FROM temp53;

-- Step 6: Clean up port names by removing numbers
CREATE OR REPLACE TEMP TABLE temp55 AS
SELECT *,
    REGEXP_REPLACE(port_loc_name, '[0-9]', '') AS updated_port_loc_name
FROM temp54;

-- Step 7: Join with Ref_Merged_Port_data to get the unified port code
CREATE OR REPLACE TEMP TABLE temp56 AS
SELECT a.*, b.Tb_Unv_Port_Code
FROM temp55 a
LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_Ref_Merged_Port_data b
    ON UPPER(TRIM(a.updated_port_loc_name)) = UPPER(TRIM(b.Dtl_Port_City_Nam));

-- Step 8: Identify records where port code is still missing
CREATE OR REPLACE TEMP TABLE temp57 AS
SELECT *
FROM temp56
WHERE Tb_Unv_Port_Code IS NULL;

-- Step 9: Try mapping missing ports using Ref_Port_data
CREATE OR REPLACE TEMP TABLE temp58 AS
SELECT 
    a.NCLH_UNI_VOYAGE_CODE,
    a.ItineraryPortContent,
    a.ItinPort_ItinDate,
    a.port_content,
    a.updated_port_loc_name,
    a.split_value,
    a.port_loc_name,
    a.row_number,
    a.itinerary_date,
    b.unv_port_code AS tb_unv_port_code
FROM temp57 a
LEFT JOIN VESSOPS_D.l10_RDV.SAT_GCI_NCH_RAW_Ref_Port_data b
    ON UPPER(TRIM(a.updated_port_loc_name)) = UPPER(TRIM(b.port_loc_name));
    
-- Step 10: Build a small lookup table for mapped missing ports
CREATE OR REPLACE TEMP TABLE temp59 AS
SELECT DISTINCT 
    tb_unv_port_code, 
    updated_port_loc_name
FROM temp58;

-- Step 11: Update original data with the newly mapped ports
CREATE OR REPLACE TEMP TABLE temp60 AS
SELECT 
    a.*, 
    b.tb_unv_port_code AS updated_tb_unv_port_code
FROM temp56 a
LEFT JOIN temp59 b
    ON UPPER(TRIM(a.updated_port_loc_name)) = UPPER(TRIM(b.updated_port_loc_name));

-- Step 12: Create final output with the correct port code
CREATE OR REPLACE TEMP TABLE temp61 AS
SELECT *,
    CASE 
        WHEN Tb_Unv_Port_Code IS NOT NULL THEN Tb_Unv_Port_Code
        ELSE updated_tb_unv_port_code
    END AS final_tb_unv_port_code
FROM temp60;


-- Step 13: Add manual port code corrections for unmatched ports
CREATE OR REPLACE TEMP TABLE temp62 AS
SELECT *,
    CASE 
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'ISLAND, MYANMAR' THEN 'MMIS5'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'TOTAL ECLIPSE OBSERVATION' THEN 'AUTOT'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'BATTLE HARBOUR' THEN 'CAYCI'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'BELFAST, UK' THEN 'GBBEL'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'CENTRAL AZORES&THINSP;(SAO JORGE' THEN 'PTPIC'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'EASTFJORDS' THEN 'ISBER'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'EDGARTOWN, MARTHAS VINEYARD, MA UNITED STATES' THEN 'USOB2'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'FIJI (OVALAU' THEN 'FJLEV'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'FULAGA/FIJI' THEN 'FJFUL'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'GULF OF FINLAND:ALAND ISLANDS/FINLAND' THEN 'FIGUN'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'HONG KONG, CHINA' THEN 'HKHKG'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'ILE D&RSQUO;YEU' THEN 'FRILE'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'KING ISLAND, ALASKA' THEN 'USKIA'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'LEITH, UK' THEN 'GBLEI'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'LIVERPOOL, UK' THEN 'GBLIV'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'LOUISIADE ARCHIPELAGO' THEN 'PGLOI'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'NAIN' THEN 'CANAI'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'NEWFOUNDLAND' THEN 'CANOR'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'NORTHEAST GREENLAND NATIONAL PARK (KING OSCAR FJORD, ELLA ISLAND' THEN 'GLKON'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'NORTH-WEST COAST OF GREENLAND (CRUISING IN SMITH SOUND' THEN 'GLETA'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'RIGOLET' THEN 'CAYRG'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'ROSYTH, UK' THEN 'GBROY'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'ROWLEY SHOALS' THEN 'AURSM'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'SCENIC CRUISING BAHIA MAGDALENA, MEXICO' THEN 'MXCBM'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'SCENIC CRUISING EVIGHEDSFJORD, GREENLAND' THEN 'GLEVH'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'SCENIC CRUISING HORNSUND, SVALBARD & JAN MAYEN ISLANDS' THEN 'SJHOR'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'SCENIC CRUISING LOCH EWE, UK' THEN 'GBLCE'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'SÃO TOMÉ, SÃO TOMÉ AND PRÍNCIPE' THEN 'STTMS'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'ST. PAUL, UNITED STATES' THEN 'USSNP'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'SYDNEY' THEN 'AUSYD'
        WHEN UPPER(TRIM(updated_port_loc_name)) = 'TINNAKARA, LAKSHADWEEP ISLANDS INDIA' THEN 'INTHI'
        ELSE final_tb_unv_port_code
    END AS final1_tb_unv_port_code
FROM temp61;

-- Step 14: Create itinerary day (dense rank)
CREATE OR REPLACE TEMP TABLE temp63 AS
SELECT *, 
    DENSE_RANK() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE ORDER BY itinerary_date) AS itinerary_day
FROM temp62;

-- Step 15: Create sequence ID
CREATE OR REPLACE TEMP TABLE temp64 AS
SELECT *, 
    ROW_NUMBER() OVER (PARTITION BY NCLH_UNI_VOYAGE_CODE ORDER BY itinerary_date) AS sequence_id
FROM temp63;

-- Step 16: Select needed columns
CREATE OR REPLACE TEMP TABLE temp65 AS
SELECT 
    a.NCLH_UNI_VOYAGE_CODE,
    a.ItineraryPortContent,
    a.updated_port_loc_name AS port_loc_name,
    a.final_tb_unv_port_code AS port_unloc_code,
    a.itinerary_date,
    a.itinerary_day,
    a.sequence_id
FROM temp64 a;

-- Step 17: Final JOIN with Ref_Port_data using UPPER(TRIM()) for cleaner matching
CREATE OR REPLACE TEMP TABLE temp66 AS
SELECT 
    a.*,
    b.PORT_LAT,
    b.PORT_LONG,
    b.SCENIC_IND
FROM temp65 a
LEFT JOIN VESSOPS_D.l10_RDV.SAT_GCI_NCH_RAW_REF_PORT_DATA b
    ON UPPER(TRIM(a.port_unloc_code)) = UPPER(TRIM(b.UNV_PORT_CODE));

-- select top 0 * from temp66

alter table temp66
add column VESSEL_IMO VARCHAR(50),
    NCLH_SHIP_CODE VARCHAR(5),
    SAIL_ID varchar(50),
    MD5_VOYAGE_HASH_KEY VARCHAR,
    HASH_DIFF VARCHAR,
    LDTS TIMESTAMP_NTZ(9),
    RCSR VARCHAR(200),
    LAST_MODIFIED_BY VARCHAR(500);



UPDATE temp66 a
SET 
    VESSEL_IMO = b.VESSEL_IMO_NO,
    NCLH_SHIP_CODE = b.SHIP_CODE_3L
FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_SHIP_DATA b
WHERE CONCAT(LEFT(a.NCLH_UNI_VOYAGE_CODE, 3), SPLIT_PART(a.NCLH_UNI_VOYAGE_CODE, '-', 2)) = b."5L_SHIP_CODE";



UPDATE temp66
SET SAIL_ID = CONCAT(
    NCLH_SHIP_CODE,
    TO_CHAR(
        TO_DATE(
            SPLIT_PART(NCLH_UNI_VOYAGE_CODE, '-', 3) || '-' ||
            SPLIT_PART(NCLH_UNI_VOYAGE_CODE, '-', 4) || '-' ||
            SPLIT_PART(NCLH_UNI_VOYAGE_CODE, '-', 5),
            'YYYY-MM-DD'
        ),
        'YYMMDD'
    )
);


UPDATE temp66
SET MD5_VOYAGE_HASH_KEY = MD5(SAIL_ID);

UPDATE temp66
SET 
    LDTS = CURRENT_TIMESTAMP(),
    RCSR = 'SAT_GCI_RSS_CONSUMPTIONPRICING_DASHBOARD',
    LAST_MODIFIED_BY = CURRENT_USER();


UPDATE temp66
SET HASH_DIFF = MD5(
        COALESCE(NCLH_UNI_VOYAGE_CODE, '') || '|' ||
        COALESCE(ITINERARYPORTCONTENT, '') || '|' ||
        COALESCE(PORT_LOC_NAME, '') || '|' ||
        COALESCE(PORT_UNLOC_CODE, '') || '|' ||
        --COALESCE(TO_CHAR(ITINERARY_DATE, 'YYYY-MM-DD'), '') || '|' ||
        COALESCE(ITINERARY_DAY::STRING, '') || '|' ||
        COALESCE(SEQUENCE_ID::STRING, '') || '|' ||
        COALESCE(PORT_LAT::STRING, '') || '|' ||
        COALESCE(PORT_LONG::STRING, '') || '|' ||
        COALESCE(SCENIC_IND, '')
    );
    

--CREATE OR REPLACE table VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL as 
TRUNCATE TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL;
insert into VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
select * from temp66;


-- ALTER TABLE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
-- ADD CONSTRAINT PK_SAT_GCI_RSS_CONS_VOYDAY
-- PRIMARY KEY (NCLH_UNI_VOYAGE_CODE, PORT_UNLOC_CODE, ITINERARY_DATE, SEQUENCE_ID);


--select top 10 * from temp66 where nclh_uni_voyage_code='SEA-SO-2024-09-06-CAVAN-CAVAN-10' and port_unLOC_CODE , itineray_date sequence_id

-- -- Step 18: Save to final destination table
-- TRUNCATE TABLE GCI_CONSUMPTION_PRICING_VOYAGE_DAY_LEVEL_RSS;
-- INSERT INTO GCI_CONSUMPTION_PRICING_VOYAGE_DAY_LEVEL_RSS
-- SELECT *
-- FROM temp66;


--select * from GCI_CONSUMPTION_PRICING_VOYAGE_DAY_LEVEL_RSS where nclh_uni_voyage_code='CRY-CR-2025-04-20-GRPIR-ESBCN-20'

-- Update for specific scenic locations
UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('CHILE&RSQUO;S NORTHERN FJORDS (CHAITEN, CRUISING THROUGH THE NORTHERN FJORDS'));

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('RELAXATION AT SEA/CRUISING IN THE SCOTTISH ISLANDS'));

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('ARSUK FJORD (SCENIC CRUISING), GREENLAND'));

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('KARRAT FJORD (SCENIC CRUISING), GREENLAND'));

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('CRUISE NORWEGIAN FJORDS'));

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('CRUISE ON THE THAMES TO TOWER BRIDGELONDON/GREAT BRITAIN**'));

-- Update for multiple scenic locations in a list
UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'Y'
WHERE UPPER(TRIM(port_loc_name)) IN (
    UPPER(TRIM('CRUISING BY OLD MAN OF HOY, ORKNEYS')),
    UPPER(TRIM('CRUISING THE INDIAN OCEAN')),
    UPPER(TRIM('CRUISING THE COAST OF BRAZIL')),
    UPPER(TRIM('CRUISING THE GULF OF SUEZ')),
    UPPER(TRIM('CRUISING THE TYRRHENIAN, IONIAN & ADRIATIC SE')),
    UPPER(TRIM('CRUISING: GULF OF MEXICO')),
    UPPER(TRIM('CRUISING AMUNDSEN TROUGH CANADA')),
    UPPER(TRIM('NORTH-WEST COAST OF GREENLAND (CRUISING IN SMITH SOUND')),
    UPPER(TRIM('CRUISING HUDSON RIVER, USA')),
    UPPER(TRIM('CRUISING THE BAHAMIAN WATERS')),
    UPPER(TRIM('CRUISING THE GULF OF ADEN')),
    UPPER(TRIM('CRUISING THE PACIFIC OCEAN')),
    UPPER(TRIM('CRUISING THE SOUTH ATLANTIC OCEAN')),
    UPPER(TRIM('CRUISING THE THAMES RIVER')),
    UPPER(TRIM('PASSING THE GOLDEN GATE BRIDGE')),
    UPPER(TRIM('SAILING ICE FLOES')),
    UPPER(TRIM('SAILING TOWARD THE ICE FLOES')),
    UPPER(TRIM('AT SEA')),
    UPPER(TRIM('AT SEA ALONG SPITSBERGEN')),
    UPPER(TRIM('EXPEDITION NORTHEAST GREENLAND GREENLAND')),
    UPPER(TRIM('GEOGRAPHIC NORTH POLE')),
    UPPER(TRIM('IN PORT')),
    UPPER(TRIM('MATAKUS ISLAND, INDONESIA')),
    UPPER(TRIM('ROSS SEA, ANTARCTICA')),
    UPPER(TRIM('SEA THE ARAFURA SEA')),
    UPPER(TRIM('SEA THE CARIBBEAN SEA')),
    UPPER(TRIM('SEA THE EAST CN SEA')),
    UPPER(TRIM('SEA THE GULF OF THAILAND')),
    UPPER(TRIM('SEA THE PHILIPPINE SEA')),
    UPPER(TRIM('SEA THE SOUTH CN SEA')),
    UPPER(TRIM('SEA THE SOUTH PACIFIC5')),
    UPPER(TRIM('SEA THE BAY OF BENGAL')),
    UPPER(TRIM('SEA THE LACCADIVE SEA')),
    UPPER(TRIM('SEA THE TASMAN SEA')),
    UPPER(TRIM('SOUND, NEW ZEALAND')),
    UPPER(TRIM('SOUTH OF THE POLAR CIRCLE ANTARCTICA')),
    UPPER(TRIM('TO BE DETERMINED, TO BE DETERMINED')),
    UPPER(TRIM('TRANSIT THE ENGLISH NARROWS')),
    UPPER(TRIM('PROLIV SENYAVINA (HOT SPRINGS), RUSSIA')),
    UPPER(TRIM('SEA THE NORTH ATLANTIC')),
    UPPER(TRIM('IN PORT,')),
    UPPER(TRIM('SEA THE SOUTH PACIFIC')),
    UPPER(TRIM('TOTAL ECLIPSE OBSERVATION'))
);

-- Update for non-scenic locations
UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'N'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('ABROLHOS ARCHIPELAGO, BRAZIL'));

UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'N'
WHERE UPPER(TRIM(port_loc_name)) = UPPER(TRIM('SAN REMO, ITALY'));

-- Update for null SCENIC_IND to 'N'
UPDATE VESSOPS_D.L20_BDV.SAT_GCI_RSS_CONSUMPTION_VOYAGE_DAY_LEVEL
SET scenic_ind = 'N'
WHERE SCENIC_IND IS NULL;
